In [1]:
import pandas as pd
import configparser
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization

with open("/etc/security/snowflake/rsa_plbiap01dy.p8", "rb") as key:
    p_key= serialization.load_pem_private_key(
        key.read(),
        password='snowflake'.encode(),
        backend=default_backend()
    )

pkb = p_key.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption())


ctx = snowflake.connector.connect(
    user='plbiap01dy',
    account='hfsg_prod.us-east-1.privatelink',
    private_key=pkb,
    warehouse='DSC_PLBI_DEV_MFG_WHS',
    database='DSC_PLBI_DB',
    schema='APP_AUTO_DEV',
    role='plbiap01dy_dev_pii_role'
    )

## Function Declaration for APP_AUTO_DEV

In [2]:
## Used for Query #6 & #7
rc1_funnel = """
-- Used for RC1
CREATE OR REPLACE FUNCTION DSC_PLBI_DB.APP_AUTO_DEV.rc1_funnel_scorecard(month_diff INT, QUOTE_DT DATE, QCN STRING)
RETURNS NUMBER(13, 0)
LANGUAGE SQL
AS
$$
    SUM(CASE WHEN DATEDIFF(MONTH, QUOTE_DT, SYSDATE()) = month_diff AND LENGTH(QCN) > 5 THEN 1 ELSE 0 END)
$$;
"""

adv_ind_funnel = """
-- Used for Success 2+
CREATE OR REPLACE FUNCTION DSC_PLBI_DB.APP_AUTO_DEV.adv_ind_funnel_scorecard(month_diff INT, QUOTE_DT DATE, SUCCESS_IND INT, CARRIERSQUOTED INT)
RETURNS NUMBER(13, 0)
LANGUAGE SQL
AS
$$
    SUM(CASE WHEN DATEDIFF(MONTH, QUOTE_DT, SYSDATE()) = month_diff AND CARRIERSQUOTED > 2 THEN ROUND(SUCCESS_IND, 0) ELSE 0 END)
$$;
"""

ind_funnel = """
-- Used for Failure, Success, Competitive, Win, Completed Quote, Issue, AND Bridge
CREATE OR REPLACE FUNCTION DSC_PLBI_DB.APP_AUTO_DEV.ind_funnel_scorecard(month_diff INT, QUOTE_DT DATE, IND INT)
RETURNS NUMBER(13, 0)
LANGUAGE SQL
AS
$$
    SUM(CASE WHEN DATEDIFF(MONTH, QUOTE_DT, SYSDATE()) = month_diff THEN ROUND(IND, 0) ELSE 0 END)
$$;
"""

region_funnel = """
CREATE OR REPLACE FUNCTION DSC_PLBI_DB.APP_AUTO_DEV.get_region_scorecard(STATE STRING)
RETURNS STRING
LANGUAGE SQL
AS
$$
    CASE WHEN STATE IN ('MS','TN','AL','AR','FL','IN','KY','NC','WV','GA','SC','DC','LA') THEN 'Southeast' 
         WHEN STATE IN ('IA','KS','NE','OK','TX','MO','IL','MI','MN','WI','NM','CO') THEN 'Central'
         WHEN STATE IN ('DE','ME','NH','RI','VT','NY','NJ','CT','MA','PA','OH','VA','MD') THEN 'Northeast'
         WHEN STATE IN ('AK','HI','MT','ND','SD','WY','CA','AZ','NV','WA','OR','ID','UT') THEN 'West' 
    ELSE 'Unknown' END
$$;
"""

div_ind_funnel = """
CREATE OR REPLACE FUNCTION DSC_PLBI_DB.APP_AUTO_DEV.div_ind_funnel_scorecard(month_diff INT, QUOTE_DT DATE, SUCCESS_IND INT, CARRIERSQUOTED INT, XHIG NUMBER, ANNUAL NUMBER)
RETURNS NUMBER(13, 0)
LANGUAGE SQL
AS
$$
    CASE WHEN DATEDIFF(MONTH, QUOTE_DT, SYSDATE()) = month_diff AND SUCCESS_IND = 1 AND CARRIERSQUOTED > 2
    THEN NULLIFZERO(ANNUAL/XHIG) ELSE NULL END
$$;
"""

## Used for Query #5
one_cond = """
CREATE OR REPLACE FUNCTION DSC_PLBI_DB.APP_AUTO_DEV.one_cond_scorecard(month_diff INT, QUOTE_DT DATE, IND STRING, VAL STRING, RETURN_NUM NUMBER, RVSE BOOLEAN DEFAULT FALSE)
RETURNS NUMBER(13, 0)
LANGUAGE SQL
AS
$$ 
    CASE WHEN RVSE THEN
        SUM(CASE WHEN DATEDIFF(MONTH, QUOTE_DT, SYSDATE()) = month_diff AND IND <> VAL THEN RETURN_NUM ELSE 0 END)
    ELSE
        SUM(CASE WHEN DATEDIFF(MONTH, QUOTE_DT, SYSDATE()) = month_diff AND IND = VAL THEN RETURN_NUM ELSE 0 END)
    END
$$;
"""

two_cond = """
CREATE OR REPLACE FUNCTION DSC_PLBI_DB.APP_AUTO_DEV.two_cond_scorecard(month_diff INT, QUOTE_DT DATE, IND STRING, VAL STRING, IND2 STRING, VAL2 STRING, RETURN_NUM NUMBER, RVSE BOOLEAN DEFAULT FALSE)
RETURNS NUMBER(13, 0)
LANGUAGE SQL
AS
$$
    CASE WHEN RVSE THEN
        SUM(CASE WHEN DATEDIFF(MONTH, QUOTE_DT, SYSDATE()) = month_diff AND IND <> VAL AND IND2 = VAL2 THEN RETURN_NUM ELSE 0 END)
    ELSE
        SUM(CASE WHEN DATEDIFF(MONTH, QUOTE_DT, SYSDATE()) = month_diff AND IND = VAL AND IND2 = VAL2 THEN RETURN_NUM ELSE 0 END)
    END
$$;
"""

# Used for Query #4
zero_cond = """
CREATE OR REPLACE FUNCTION DSC_PLBI_DB.APP_AUTO_DEV.zero_cond_scorecard(month_diff INT, QUOTE_DT DATE, RETURN_NUM NUMBER)
RETURNS NUMBER(13, 0)
LANGUAGE SQL
AS
$$ 
    SUM(CASE WHEN DATEDIFF(MONTH, QUOTE_DT, SYSDATE()) = month_diff THEN RETURN_NUM ELSE 0 END) 
$$;
"""

## Used for Query #3
perst_calc = """
CREATE OR REPLACE FUNCTION DSC_PLBI_DB.APP_AUTO_DEV.perst_calc_scorecard(monthdiff INT, POL_EFF_DT DATE, CANCEL INT, ATR INT)
RETURNS NUMBER(13, 0)
LANGUAGE SQL
AS
$$
    1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = monthdiff THEN CANCEL ELSE 0 END), 
                 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = monthdiff THEN ATR ELSE 0 END))
$$;
"""

## Function Creation
cs = ctx.cursor()
try:
    cs.execute(rc1_funnel)
    cs.execute(adv_ind_funnel)
    cs.execute(ind_funnel)
    cs.execute(region_funnel)
    cs.execute(div_ind_funnel)
    cs.execute(one_cond)
    cs.execute(two_cond)
    cs.execute(zero_cond)
    cs.execute(perst_calc)
finally:
    cs.close()
print('Done')

Done


## 1. Auto Flow Table Creation (DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_AUTO_FLOW)

In [3]:
auto_flow = """
CREATE OR REPLACE TRANSIENT TABLE DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_AUTO_FLOW AS

WITH state_aq_response AS -- State Response Count for Auto Quote
(
	SELECT CONCAT('Response','NewCo',RISK_ST_ABBR,BU_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, TRANS_DT, RESP_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, TRANS_DT, RESP_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, TRANS_DT, RESP_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, TRANS_DT, RESP_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, TRANS_DT, RESP_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, TRANS_DT, RESP_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, TRANS_DT, RESP_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, TRANS_DT, RESP_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, TRANS_DT, RESP_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, TRANS_DT, RESP_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, TRANS_DT, RESP_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, TRANS_DT, RESP_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, TRANS_DT, RESP_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, TRANS_DT, RESP_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, TRANS_DT, RESP_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, TRANS_DT, RESP_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, TRANS_DT, RESP_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, TRANS_DT, RESP_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, TRANS_DT, RESP_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN RESP_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	GROUP BY Key
),

region_aq_response AS -- Regional Response Count for Auto Quote
(
	SELECT CONCAT('Response','NewCo',GET_REGION_SCORECARD(RISK_ST_ABBR),BU_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, TRANS_DT, RESP_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, TRANS_DT, RESP_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, TRANS_DT, RESP_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, TRANS_DT, RESP_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, TRANS_DT, RESP_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, TRANS_DT, RESP_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, TRANS_DT, RESP_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, TRANS_DT, RESP_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, TRANS_DT, RESP_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, TRANS_DT, RESP_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, TRANS_DT, RESP_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, TRANS_DT, RESP_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, TRANS_DT, RESP_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, TRANS_DT, RESP_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, TRANS_DT, RESP_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, TRANS_DT, RESP_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, TRANS_DT, RESP_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, TRANS_DT, RESP_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, TRANS_DT, RESP_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN RESP_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	GROUP BY Key
),

cw_aq_response AS -- CW Response Count for Auto Quote
(
	SELECT CONCAT('Response','NewCo','CW',BU_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, TRANS_DT, RESP_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, TRANS_DT, RESP_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, TRANS_DT, RESP_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, TRANS_DT, RESP_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, TRANS_DT, RESP_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, TRANS_DT, RESP_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, TRANS_DT, RESP_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, TRANS_DT, RESP_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, TRANS_DT, RESP_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, TRANS_DT, RESP_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, TRANS_DT, RESP_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, TRANS_DT, RESP_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, TRANS_DT, RESP_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, TRANS_DT, RESP_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, TRANS_DT, RESP_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, TRANS_DT, RESP_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, TRANS_DT, RESP_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, TRANS_DT, RESP_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, TRANS_DT, RESP_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN RESP_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	GROUP BY Key
),

state_scdi_response AS -- State Response Count for Auto SCDI
(
	SELECT CONCAT('Response','NowCo',RISK_STATE_ABBR,BUS_UNIT_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN RESPONSE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT'
	GROUP BY Key
),

region_scdi_response AS -- Regional Response Count for Auto SCDI
(
	SELECT CONCAT('Response','NowCo',GET_REGION_SCORECARD(RISK_STATE_ABBR),BUS_UNIT_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN RESPONSE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT'
	GROUP BY Key
),

cw_scdi_response AS -- CW Response Count for Auto SCDI
(
	SELECT CONCAT('Response','NowCo','CW',BUS_UNIT_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN RESPONSE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT'
	GROUP BY Key
),

state_aq_phone AS -- State Phone Response Count for Auto Quote
(
	SELECT CONCAT('Phone Response','NewCo',RISK_ST_ABBR,BU_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, TRANS_DT, RESP_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, TRANS_DT, RESP_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, TRANS_DT, RESP_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, TRANS_DT, RESP_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, TRANS_DT, RESP_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, TRANS_DT, RESP_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, TRANS_DT, RESP_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, TRANS_DT, RESP_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, TRANS_DT, RESP_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, TRANS_DT, RESP_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, TRANS_DT, RESP_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, TRANS_DT, RESP_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, TRANS_DT, RESP_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, TRANS_DT, RESP_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, TRANS_DT, RESP_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, TRANS_DT, RESP_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, TRANS_DT, RESP_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, TRANS_DT, RESP_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, TRANS_DT, RESP_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN RESP_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE ORIG_QTE_CNTCT_METH_CD = 'P'
	GROUP BY Key
),

region_aq_phone AS -- Regional Phone Response Count for Auto Quote
(
	SELECT CONCAT('Phone Response','NewCo',GET_REGION_SCORECARD(RISK_ST_ABBR),BU_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, TRANS_DT, RESP_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, TRANS_DT, RESP_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, TRANS_DT, RESP_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, TRANS_DT, RESP_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, TRANS_DT, RESP_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, TRANS_DT, RESP_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, TRANS_DT, RESP_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, TRANS_DT, RESP_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, TRANS_DT, RESP_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, TRANS_DT, RESP_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, TRANS_DT, RESP_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, TRANS_DT, RESP_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, TRANS_DT, RESP_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, TRANS_DT, RESP_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, TRANS_DT, RESP_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, TRANS_DT, RESP_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, TRANS_DT, RESP_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, TRANS_DT, RESP_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, TRANS_DT, RESP_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN RESP_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE ORIG_QTE_CNTCT_METH_CD = 'P'
	GROUP BY Key
),

cw_aq_phone AS -- CW Phone Response Count for Auto Quote
(
	SELECT CONCAT('Phone Response','NewCo','CW',BU_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, TRANS_DT, RESP_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, TRANS_DT, RESP_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, TRANS_DT, RESP_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, TRANS_DT, RESP_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, TRANS_DT, RESP_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, TRANS_DT, RESP_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, TRANS_DT, RESP_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, TRANS_DT, RESP_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, TRANS_DT, RESP_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, TRANS_DT, RESP_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, TRANS_DT, RESP_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, TRANS_DT, RESP_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, TRANS_DT, RESP_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, TRANS_DT, RESP_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, TRANS_DT, RESP_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, TRANS_DT, RESP_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, TRANS_DT, RESP_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, TRANS_DT, RESP_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, TRANS_DT, RESP_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN RESP_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE ORIG_QTE_CNTCT_METH_CD = 'P'
	GROUP BY Key
),

state_scdi_phone AS -- State Phone Response Count for Auto SCDI
(
	SELECT CONCAT('Phone Response','NowCo',RISK_STATE_ABBR,BUS_UNIT_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN RESPONSE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND ORIG_QUOTE_PROCESS_METH_CD = 'T'
	GROUP BY Key
),

region_scdi_phone AS -- Regional Phone Response Count for Auto SCDI
(
	SELECT CONCAT('Phone Response','NowCo',GET_REGION_SCORECARD(RISK_STATE_ABBR),BUS_UNIT_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN RESPONSE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND ORIG_QUOTE_PROCESS_METH_CD = 'T'
	GROUP BY Key
),

cw_scdi_phone AS -- CW Phone Response Count for Auto SCDI
(
	SELECT CONCAT('Phone Response','NowCo','CW',BUS_UNIT_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, RESPONSE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN RESPONSE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND ORIG_QUOTE_PROCESS_METH_CD = 'T'
	GROUP BY Key
),

state_aq_phone_cq AS -- State Phone Completed Quote Count for Auto Quote
(
	SELECT CONCAT('Phone Completed Quote','NewCo',RISK_ST_ABBR,BU_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN COMPL_QTE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE ORIG_QTE_CNTCT_METH_CD = 'P'
	GROUP BY Key
),

region_aq_phone_cq AS -- Regional Phone Completed Quote Count for Auto Quote
(
	SELECT CONCAT('Phone Completed Quote','NewCo',GET_REGION_SCORECARD(RISK_ST_ABBR),BU_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN COMPL_QTE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE ORIG_QTE_CNTCT_METH_CD = 'P'
	GROUP BY Key
),

cw_aq_phone_cq AS -- CW Phone Completed Quote Count for Auto Quote
(
	SELECT CONCAT('Phone Completed Quote','NewCo','CW',BU_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, TRANS_DT, COMPL_QTE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN COMPL_QTE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE ORIG_QTE_CNTCT_METH_CD = 'P'
	GROUP BY Key
),

state_scdi_phone_cq AS -- State Phone Completed Quote Count for Auto SCDI
(
	SELECT CONCAT('Phone Completed Quote','NowCo',RISK_STATE_ABBR,BUS_UNIT_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN COMPL_QUOTE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND ORIG_QUOTE_PROCESS_METH_CD = 'T'
	GROUP BY Key
),

region_scdi_phone_cq AS -- Regional Phone Completed Quote Count for Auto SCDI
(
	SELECT CONCAT('Phone Completed Quote','NowCo',GET_REGION_SCORECARD(RISK_STATE_ABBR),BUS_UNIT_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN COMPL_QUOTE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND ORIG_QUOTE_PROCESS_METH_CD = 'T'
	GROUP BY Key
),

cw_scdi_phone_cq AS -- CW Phone Completed Quote Count for Auto SCDI
(
	SELECT CONCAT('Phone Completed Quote','NowCo','CW',BUS_UNIT_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN COMPL_QUOTE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND ORIG_QUOTE_PROCESS_METH_CD = 'T'
	GROUP BY Key
),

state_aq_issue AS -- State Issue Count for Auto Quote
(
	SELECT CONCAT('Issue','NewCo',RISK_ST_ABBR,BU_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, TRANS_DT, ISSUE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, TRANS_DT, ISSUE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, TRANS_DT, ISSUE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, TRANS_DT, ISSUE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, TRANS_DT, ISSUE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, TRANS_DT, ISSUE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, TRANS_DT, ISSUE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, TRANS_DT, ISSUE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, TRANS_DT, ISSUE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, TRANS_DT, ISSUE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, TRANS_DT, ISSUE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, TRANS_DT, ISSUE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, TRANS_DT, ISSUE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, TRANS_DT, ISSUE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, TRANS_DT, ISSUE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, TRANS_DT, ISSUE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, TRANS_DT, ISSUE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, TRANS_DT, ISSUE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, TRANS_DT, ISSUE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN ISSUE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	GROUP BY Key
),

region_aq_issue AS -- Regional Issue Count for Auto Quote
(
	SELECT CONCAT('Issue','NewCo',GET_REGION_SCORECARD(RISK_ST_ABBR),BU_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, TRANS_DT, ISSUE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, TRANS_DT, ISSUE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, TRANS_DT, ISSUE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, TRANS_DT, ISSUE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, TRANS_DT, ISSUE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, TRANS_DT, ISSUE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, TRANS_DT, ISSUE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, TRANS_DT, ISSUE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, TRANS_DT, ISSUE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, TRANS_DT, ISSUE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, TRANS_DT, ISSUE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, TRANS_DT, ISSUE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, TRANS_DT, ISSUE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, TRANS_DT, ISSUE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, TRANS_DT, ISSUE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, TRANS_DT, ISSUE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, TRANS_DT, ISSUE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, TRANS_DT, ISSUE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, TRANS_DT, ISSUE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN ISSUE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	GROUP BY Key
),

cw_aq_issue AS -- CW Issue Count for Auto Quote
(
	SELECT CONCAT('Issue','NewCo','CW',BU_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, TRANS_DT, ISSUE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, TRANS_DT, ISSUE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, TRANS_DT, ISSUE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, TRANS_DT, ISSUE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, TRANS_DT, ISSUE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, TRANS_DT, ISSUE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, TRANS_DT, ISSUE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, TRANS_DT, ISSUE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, TRANS_DT, ISSUE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, TRANS_DT, ISSUE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, TRANS_DT, ISSUE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, TRANS_DT, ISSUE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, TRANS_DT, ISSUE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, TRANS_DT, ISSUE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, TRANS_DT, ISSUE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, TRANS_DT, ISSUE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, TRANS_DT, ISSUE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, TRANS_DT, ISSUE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, TRANS_DT, ISSUE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN ISSUE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	GROUP BY Key
),

state_scdi_issue AS -- State Issue Count for Auto SCDI
(
	SELECT CONCAT('Issue','NowCo',RISK_STATE_ABBR,BUS_UNIT_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN ISSUE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT'
	GROUP BY Key
),

region_scdi_issue AS -- Regional Issue Count for Auto SCDI
(
	SELECT CONCAT('Issue','NowCo',GET_REGION_SCORECARD(RISK_STATE_ABBR),BUS_UNIT_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN ISSUE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT'
	GROUP BY Key
),

cw_scdi_issue AS -- CW Issue Count for Auto SCDI
(
	SELECT CONCAT('Issue','NowCo','CW',BUS_UNIT_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN ISSUE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT'
	GROUP BY Key
),

state_aq_phone_issue AS -- State Phone Issue Count for Auto Quote
(
	SELECT CONCAT('Phone Issue','NewCo',RISK_ST_ABBR,BU_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, TRANS_DT, ISSUE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, TRANS_DT, ISSUE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, TRANS_DT, ISSUE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, TRANS_DT, ISSUE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, TRANS_DT, ISSUE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, TRANS_DT, ISSUE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, TRANS_DT, ISSUE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, TRANS_DT, ISSUE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, TRANS_DT, ISSUE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, TRANS_DT, ISSUE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, TRANS_DT, ISSUE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, TRANS_DT, ISSUE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, TRANS_DT, ISSUE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, TRANS_DT, ISSUE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, TRANS_DT, ISSUE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, TRANS_DT, ISSUE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, TRANS_DT, ISSUE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, TRANS_DT, ISSUE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, TRANS_DT, ISSUE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN ISSUE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE ORIG_QTE_CNTCT_METH_CD = 'P'
	GROUP BY Key
),

region_aq_phone_issue AS -- Regional Phone Issue Count for Auto Quote
(
	SELECT CONCAT('Phone Issue','NewCo',GET_REGION_SCORECARD(RISK_ST_ABBR),BU_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, TRANS_DT, ISSUE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, TRANS_DT, ISSUE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, TRANS_DT, ISSUE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, TRANS_DT, ISSUE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, TRANS_DT, ISSUE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, TRANS_DT, ISSUE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, TRANS_DT, ISSUE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, TRANS_DT, ISSUE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, TRANS_DT, ISSUE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, TRANS_DT, ISSUE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, TRANS_DT, ISSUE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, TRANS_DT, ISSUE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, TRANS_DT, ISSUE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, TRANS_DT, ISSUE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, TRANS_DT, ISSUE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, TRANS_DT, ISSUE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, TRANS_DT, ISSUE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, TRANS_DT, ISSUE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, TRANS_DT, ISSUE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN ISSUE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE ORIG_QTE_CNTCT_METH_CD = 'P'
	GROUP BY Key
),

cw_aq_phone_issue AS -- CW Phone Issue Count for Auto Quote
(
	SELECT CONCAT('Phone Issue','NewCo','CW',BU_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, TRANS_DT, ISSUE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, TRANS_DT, ISSUE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, TRANS_DT, ISSUE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, TRANS_DT, ISSUE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, TRANS_DT, ISSUE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, TRANS_DT, ISSUE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, TRANS_DT, ISSUE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, TRANS_DT, ISSUE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, TRANS_DT, ISSUE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, TRANS_DT, ISSUE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, TRANS_DT, ISSUE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, TRANS_DT, ISSUE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, TRANS_DT, ISSUE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, TRANS_DT, ISSUE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, TRANS_DT, ISSUE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, TRANS_DT, ISSUE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, TRANS_DT, ISSUE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, TRANS_DT, ISSUE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, TRANS_DT, ISSUE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN ISSUE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE ORIG_QTE_CNTCT_METH_CD = 'P'
	GROUP BY Key
),

state_scdi_phone_issue AS -- State Phone Issue Count for Auto SCDI
(
	SELECT CONCAT('Phone Issue','NowCo',RISK_STATE_ABBR,BUS_UNIT_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN ISSUE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND ORIG_QUOTE_PROCESS_METH_CD = 'T'
	GROUP BY Key
),

region_scdi_phone_issue AS -- Regional Phone Issue Count for Auto SCDI
(
	SELECT CONCAT('Phone Issue','NowCo',GET_REGION_SCORECARD(RISK_STATE_ABBR),BUS_UNIT_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN ISSUE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND ORIG_QUOTE_PROCESS_METH_CD = 'T'
	GROUP BY Key
),

cw_scdi_phone_issue AS -- CW Phone Issue Count for Auto SCDI
(
	SELECT CONCAT('Phone Issue','NowCo','CW',BUS_UNIT_ABBR) as Key,
	TRUNC(zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "18",
	TRUNC(zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "17",
	TRUNC(zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "16",
	TRUNC(zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "15",
	TRUNC(zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "14",
	TRUNC(zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "13",
	TRUNC(zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "12",
	TRUNC(zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "11",
	TRUNC(zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "10",
	TRUNC(zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "9",
	TRUNC(zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "8",
	TRUNC(zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "7",
	TRUNC(zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "6",
	TRUNC(zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "5",
	TRUNC(zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "4",
	TRUNC(zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "3",
	TRUNC(zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "2",
	TRUNC(zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "1",
	TRUNC(zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT), 0) AS "0",
	TRUNC(SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN ISSUE_IND_CNT ELSE 0 END), 0) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND ORIG_QUOTE_PROCESS_METH_CD = 'T'
	GROUP BY Key
),

state_aq_aip AS -- State AIP for Auto Quote
(
	SELECT CONCAT('AIP','NewCo',RISK_ST_ABBR,BU_ABBR) as Key,
	zero_cond_scorecard(18, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "18",
	zero_cond_scorecard(17, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "17",
	zero_cond_scorecard(16, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "16",
	zero_cond_scorecard(15, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "15",
	zero_cond_scorecard(14, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "14",
	zero_cond_scorecard(13, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "13",
	zero_cond_scorecard(12, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "12",
	zero_cond_scorecard(11, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "11",
	zero_cond_scorecard(10, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "10",
	zero_cond_scorecard(9, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "9",
	zero_cond_scorecard(8, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "8",
	zero_cond_scorecard(7, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "7",
	zero_cond_scorecard(6, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "6",
	zero_cond_scorecard(5, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "5",
	zero_cond_scorecard(4, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "4",
	zero_cond_scorecard(3, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "3",
	zero_cond_scorecard(2, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "2",
	zero_cond_scorecard(1, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "1",
	zero_cond_scorecard(0, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN POL_TTL_BILL_PREM_AMT * 2 ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE ISSUE_IND = 'Y'
	GROUP BY Key
),

region_aq_aip AS -- Regional AIP for Auto Quote
(
	SELECT CONCAT('AIP','NewCo',GET_REGION_SCORECARD(RISK_ST_ABBR),BU_ABBR) as Key,
	zero_cond_scorecard(18, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "18",
	zero_cond_scorecard(17, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "17",
	zero_cond_scorecard(16, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "16",
	zero_cond_scorecard(15, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "15",
	zero_cond_scorecard(14, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "14",
	zero_cond_scorecard(13, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "13",
	zero_cond_scorecard(12, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "12",
	zero_cond_scorecard(11, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "11",
	zero_cond_scorecard(10, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "10",
	zero_cond_scorecard(9, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "9",
	zero_cond_scorecard(8, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "8",
	zero_cond_scorecard(7, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "7",
	zero_cond_scorecard(6, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "6",
	zero_cond_scorecard(5, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "5",
	zero_cond_scorecard(4, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "4",
	zero_cond_scorecard(3, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "3",
	zero_cond_scorecard(2, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "2",
	zero_cond_scorecard(1, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "1",
	zero_cond_scorecard(0, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN POL_TTL_BILL_PREM_AMT * 2 ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE ISSUE_IND = 'Y'
	GROUP BY Key
),

cw_aq_aip AS -- CW AIP for Auto Quote
(
	SELECT CONCAT('AIP','NewCo','CW',BU_ABBR) as Key,
	zero_cond_scorecard(18, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "18",
	zero_cond_scorecard(17, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "17",
	zero_cond_scorecard(16, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "16",
	zero_cond_scorecard(15, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "15",
	zero_cond_scorecard(14, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "14",
	zero_cond_scorecard(13, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "13",
	zero_cond_scorecard(12, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "12",
	zero_cond_scorecard(11, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "11",
	zero_cond_scorecard(10, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "10",
	zero_cond_scorecard(9, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "9",
	zero_cond_scorecard(8, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "8",
	zero_cond_scorecard(7, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "7",
	zero_cond_scorecard(6, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "6",
	zero_cond_scorecard(5, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "5",
	zero_cond_scorecard(4, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "4",
	zero_cond_scorecard(3, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "3",
	zero_cond_scorecard(2, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "2",
	zero_cond_scorecard(1, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "1",
	zero_cond_scorecard(0, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN POL_TTL_BILL_PREM_AMT * 2 ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE ISSUE_IND = 'Y'
	GROUP BY Key
),

state_scdi_aip AS -- State AIP for Auto SCDI
(
	SELECT CONCAT('AIP','NowCo',RISK_STATE_ABBR,BUS_UNIT_ABBR) as Key,
	zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "18",
	zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "17",
	zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "16",
	zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "15",
	zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "14",
	zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "13",
	zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "12",
	zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "11",
	zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "10",
	zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "9",
	zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "8",
	zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "7",
	zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "6",
	zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "5",
	zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "4",
	zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "3",
	zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "2",
	zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "1",
	zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN ANNUALIZED_PREM_AMT ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND ISSUE_IND = 'Y'
	GROUP BY Key
),

region_scdi_aip AS -- Regional AIP for Auto SCDI
(
	SELECT CONCAT('AIP','NowCo',GET_REGION_SCORECARD(RISK_STATE_ABBR),BUS_UNIT_ABBR) as Key,
	zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "18",
	zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "17",
	zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "16",
	zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "15",
	zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "14",
	zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "13",
	zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "12",
	zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "11",
	zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "10",
	zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "9",
	zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "8",
	zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "7",
	zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "6",
	zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "5",
	zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "4",
	zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "3",
	zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "2",
	zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "1",
	zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN ANNUALIZED_PREM_AMT ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND ISSUE_IND = 'Y'
	GROUP BY Key
),

cw_scdi_aip AS -- CW AIP for Auto SCDI
(
	SELECT CONCAT('AIP','NowCo','CW',BUS_UNIT_ABBR) as Key,
	zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "18",
	zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "17",
	zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "16",
	zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "15",
	zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "14",
	zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "13",
	zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "12",
	zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "11",
	zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "10",
	zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "9",
	zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "8",
	zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "7",
	zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "6",
	zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "5",
	zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "4",
	zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "3",
	zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "2",
	zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "1",
	zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN ANNUALIZED_PREM_AMT ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND ISSUE_IND = 'Y'
	GROUP BY Key
),

state_aq_cq AS -- State Completed Quote Count for Auto Quote
(
	SELECT CONCAT('Completed Quotes','NewCo',RISK_ST_ABBR,BU_ABBR) as Key,
	zero_cond_scorecard(18, TRANS_DT, COMPL_QTE_IND_CNT) AS "18",
	zero_cond_scorecard(17, TRANS_DT, COMPL_QTE_IND_CNT) AS "17",
	zero_cond_scorecard(16, TRANS_DT, COMPL_QTE_IND_CNT) AS "16",
	zero_cond_scorecard(15, TRANS_DT, COMPL_QTE_IND_CNT) AS "15",
	zero_cond_scorecard(14, TRANS_DT, COMPL_QTE_IND_CNT) AS "14",
	zero_cond_scorecard(13, TRANS_DT, COMPL_QTE_IND_CNT) AS "13",
	zero_cond_scorecard(12, TRANS_DT, COMPL_QTE_IND_CNT) AS "12",
	zero_cond_scorecard(11, TRANS_DT, COMPL_QTE_IND_CNT) AS "11",
	zero_cond_scorecard(10, TRANS_DT, COMPL_QTE_IND_CNT) AS "10",
	zero_cond_scorecard(9, TRANS_DT, COMPL_QTE_IND_CNT) AS "9",
	zero_cond_scorecard(8, TRANS_DT, COMPL_QTE_IND_CNT) AS "8",
	zero_cond_scorecard(7, TRANS_DT, COMPL_QTE_IND_CNT) AS "7",
	zero_cond_scorecard(6, TRANS_DT, COMPL_QTE_IND_CNT) AS "6",
	zero_cond_scorecard(5, TRANS_DT, COMPL_QTE_IND_CNT) AS "5",
	zero_cond_scorecard(4, TRANS_DT, COMPL_QTE_IND_CNT) AS "4",
	zero_cond_scorecard(3, TRANS_DT, COMPL_QTE_IND_CNT) AS "3",
	zero_cond_scorecard(2, TRANS_DT, COMPL_QTE_IND_CNT) AS "2",
	zero_cond_scorecard(1, TRANS_DT, COMPL_QTE_IND_CNT) AS "1",
	zero_cond_scorecard(0, TRANS_DT, COMPL_QTE_IND_CNT) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN COMPL_QTE_IND_CNT ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE COMPL_QTE_IND = 'Y'
	GROUP BY Key
),

region_aq_cq AS -- Regional Completed Quote Count for Auto Quote
(
	SELECT CONCAT('Completed Quotes','NewCo',GET_REGION_SCORECARD(RISK_ST_ABBR),BU_ABBR) as Key,
	zero_cond_scorecard(18, TRANS_DT, COMPL_QTE_IND_CNT) AS "18",
	zero_cond_scorecard(17, TRANS_DT, COMPL_QTE_IND_CNT) AS "17",
	zero_cond_scorecard(16, TRANS_DT, COMPL_QTE_IND_CNT) AS "16",
	zero_cond_scorecard(15, TRANS_DT, COMPL_QTE_IND_CNT) AS "15",
	zero_cond_scorecard(14, TRANS_DT, COMPL_QTE_IND_CNT) AS "14",
	zero_cond_scorecard(13, TRANS_DT, COMPL_QTE_IND_CNT) AS "13",
	zero_cond_scorecard(12, TRANS_DT, COMPL_QTE_IND_CNT) AS "12",
	zero_cond_scorecard(11, TRANS_DT, COMPL_QTE_IND_CNT) AS "11",
	zero_cond_scorecard(10, TRANS_DT, COMPL_QTE_IND_CNT) AS "10",
	zero_cond_scorecard(9, TRANS_DT, COMPL_QTE_IND_CNT) AS "9",
	zero_cond_scorecard(8, TRANS_DT, COMPL_QTE_IND_CNT) AS "8",
	zero_cond_scorecard(7, TRANS_DT, COMPL_QTE_IND_CNT) AS "7",
	zero_cond_scorecard(6, TRANS_DT, COMPL_QTE_IND_CNT) AS "6",
	zero_cond_scorecard(5, TRANS_DT, COMPL_QTE_IND_CNT) AS "5",
	zero_cond_scorecard(4, TRANS_DT, COMPL_QTE_IND_CNT) AS "4",
	zero_cond_scorecard(3, TRANS_DT, COMPL_QTE_IND_CNT) AS "3",
	zero_cond_scorecard(2, TRANS_DT, COMPL_QTE_IND_CNT) AS "2",
	zero_cond_scorecard(1, TRANS_DT, COMPL_QTE_IND_CNT) AS "1",
	zero_cond_scorecard(0, TRANS_DT, COMPL_QTE_IND_CNT) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN COMPL_QTE_IND_CNT ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE COMPL_QTE_IND = 'Y'
	GROUP BY Key
),

cw_aq_cq AS -- CW Completed Quote Count for Auto Quote
(
	SELECT CONCAT('Completed Quotes','NewCo','CW',BU_ABBR) as Key,
	zero_cond_scorecard(18, TRANS_DT, COMPL_QTE_IND_CNT) AS "18",
	zero_cond_scorecard(17, TRANS_DT, COMPL_QTE_IND_CNT) AS "17",
	zero_cond_scorecard(16, TRANS_DT, COMPL_QTE_IND_CNT) AS "16",
	zero_cond_scorecard(15, TRANS_DT, COMPL_QTE_IND_CNT) AS "15",
	zero_cond_scorecard(14, TRANS_DT, COMPL_QTE_IND_CNT) AS "14",
	zero_cond_scorecard(13, TRANS_DT, COMPL_QTE_IND_CNT) AS "13",
	zero_cond_scorecard(12, TRANS_DT, COMPL_QTE_IND_CNT) AS "12",
	zero_cond_scorecard(11, TRANS_DT, COMPL_QTE_IND_CNT) AS "11",
	zero_cond_scorecard(10, TRANS_DT, COMPL_QTE_IND_CNT) AS "10",
	zero_cond_scorecard(9, TRANS_DT, COMPL_QTE_IND_CNT) AS "9",
	zero_cond_scorecard(8, TRANS_DT, COMPL_QTE_IND_CNT) AS "8",
	zero_cond_scorecard(7, TRANS_DT, COMPL_QTE_IND_CNT) AS "7",
	zero_cond_scorecard(6, TRANS_DT, COMPL_QTE_IND_CNT) AS "6",
	zero_cond_scorecard(5, TRANS_DT, COMPL_QTE_IND_CNT) AS "5",
	zero_cond_scorecard(4, TRANS_DT, COMPL_QTE_IND_CNT) AS "4",
	zero_cond_scorecard(3, TRANS_DT, COMPL_QTE_IND_CNT) AS "3",
	zero_cond_scorecard(2, TRANS_DT, COMPL_QTE_IND_CNT) AS "2",
	zero_cond_scorecard(1, TRANS_DT, COMPL_QTE_IND_CNT) AS "1",
	zero_cond_scorecard(0, TRANS_DT, COMPL_QTE_IND_CNT) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN COMPL_QTE_IND_CNT ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE COMPL_QTE_IND = 'Y'
	GROUP BY Key
),

state_scdi_cq AS -- State Completed Quote Count for Auto SCDI
(
	SELECT CONCAT('Completed Quotes','NowCo',RISK_STATE_ABBR,BUS_UNIT_ABBR) as Key,
	zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "18",
	zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "17",
	zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "16",
	zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "15",
	zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "14",
	zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "13",
	zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "12",
	zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "11",
	zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "10",
	zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "9",
	zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "8",
	zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "7",
	zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "6",
	zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "5",
	zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "4",
	zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "3",
	zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "2",
	zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "1",
	zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN COMPL_QUOTE_IND_CNT ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND COMPL_QUOTE_IND = 'Y'
	GROUP BY Key
),

region_scdi_cq AS -- Regional Completed Quote Count for Auto SCDI
(
	SELECT CONCAT('Completed Quotes','NowCo',GET_REGION_SCORECARD(RISK_STATE_ABBR),BUS_UNIT_ABBR) as Key,
	zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "18",
	zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "17",
	zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "16",
	zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "15",
	zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "14",
	zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "13",
	zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "12",
	zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "11",
	zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "10",
	zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "9",
	zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "8",
	zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "7",
	zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "6",
	zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "5",
	zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "4",
	zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "3",
	zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "2",
	zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "1",
	zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN COMPL_QUOTE_IND_CNT ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND COMPL_QUOTE_IND = 'Y'
	GROUP BY Key
),

cw_scdi_cq AS -- CW Completed Quote Count for Auto SCDI
(
	SELECT CONCAT('Completed Quotes','NowCo','CW',BUS_UNIT_ABBR) as Key,
	zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "18",
	zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "17",
	zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "16",
	zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "15",
	zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "14",
	zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "13",
	zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "12",
	zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "11",
	zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "10",
	zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "9",
	zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "8",
	zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "7",
	zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "6",
	zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "5",
	zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "4",
	zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "3",
	zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "2",
	zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "1",
	zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN COMPL_QUOTE_IND_CNT ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND COMPL_QUOTE_IND = 'Y'
	GROUP BY Key
),

state_aq_higsrp AS -- State HIGSRP for Auto Quote
(
	SELECT CONCAT('HIGSRP','NewCo',RISK_ST_ABBR,BU_ABBR) AS Key,
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=18 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "18",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=17 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "17",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=16 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "16",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=15 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "15",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=14 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "14",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=13 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "13",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=12 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "12",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=11 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "11",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=10 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "10",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 9 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "9",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 8 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "8",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 7 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "7",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 6 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "6",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 5 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "5",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 4 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "4",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 3 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "3",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 2 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "2",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 1 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "1",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 0 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "0",
	AVG(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE())= 0 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6
	AND PCARR_POL_ANNL_PREM_AMT > 249 AND PCARR_POL_ANNL_PREM_AMT < 6000
	AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND  CNTCT_METH_CD = 'P'
	GROUP BY Key
),

region_aq_higsrp AS -- Regional HIGSRP for Auto Quote
(
	SELECT CONCAT('HIGSRP','NewCo',GET_REGION_SCORECARD(RISK_ST_ABBR),BU_ABBR) AS Key,
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=18 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "18",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=17 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "17",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=16 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "16",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=15 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "15",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=14 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "14",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=13 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "13",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=12 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "12",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=11 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "11",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=10 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "10",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 9 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "9",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 8 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "8",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 7 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "7",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 6 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "6",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 5 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "5",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 4 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "4",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 3 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "3",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 2 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "2",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 1 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "1",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 0 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "0",
	AVG(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE())= 0 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6
	AND PCARR_POL_ANNL_PREM_AMT > 249 AND PCARR_POL_ANNL_PREM_AMT < 6000
	AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND  CNTCT_METH_CD = 'P'
	GROUP BY Key
),

cw_aq_higsrp AS -- CW HIGSRP for Auto Quote
(
	SELECT CONCAT('HIGSRP','NewCo','CW',BU_ABBR) AS Key,
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=18 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "18",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=17 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "17",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=16 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "16",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=15 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "15",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=14 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "14",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=13 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "13",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=12 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "12",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=11 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "11",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())=10 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "10",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 9 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "9",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 8 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "8",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 7 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "7",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 6 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "6",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 5 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "5",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 4 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "4",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 3 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "3",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 2 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "2",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 1 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "1",
	AVG(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE())= 0 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "0",
	AVG(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE())= 0 THEN POL_ANNL_TTL_BILL_PREM_AMT / PCARR_POL_ANNL_PREM_AMT ELSE NULL END) as "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6
	AND PCARR_POL_ANNL_PREM_AMT > 249 AND PCARR_POL_ANNL_PREM_AMT < 6000
	AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND  CNTCT_METH_CD = 'P'
	GROUP BY Key
),

state_scdi_higsrp AS -- State HIGSRP for Auto SCDI
(
	SELECT CONCAT('HIGSRP','NowCo',RISK_STATE_ABBR,BUS_UNIT_ABBR) AS Key,
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=18 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "18",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=17 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "17",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=16 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "16",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=15 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "15",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=14 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "14",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=13 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "13",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=12 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "12",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=11 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "11",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=10 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "10",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 9 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "9",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 8 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "8",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 7 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "7",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 6 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "6",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 5 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "5",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 4 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "4",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 3 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "3",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 2 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "2",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 1 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "1",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 0 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "0",
	AVG(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 0 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR = 'AARP' AND COMPL_QUOTE_IND_CNT = 1
	AND PRIOR_CARR_POL_ANNUAL_PREM_AMT >= 250 AND ANNUALIZED_PREM_AMT > 300
	AND ORIG_QUOTE_PROCESS_METH_CD = 'T'
	GROUP BY Key
),

region_scdi_higsrp AS -- Regional HIGSRP for Auto SCDI
(
	SELECT CONCAT('HIGSRP','NowCo',GET_REGION_SCORECARD(RISK_STATE_ABBR),BUS_UNIT_ABBR) AS Key,
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=18 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "18",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=17 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "17",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=16 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "16",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=15 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "15",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=14 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "14",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=13 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "13",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=12 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "12",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=11 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "11",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=10 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "10",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 9 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "9",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 8 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "8",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 7 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "7",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 6 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "6",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 5 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "5",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 4 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "4",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 3 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "3",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 2 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "2",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 1 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "1",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 0 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "0",
	AVG(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 0 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR = 'AARP' AND COMPL_QUOTE_IND_CNT = 1
	AND PRIOR_CARR_POL_ANNUAL_PREM_AMT >= 250 AND ANNUALIZED_PREM_AMT > 300
	AND ORIG_QUOTE_PROCESS_METH_CD = 'T'
	GROUP BY Key
),

cw_scdi_higsrp AS -- CW HIGSRP for Auto SCDI
(
	SELECT CONCAT('HIGSRP','NowCo','CW',BUS_UNIT_ABBR) AS Key,
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=18 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "18",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=17 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "17",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=16 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "16",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=15 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "15",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=14 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "14",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=13 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "13",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=12 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "12",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=11 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "11",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())=10 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "10",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 9 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "9",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 8 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "8",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 7 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "7",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 6 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "6",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 5 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "5",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 4 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "4",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 3 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "3",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 2 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "2",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 1 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "1",
	AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 0 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "0",
	AVG(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE())= 0 THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) as "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR = 'AARP' AND COMPL_QUOTE_IND_CNT = 1
	AND PRIOR_CARR_POL_ANNUAL_PREM_AMT >= 250 AND ANNUALIZED_PREM_AMT > 300
	AND ORIG_QUOTE_PROCESS_METH_CD = 'T'
	GROUP BY Key
),

state_aq_aqp AS -- State AQP for Auto Quote
(
	SELECT CONCAT('AQP','NewCo',RISK_ST_ABBR,BU_ABBR) as Key,
	zero_cond_scorecard(18, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "18",
	zero_cond_scorecard(17, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "17",
	zero_cond_scorecard(16, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "16",
	zero_cond_scorecard(15, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "15",
	zero_cond_scorecard(14, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "14",
	zero_cond_scorecard(13, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "13",
	zero_cond_scorecard(12, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "12",
	zero_cond_scorecard(11, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "11",
	zero_cond_scorecard(10, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "10",
	zero_cond_scorecard(9, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "9",
	zero_cond_scorecard(8, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "8",
	zero_cond_scorecard(7, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "7",
	zero_cond_scorecard(6, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "6",
	zero_cond_scorecard(5, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "5",
	zero_cond_scorecard(4, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "4",
	zero_cond_scorecard(3, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "3",
	zero_cond_scorecard(2, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "2",
	zero_cond_scorecard(1, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "1",
	zero_cond_scorecard(0, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN POL_TTL_BILL_PREM_AMT * 2 ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE COMPL_QTE_IND_CNT = '1'
	GROUP BY Key
),

region_aq_aqp AS -- Regional AQP for Auto Quote
(
	SELECT CONCAT('AQP','NewCo',GET_REGION_SCORECARD(RISK_ST_ABBR),BU_ABBR) as Key,
	zero_cond_scorecard(18, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "18",
	zero_cond_scorecard(17, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "17",
	zero_cond_scorecard(16, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "16",
	zero_cond_scorecard(15, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "15",
	zero_cond_scorecard(14, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "14",
	zero_cond_scorecard(13, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "13",
	zero_cond_scorecard(12, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "12",
	zero_cond_scorecard(11, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "11",
	zero_cond_scorecard(10, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "10",
	zero_cond_scorecard(9, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "9",
	zero_cond_scorecard(8, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "8",
	zero_cond_scorecard(7, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "7",
	zero_cond_scorecard(6, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "6",
	zero_cond_scorecard(5, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "5",
	zero_cond_scorecard(4, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "4",
	zero_cond_scorecard(3, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "3",
	zero_cond_scorecard(2, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "2",
	zero_cond_scorecard(1, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "1",
	zero_cond_scorecard(0, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN POL_TTL_BILL_PREM_AMT * 2 ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE COMPL_QTE_IND_CNT = '1'
	GROUP BY Key
),

cw_aq_aqp AS -- CW AQP for Auto Quote
(
	SELECT CONCAT('AQP','NewCo','CW',BU_ABBR) as Key,
	zero_cond_scorecard(18, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "18",
	zero_cond_scorecard(17, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "17",
	zero_cond_scorecard(16, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "16",
	zero_cond_scorecard(15, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "15",
	zero_cond_scorecard(14, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "14",
	zero_cond_scorecard(13, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "13",
	zero_cond_scorecard(12, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "12",
	zero_cond_scorecard(11, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "11",
	zero_cond_scorecard(10, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "10",
	zero_cond_scorecard(9, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "9",
	zero_cond_scorecard(8, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "8",
	zero_cond_scorecard(7, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "7",
	zero_cond_scorecard(6, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "6",
	zero_cond_scorecard(5, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "5",
	zero_cond_scorecard(4, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "4",
	zero_cond_scorecard(3, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "3",
	zero_cond_scorecard(2, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "2",
	zero_cond_scorecard(1, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "1",
	zero_cond_scorecard(0, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN POL_TTL_BILL_PREM_AMT * 2 ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW"
	WHERE COMPL_QTE_IND_CNT = '1'
	GROUP BY Key
),

state_scdi_aqp AS -- State AQP for Auto SCDI
(
	SELECT CONCAT('AQP','NowCo',RISK_STATE_ABBR,BUS_UNIT_ABBR) as Key,
	zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "18",
	zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "17",
	zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "16",
	zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "15",
	zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "14",
	zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "13",
	zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "12",
	zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "11",
	zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "10",
	zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "9",
	zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "8",
	zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "7",
	zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "6",
	zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "5",
	zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "4",
	zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "3",
	zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "2",
	zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "1",
	zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN ANNUALIZED_PREM_AMT ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND COMPL_QUOTE_IND_CNT = '1'
	GROUP BY Key
),

region_scdi_aqp AS -- Regional AQP for Auto SCDI
(
	SELECT CONCAT('AQP','NowCo',GET_REGION_SCORECARD(RISK_STATE_ABBR),BUS_UNIT_ABBR) as Key,
	zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "18",
	zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "17",
	zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "16",
	zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "15",
	zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "14",
	zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "13",
	zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "12",
	zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "11",
	zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "10",
	zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "9",
	zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "8",
	zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "7",
	zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "6",
	zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "5",
	zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "4",
	zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "3",
	zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "2",
	zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "1",
	zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN ANNUALIZED_PREM_AMT ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND COMPL_QUOTE_IND_CNT = '1'
	GROUP BY Key
),

cw_scdi_aqp AS -- CW AQP for Auto SCDI
(
	SELECT CONCAT('AQP','NowCo','CW',BUS_UNIT_ABBR) as Key,
	zero_cond_scorecard(18, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "18",
	zero_cond_scorecard(17, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "17",
	zero_cond_scorecard(16, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "16",
	zero_cond_scorecard(15, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "15",
	zero_cond_scorecard(14, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "14",
	zero_cond_scorecard(13, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "13",
	zero_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "12",
	zero_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "11",
	zero_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "10",
	zero_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "9",
	zero_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "8",
	zero_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "7",
	zero_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "6",
	zero_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "5",
	zero_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "4",
	zero_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "3",
	zero_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "2",
	zero_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "1",
	zero_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ANNUALIZED_PREM_AMT) AS "0",
	SUM(CASE WHEN DATEDIFF(YEAR, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 THEN ANNUALIZED_PREM_AMT ELSE 0 END) AS "YTD",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW"
	WHERE BUS_UNIT_ABBR <> 'CDRT' AND COMPL_QUOTE_IND_CNT = '1'
	GROUP BY Key
),

state_aq_ulm AS -- State ULM from Auto Quote
(
	SELECT CONCAT('ULM','NewCo',RISK_ST_ABBR,BU_ABBR) as Key,
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 18 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 18 THEN RESP_IND_CNT ELSE 0 END)) AS "18",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 17 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 17 THEN RESP_IND_CNT ELSE 0 END)) AS "17",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 16 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 16 THEN RESP_IND_CNT ELSE 0 END)) AS "16",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 15 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 15 THEN RESP_IND_CNT ELSE 0 END)) AS "15",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 14 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 14 THEN RESP_IND_CNT ELSE 0 END)) AS "14",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 13 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 13 THEN RESP_IND_CNT ELSE 0 END)) AS "13",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 12 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 12 THEN RESP_IND_CNT ELSE 0 END)) AS "12",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 11 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 11 THEN RESP_IND_CNT ELSE 0 END)) AS "11",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 10 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 10 THEN RESP_IND_CNT ELSE 0 END)) AS "10",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 9 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 9 THEN RESP_IND_CNT ELSE 0 END)) AS "9",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 8 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 8 THEN RESP_IND_CNT ELSE 0 END)) AS "8",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 7 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 7 THEN RESP_IND_CNT ELSE 0 END)) AS "7",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 6 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 6 THEN RESP_IND_CNT ELSE 0 END)) AS "6",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 5 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 5 THEN RESP_IND_CNT ELSE 0 END)) AS "5",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 4 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 4 THEN RESP_IND_CNT ELSE 0 END)) AS "4",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 3 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 3 THEN RESP_IND_CNT ELSE 0 END)) AS "3",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 2 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 2 THEN RESP_IND_CNT ELSE 0 END)) AS "2",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 1 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 1 THEN RESP_IND_CNT ELSE 0 END)) AS "1",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 0 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 0 THEN RESP_IND_CNT ELSE 0 END)) AS "0",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN RESP_IND_CNT ELSE 0 END)) AS "0",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW" t1
	LEFT JOIN (SELECT INPUT_QUOTENUMBER, TREATMENTDECISION
			   FROM "DSC_PLBI_DB"."APP_AUTO_DEV"."AUTO_ULM_QUOTE_VW"
			   GROUP BY INPUT_QUOTENUMBER, TREATMENTDECISION) t2
	ON t1.QCN = t2.INPUT_QUOTENUMBER
	WHERE ORIG_QTE_CNTCT_METH_CD = 'I' AND TREATMENTDECISION IN (0, 1)
	GROUP BY Key
),

region_aq_ulm AS -- Regional ULM from Auto Quote
(
	SELECT CONCAT('ULM','NewCo',GET_REGION_SCORECARD(RISK_ST_ABBR),BU_ABBR) as Key,
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 18 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 18 THEN RESP_IND_CNT ELSE 0 END)) AS "18",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 17 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 17 THEN RESP_IND_CNT ELSE 0 END)) AS "17",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 16 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 16 THEN RESP_IND_CNT ELSE 0 END)) AS "16",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 15 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 15 THEN RESP_IND_CNT ELSE 0 END)) AS "15",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 14 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 14 THEN RESP_IND_CNT ELSE 0 END)) AS "14",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 13 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 13 THEN RESP_IND_CNT ELSE 0 END)) AS "13",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 12 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 12 THEN RESP_IND_CNT ELSE 0 END)) AS "12",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 11 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 11 THEN RESP_IND_CNT ELSE 0 END)) AS "11",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 10 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 10 THEN RESP_IND_CNT ELSE 0 END)) AS "10",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 9 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 9 THEN RESP_IND_CNT ELSE 0 END)) AS "9",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 8 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 8 THEN RESP_IND_CNT ELSE 0 END)) AS "8",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 7 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 7 THEN RESP_IND_CNT ELSE 0 END)) AS "7",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 6 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 6 THEN RESP_IND_CNT ELSE 0 END)) AS "6",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 5 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 5 THEN RESP_IND_CNT ELSE 0 END)) AS "5",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 4 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 4 THEN RESP_IND_CNT ELSE 0 END)) AS "4",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 3 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 3 THEN RESP_IND_CNT ELSE 0 END)) AS "3",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 2 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 2 THEN RESP_IND_CNT ELSE 0 END)) AS "2",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 1 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 1 THEN RESP_IND_CNT ELSE 0 END)) AS "1",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 0 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 0 THEN RESP_IND_CNT ELSE 0 END)) AS "0",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN RESP_IND_CNT ELSE 0 END)) AS "0",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW" t1
	LEFT JOIN (SELECT INPUT_QUOTENUMBER, TREATMENTDECISION
			   FROM "DSC_PLBI_DB"."APP_AUTO_DEV"."AUTO_ULM_QUOTE_VW"
			   GROUP BY INPUT_QUOTENUMBER, TREATMENTDECISION) t2
	ON t1.QCN = t2.INPUT_QUOTENUMBER
	WHERE ORIG_QTE_CNTCT_METH_CD = 'I' AND TREATMENTDECISION IN (0, 1)
	GROUP BY Key
),

cw_aq_ulm AS -- CW ULM from Auto Quote
(
	SELECT CONCAT('ULM','NewCo','CW',BU_ABBR) as Key,
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 18 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 18 THEN RESP_IND_CNT ELSE 0 END)) AS "18",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 17 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 17 THEN RESP_IND_CNT ELSE 0 END)) AS "17",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 16 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 16 THEN RESP_IND_CNT ELSE 0 END)) AS "16",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 15 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 15 THEN RESP_IND_CNT ELSE 0 END)) AS "15",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 14 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 14 THEN RESP_IND_CNT ELSE 0 END)) AS "14",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 13 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 13 THEN RESP_IND_CNT ELSE 0 END)) AS "13",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 12 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 12 THEN RESP_IND_CNT ELSE 0 END)) AS "12",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 11 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 11 THEN RESP_IND_CNT ELSE 0 END)) AS "11",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 10 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 10 THEN RESP_IND_CNT ELSE 0 END)) AS "10",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 9 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 9 THEN RESP_IND_CNT ELSE 0 END)) AS "9",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 8 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 8 THEN RESP_IND_CNT ELSE 0 END)) AS "8",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 7 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 7 THEN RESP_IND_CNT ELSE 0 END)) AS "7",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 6 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 6 THEN RESP_IND_CNT ELSE 0 END)) AS "6",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 5 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 5 THEN RESP_IND_CNT ELSE 0 END)) AS "5",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 4 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 4 THEN RESP_IND_CNT ELSE 0 END)) AS "4",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 3 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 3 THEN RESP_IND_CNT ELSE 0 END)) AS "3",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 2 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 2 THEN RESP_IND_CNT ELSE 0 END)) AS "2",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 1 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 1 THEN RESP_IND_CNT ELSE 0 END)) AS "1",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 0 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(MONTH, TRANS_DT, SYSDATE()) = 0 THEN RESP_IND_CNT ELSE 0 END)) AS "0",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 AND TREATMENTDECISION = 1 THEN RESP_IND_CNT ELSE 0 END), 
			 SUM(CASE WHEN DATEDIFF(YEAR, TRANS_DT, SYSDATE()) = 0 THEN RESP_IND_CNT ELSE 0 END)) AS "0",
	TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW" t1
	LEFT JOIN (SELECT INPUT_QUOTENUMBER, TREATMENTDECISION
			   FROM "DSC_PLBI_DB"."APP_AUTO_DEV"."AUTO_ULM_QUOTE_VW"
			   GROUP BY INPUT_QUOTENUMBER, TREATMENTDECISION) t2
	ON t1.QCN = t2.INPUT_QUOTENUMBER
	WHERE ORIG_QTE_CNTCT_METH_CD = 'I' AND TREATMENTDECISION IN (0, 1)
	GROUP BY Key
)

SELECT * FROM
(
	SELECT * FROM state_aq_response
	UNION ALL
	SELECT * FROM region_aq_response
	UNION ALL
	SELECT * FROM cw_aq_response
	UNION ALL
	SELECT * FROM state_scdi_response
	UNION ALL
	SELECT * FROM region_scdi_response
	UNION ALL
	SELECT * FROM cw_scdi_response
	UNION ALL
	SELECT * FROM state_aq_phone
	UNION ALL
	SELECT * FROM region_aq_phone
	UNION ALL
	SELECT * FROM cw_aq_phone
	UNION ALL
	SELECT * FROM state_scdi_phone
	UNION ALL
	SELECT * FROM region_scdi_phone
    UNION ALL
    SELECT * FROM cw_scdi_phone
	UNION ALL
	SELECT * FROM state_aq_phone_cq
	UNION ALL
	SELECT * FROM region_aq_phone_cq
	UNION ALL
	SELECT * FROM cw_aq_phone_cq
	UNION ALL
	SELECT * FROM state_scdi_phone_cq
	UNION ALL
	SELECT * FROM region_scdi_phone_cq
	UNION ALL
	SELECT * FROM cw_scdi_phone_cq
	UNION ALL
	SELECT * FROM state_aq_issue
	UNION ALL
	SELECT * FROM region_aq_issue
	UNION ALL
	SELECT * FROM cw_aq_issue
	UNION ALL
	SELECT * FROM state_scdi_issue
	UNION ALL
	SELECT * FROM region_scdi_issue
	UNION ALL
	SELECT * FROM cw_scdi_issue
	UNION ALL
	SELECT * FROM state_aq_phone_issue
	UNION ALL
	SELECT * FROM region_aq_phone_issue
	UNION ALL
	SELECT * FROM cw_aq_phone_issue
	UNION ALL
	SELECT * FROM state_scdi_phone_issue
	UNION ALL
	SELECT * FROM region_scdi_phone_issue
	UNION ALL
	SELECT * FROM cw_scdi_phone_issue
	UNION ALL
	SELECT * FROM state_aq_aip
	UNION ALL
	SELECT * FROM region_aq_aip
	UNION ALL
	SELECT * FROM cw_aq_aip
	UNION ALL
	SELECT * FROM state_scdi_aip
	UNION ALL
	SELECT * FROM region_scdi_aip
	UNION ALL
	SELECT * FROM cw_scdi_aip
	UNION ALL
	SELECT * FROM state_aq_cq
	UNION ALL
	SELECT * FROM region_aq_cq
	UNION ALL
	SELECT * FROM cw_aq_cq
	UNION ALL
	SELECT * FROM state_scdi_cq
	UNION ALL
	SELECT * FROM region_scdi_cq
	UNION ALL
	SELECT * FROM cw_scdi_cq
	UNION ALL
	SELECT * FROM state_aq_higsrp
	UNION ALL
	SELECT * FROM region_aq_higsrp
	UNION ALL
	SELECT * FROM cw_aq_higsrp
	UNION ALL
	SELECT * FROM state_scdi_higsrp
	UNION ALL
	SELECT * FROM region_scdi_higsrp
	UNION ALL
	SELECT * FROM cw_scdi_higsrp
	UNION ALL
	SELECT * FROM state_aq_aqp
	UNION ALL
	SELECT * FROM region_aq_aqp
	UNION ALL
	SELECT * FROM cw_aq_aqp
	UNION ALL
	SELECT * FROM state_scdi_aqp
	UNION ALL
	SELECT * FROM region_scdi_aqp
	UNION ALL
	SELECT * FROM cw_scdi_aqp
	UNION ALL
	SELECT * FROM state_aq_ulm
	UNION ALL
	SELECT * FROM region_aq_ulm
	UNION ALL
	SELECT * FROM cw_aq_ulm
)
ORDER BY KEY
"""

In [4]:
cs = ctx.cursor()
try:
    cs.execute(auto_flow)
finally:
    cs.close()
print('Done')

Done


## 2. Auto Seg State Table Creation (DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_AUTO_SEG_STATE)

In [5]:
query_2_body = """
    -- Total Completed Quotes
    zero_cond_scorecard(12, TRANS_DT, COMPL_QTE_IND_CNT) AS "12 Total CQ",
    zero_cond_scorecard(11, TRANS_DT, COMPL_QTE_IND_CNT) AS "11 Total CQ",
    zero_cond_scorecard(10, TRANS_DT, COMPL_QTE_IND_CNT) AS "10 Total CQ",
    zero_cond_scorecard(9, TRANS_DT, COMPL_QTE_IND_CNT) AS "9 Total CQ",
    zero_cond_scorecard(8, TRANS_DT, COMPL_QTE_IND_CNT) AS "8 Total CQ",
    zero_cond_scorecard(7, TRANS_DT, COMPL_QTE_IND_CNT) AS "7 Total CQ",
    zero_cond_scorecard(6, TRANS_DT, COMPL_QTE_IND_CNT) AS "6 Total CQ",
    zero_cond_scorecard(5, TRANS_DT, COMPL_QTE_IND_CNT) AS "5 Total CQ",
    zero_cond_scorecard(4, TRANS_DT, COMPL_QTE_IND_CNT) AS "4 Total CQ",
    zero_cond_scorecard(3, TRANS_DT, COMPL_QTE_IND_CNT) AS "3 Total CQ",
    zero_cond_scorecard(2, TRANS_DT, COMPL_QTE_IND_CNT) AS "2 Total CQ",
    zero_cond_scorecard(1, TRANS_DT, COMPL_QTE_IND_CNT) AS "1 Total CQ",
    zero_cond_scorecard(0, TRANS_DT, COMPL_QTE_IND_CNT) AS "0 Total CQ",
    -- Total Issues
    one_cond_scorecard(12, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "12 Total I",
    one_cond_scorecard(11, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "11 Total I",
    one_cond_scorecard(10, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "10 Total I",
    one_cond_scorecard(9, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "9 Total I",
    one_cond_scorecard(8, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "8 Total I",
    one_cond_scorecard(7, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "7 Total I",
    one_cond_scorecard(6, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "6 Total I",
    one_cond_scorecard(5, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "5 Total I",
    one_cond_scorecard(4, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "4 Total I",
    one_cond_scorecard(3, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "3 Total I",
    one_cond_scorecard(2, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "2 Total I",
    one_cond_scorecard(1, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "1 Total I",
    one_cond_scorecard(0, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "0 Total I",
    -- Total AQP
    zero_cond_scorecard(12, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "12 Total AQP",
    zero_cond_scorecard(11, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "11 Total AQP",
    zero_cond_scorecard(10, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "10 Total AQP",
    zero_cond_scorecard(9, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "9 Total AQP",
    zero_cond_scorecard(8, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "8 Total AQP",
    zero_cond_scorecard(7, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "7 Total AQP",
    zero_cond_scorecard(6, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "6 Total AQP",
    zero_cond_scorecard(5, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "5 Total AQP",
    zero_cond_scorecard(4, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "4 Total AQP",
    zero_cond_scorecard(3, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "3 Total AQP",
    zero_cond_scorecard(2, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "2 Total AQP",
    zero_cond_scorecard(1, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "1 Total AQP",
    zero_cond_scorecard(0, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "0 Total AQP",
    -- Total AIP
    one_cond_scorecard(12, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "12 Total AIP",
    one_cond_scorecard(11, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "11 Total AIP",
    one_cond_scorecard(10, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "10 Total AIP",
    one_cond_scorecard(9, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "9 Total AIP",
    one_cond_scorecard(8, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "8 Total AIP",
    one_cond_scorecard(7, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "7 Total AIP",
    one_cond_scorecard(6, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "6 Total AIP",
    one_cond_scorecard(5, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "5 Total AIP",
    one_cond_scorecard(4, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "4 Total AIP",
    one_cond_scorecard(3, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "3 Total AIP",
    one_cond_scorecard(2, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "2 Total AIP",
    one_cond_scorecard(1, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "1 Total AIP",
    one_cond_scorecard(0, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "0 Total AIP",
    -- Total ULM
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 12 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "12 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 11 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "11 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 10 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "10 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 9 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "9 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 8 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "8 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 7 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "7 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 6 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "6 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 5 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "5 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 4 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "4 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 3 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "3 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 2 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "2 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 1 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "1 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 0 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "0 Total ULM",
    -- Phone HIGSRP
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())=12 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT 
    ELSE NULL END)) AS "12 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())=11 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "11 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())=10 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "10 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 9 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "9 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 8 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "8 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 7 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "7 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 6 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "6 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 5 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "5 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 4 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "4 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 3 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "3 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 2 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "2 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 1 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "1 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 0 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "0 Phone HIGSRP",																								
    -- Phone CQ
    one_cond_scorecard(12, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "12 Phone CQ",
    one_cond_scorecard(11, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "11 Phone CQ",
    one_cond_scorecard(10, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "10 Phone CQ",
    one_cond_scorecard(9, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "9 Phone CQ",
    one_cond_scorecard(8, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "8 Phone CQ",
    one_cond_scorecard(7, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "7 Phone CQ",
    one_cond_scorecard(6, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "6 Phone CQ",
    one_cond_scorecard(5, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "5 Phone CQ",
    one_cond_scorecard(4, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "4 Phone CQ",
    one_cond_scorecard(3, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "3 Phone CQ",
    one_cond_scorecard(2, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "2 Phone CQ",
    one_cond_scorecard(1, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "1 Phone CQ",
    one_cond_scorecard(0, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "0 Phone CQ",
    -- Phone Issues
    two_cond_scorecard(12, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "12 Phone I",
    two_cond_scorecard(11, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "11 Phone I",
    two_cond_scorecard(10, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "10 Phone I",
    two_cond_scorecard(9, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "9 Phone I",
    two_cond_scorecard(8, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "8 Phone I",
    two_cond_scorecard(7, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "7 Phone I",
    two_cond_scorecard(6, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "6 Phone I",
    two_cond_scorecard(5, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "5 Phone I",
    two_cond_scorecard(4, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "4 Phone I",
    two_cond_scorecard(3, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "3 Phone I",
    two_cond_scorecard(2, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "2 Phone I",
    two_cond_scorecard(1, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "1 Phone I",
    two_cond_scorecard(0, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "0 Phone I",
    -- Phone AQP
    one_cond_scorecard(12, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "12 Phone AQP",
    one_cond_scorecard(11, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "11 Phone AQP",
    one_cond_scorecard(10, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "10 Phone AQP",
    one_cond_scorecard(9, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "9 Phone AQP",
    one_cond_scorecard(8, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "8 Phone AQP",
    one_cond_scorecard(7, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "7 Phone AQP",
    one_cond_scorecard(6, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "6 Phone AQP",
    one_cond_scorecard(5, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "5 Phone AQP",
    one_cond_scorecard(4, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "4 Phone AQP",
    one_cond_scorecard(3, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "3 Phone AQP",
    one_cond_scorecard(2, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "2 Phone AQP",
    one_cond_scorecard(1, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "1 Phone AQP",
    one_cond_scorecard(0, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "0 Phone AQP",
    -- Phone AIP
    two_cond_scorecard(12, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "12 Phone AIP",
    two_cond_scorecard(11, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "11 Phone AIP",
    two_cond_scorecard(10, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "10 Phone AIP",
    two_cond_scorecard(9, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "9 Phone AIP",
    two_cond_scorecard(8, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "8 Phone AIP",
    two_cond_scorecard(7, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "7 Phone AIP",
    two_cond_scorecard(6, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "6 Phone AIP",
    two_cond_scorecard(5, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "5 Phone AIP",
    two_cond_scorecard(4, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "4 Phone AIP",
    two_cond_scorecard(3, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "3 Phone AIP",
    two_cond_scorecard(2, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "2 Phone AIP",
    two_cond_scorecard(1, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "1 Phone AIP",
    two_cond_scorecard(0, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "0 Phone AIP",
    -- Phone ULM
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 12 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "12 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 11 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "11 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 10 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "10 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 9 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "9 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 8 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "8 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 7 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "7 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 6 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "6 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 5 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "5 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 4 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "4 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 3 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "3 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 2 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "2 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 1 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "1 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 0 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "0 Phone ULM",
    -- Internet ULM 
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 12 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "12 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 11 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "11 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 10 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "10 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 9 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "9 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 8 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "8 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 7 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "7 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 6 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "6 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 5 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "5 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 4 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "4 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 3 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "3 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 2 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "2 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 1 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "1 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 0 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "0 Internet ULM",
    TO_CHAR(SYSDATE(),'MM-DD-YYYY') AS AS_OF
"""

query_2_from = """
    FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW" t1
    LEFT JOIN (SELECT AUTO_QTE_TRANS_ID,EXPECTED_LOSS_RATIO FROM "DSC_PLDS_DB"."APP_AUTOMATA_PRD"."PREVAIL_AUTO_ULM_QUOTE_POL_AGG_NB_QUALITY")
    USING (AUTO_QTE_TRANS_ID)
    LEFT JOIN (SELECT state, TO_CHAR(TO_DATE(TRANS_DT, 'YYYY-MM-DD'),'MM-01-YYYY') AS TRANSDT2, AVG(expected_loss_ratio) AS state_avg_lr
               FROM "DSC_PLDS_DB"."APP_AUTOMATA_PRD"."PREVAIL_AUTO_ULM_QUOTE_POL_AGG_NB_QUALITY" WHERE COMPL_QTE_IND =  'Y' GROUP BY state, TRANSDT2) AS t4
	ON t1.RISK_ST_ABBR = t4.state AND TO_CHAR(TRANS_DT,'MM-01-YYYY') = t4.TRANSDT2
	WHERE COMPL_QTE_IND = 'Y'
	GROUP BY Key
"""

In [6]:
auto_seg_state = f"""
CREATE OR REPLACE TRANSIENT TABLE DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_AUTO_SEG_STATE AS
																								
WITH ins_score AS -- Segmenting indicator counts by State insurance score
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'Ins Score Decile',(																								
    CASE WHEN INS_SCR_CD in (000,0,999)  then 'NHNS'																								
         WHEN INS_SCR_CD <= 386 then 'Decile 10 Worst'																								
         WHEN INS_SCR_CD <= 460 then 'Decile 09'																								
         WHEN INS_SCR_CD <= 539 then 'Decile 08'																								
         WHEN INS_SCR_CD <= 617 then 'Decile 07'																								
         WHEN INS_SCR_CD <= 679 then 'Decile 06'																								
         WHEN INS_SCR_CD <= 724 then 'Decile 05'																								
         WHEN INS_SCR_CD <= 756 then 'Decile 04'																								
         WHEN INS_SCR_CD <= 782 then 'Decile 03'																								
         WHEN INS_SCR_CD <= 809 then 'Decile 02'																								
         WHEN INS_SCR_CD <= 980 then 'Decile 01 Best' else 'NHNS' end)) AS Key,
    {query_2_body} {query_2_from}
),

household_comp AS -- Segmenting indicator counts by State household comp
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'HH Comp',(																								
	CASE WHEN HH_COMPOS_PPV_CD IN ('SC1D') THEN 'a. single vehicle, 1 driver'																								
		 WHEN  HH_COMPOS_PPV_CD IN ('MC C=D') THEN 'd. multi vehicle, vehicle = driver'																								
		 WHEN  HH_COMPOS_PPV_CD IN ('SCMT1D') THEN 'b. single vehicle, more than 1 driver' 																								
		 WHEN  HH_COMPOS_PPV_CD IN ('MCMCTD') THEN 'c. multi vehicle, vehicle > drivers' 																								
		 WHEN  HH_COMPOS_PPV_CD IN ('MCMDTC') THEN 'e. multi vehicle, vehicle < drivers' 																								
		 WHEN  HH_COMPOS_PPV_CD IN ('0 CARS') THEN 'f. 0 vehicle'																								
		 ELSE 'g. other' END)) as Key,
	{query_2_body} {query_2_from}
),

clean_dirty AS -- Segmenting indicator counts by State Clean/Dirty
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'CLEAN DIRTY DESC',CLEAN_DIRTY_DESC) as Key,
	{query_2_body} {query_2_from}
),

max_age_group AS -- Segmenting indicator counts by State max age
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'Max Age Group',(																								
	CASE WHEN POL_RATE_DRVR_MAX_AGE < 50 THEN '50'																								
		 WHEN POL_RATE_DRVR_MAX_AGE <= 59 THEN '50-59' 																								
		 WHEN POL_RATE_DRVR_MAX_AGE <= 69 THEN '50-69' 																								
		 WHEN POL_RATE_DRVR_MAX_AGE <= 79 THEN '70-79' 																								
		 WHEN POL_RATE_DRVR_MAX_AGE <= 130 THEN '80+'
		 ELSE 'Unknown' END)) as Key,
	{query_2_body} {query_2_from}
),

min_age_group AS -- Segmenting indicator counts by State min age
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'Min Age Group',(																								
	CASE WHEN POL_RATE_DRVR_MIN_AGE < 50 THEN '50'																								
		 WHEN POL_RATE_DRVR_MIN_AGE <= 59 THEN '50-59' 																								
		 WHEN POL_RATE_DRVR_MIN_AGE <= 69 THEN '50-69' 																								
		 WHEN POL_RATE_DRVR_MIN_AGE <= 79 THEN '70-79' 																								
		 WHEN POL_RATE_DRVR_MIN_AGE <= 130 THEN '80+'
		 ELSE 'Unknown' END)) as Key,
	{query_2_body} {query_2_from}
),

max_minus_min AS -- Segmenting indicator counts by State max-min age
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'Max Minus Min Age Group',(																								
	CASE WHEN POL_RATE_DRVR_MAX_AGE - POL_RATE_DRVR_MIN_AGE = 0 THEN 'a. 0'
		 WHEN POL_RATE_DRVR_MAX_AGE - POL_RATE_DRVR_MIN_AGE BETWEEN 1 AND 2 THEN 'b. 1-2'
		 WHEN POL_RATE_DRVR_MAX_AGE - POL_RATE_DRVR_MIN_AGE BETWEEN 3 AND 6 THEN 'c. 3-6'
		 WHEN POL_RATE_DRVR_MAX_AGE - POL_RATE_DRVR_MIN_AGE BETWEEN 7 AND 20 THEN 'd. 7-20'
		 WHEN POL_RATE_DRVR_MAX_AGE - POL_RATE_DRVR_MIN_AGE >= 21 THEN 'e. 21+'																							
		 ELSE 'f. n/a' END)) as Key,
	{query_2_body} {query_2_from}	 
),

pcarr_yr_cnt AS -- Segmenting indicator counts by State pcarr year count
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'PCARR YR CNT',(																								
	CASE WHEN PCARR_YR_CNT = 0.5 THEN 'a 0.5'																								
		 WHEN PCARR_YR_CNT = 1.0 THEN 'b 1' 																								
		 WHEN PCARR_YR_CNT = 2.0 THEN 'c 2' 																								
		 WHEN PCARR_YR_CNT = 3.0 THEN 'd 3' 																								
		 WHEN PCARR_YR_CNT = 4.0 THEN 'e 4' 																								
		 WHEN PCARR_YR_CNT = 5.0 THEN 'f 5+' 																								
		 ELSE 'g Unknown' END)) as Key,
	{query_2_body} {query_2_from}
),

account_credit AS -- Segmenting indicator counts by State account credit
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'POL ACCT CR IND', POL_ACCT_CR_IND) as Key,
	{query_2_body} {query_2_from}
),

pol_coverage_pkg AS -- Segmenting indicator counts by State policy coverage package
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'POL COV PKG CD',(																								
	CASE WHEN POL_COV_PKG_CD IS NULL OR POL_COV_PKG_CD = 'NO COV' THEN 'Unknown'
		 ELSE POL_COV_PKG_CD END)) as Key,
	{query_2_body} {query_2_from}
),

pcarr AS -- Segmenting indicator counts by State prior carrier
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'Prior Carrier',(																								
    CASE WHEN PCARR_DESC IN ('STATE FARM INSURANCE') THEN 'STATE FARM'
		 WHEN PCARR_DESC IN ('IMPERIAL FIRE AND CA','SAFE AUTO INS CO','ALLSTATE CTY MUTUAL','ALLSTATE FIRE CAS IN','ALLSTATE INDEMNITY C','ALLSTATE INSURANCE C','ALLSTATE OF NEW JERS','ALLSTATE PROPERTY',
		 'ANJP C INS CO','CENTURY NATIONAL INS','CENTURY-NATIONAL INS','DIRECT GENERAL INS','DIRECT GENERAL INSUR','DIRECT INSURANCE CO','DIRECT INSURANCE COM','DIRECT NATIONAL INSU','ENCOM H A INS COMP',
		 'ENCOM INDEPENDENT IN','ENCOM INS CO OF NJ','ENCOM P C COMP','ENCOMPASS INDEMNITY','ENCOMPASS INS OF AM','ENCOMPASS INSURANCE','ENCOMPASS P C OF NJ','ESURANCE INS CO','ESURANCE INS CO NJ','ESURANCE PROP CAS',
		 'IMPERIAL FIRE CASU','INTEGON CASUALTY INS','INTEGON GENERAL INSU','INTEGON INDEMNITY CO','INTEGON NATIONAL','INTEGON NATIONAL INS','INTEGON PREFERRED IN','MIC GENERAL INSURANC','MS-DIRECT GENERAL IN',
		 'NATIONAL FARMERS UNI','NATIONAL GENERAL ASS','NATIONAL GENERAL INS','NATIONAL GENERAL PRE','NEW SOUTH INSURANCE','NORTHBROOK INSURANCE','STANDARD PROPERTY AN') THEN 'ALLSTATE'
		 WHEN PCARR_DESC IN ('GEICO ADVANTAGE INS','GEICO CASUALTY COMP','GEICO CHOICE INS CO','GEICO COUNTY MTL','GEICO GENERAL INS CO','GEICO INDEMNITY COMP','GEICO SECURE INS CO','GEICOTXCOUNTYMUTUAL',
		 'GOVERNMENT EMP INSCO','SOUTHERN COUNTY') THEN 'GEICO'
		 WHEN PCARR_DESC IN ('PROGRESSIVE GROUPS') THEN 'PROGRESSIVE'
		 WHEN PCARR_DESC IN ('EMPLOYERS INSURANCE','FIRST NAT INS CO AME','GENERAL INS CO AMER','LIB MUTUAL PERSONAL','LIBERTY COUNTY MUTUA','LIBERTY INSURANCE CO','LIBERTY MUTUAL FIRE','LIBERTY MUTUAL INSUR',
		 'LIBERTY MUTUAL MID A','LIBERTY PERSONAL INS','LM GENERAL INSURANCE','LM INSURANCE CORPORA','MERIDIAN SECURITY','MERIDIAN SECURITY IN','MILBANK INSURANCE CO','MONTGOMERY MUTUAL IN','PATRONS MUTUAL INS C',
		 'PEERLESS INDEMNITY I','SAFECO INS CO AMER','SAFECO INSUR CO ILL','SAFECO INSUR CO IND','SAFECO INSUR CO ORE','SAFECO LLOYDS INS CO','SAFECO NAT INSUR CO','SAIC OF OHIO','SAIC OF WISCONSIN','STATE AUTO MUTUAL',
		 'STATE AUTO P C INS','STATE AUTO PROP C','STATE AUTOMOBILE MUT','THE FIRST LIBERTY IN','THE NETHERLANDS INSU','WAUSAU UNDERWRITERS','AMER ECONOMY INS CO','AMER STATES INSUR CO','AMER STATES PREF INS',
		 'AMERICA FIRST INSURA','CONSOLIDATED INSURAN') THEN 'LIBERTY'
		 WHEN PCARR_DESC IN ('FID GUAR INS UNDRS','PHOENIX INSURANCE CO','ST PAUL PROTECTIVE I','STANDARD FIRE INS CO','TRAVCO INSURANCE CO','TRAVELERS CAS CONN','TRAVELERS COMMERCIAL','TRAVELERS HOME MAR',
		 'TRAVELERS INDEM AMER','TRAVELERS INDEM CONN','TRAVELERS INDEMNITY','TRAVELERS P C INS CO','TRAVELERS P C OF AM','TRAVELERS PERSON SEC','TRAVELERS PERSONAL','AUTO INS CO OF HARTF',
		 'CHARTER OAK FIRE INS','FIRST FLORIDIAN A H') THEN 'TRAVELERS'
		 WHEN PCARR_DESC IN ('AMFAM CONNECT INS', 'AMFAM CONNECT PC', 'AMFAM INS CO', 'AMER FAMILY MUTUAL', 'AMERICAN STD INS OH', 'THE GENERAL AUTOMOBI', 'MIDVALE INDEMNITY', 'AFMICSI', 'AMER STANDARD INS',
		 'AMERICAN FAM INS OH',	'AMERIPRISE INSURANCE',	'HOMESITE INSURANCE C',	'IDS PROPERTY CASUALT',	'MSAA',	'MSAA INSURANCE CO','MSAPIC','NGM INSURANCE CO','ODIC') THEN 'AMERICAN FAMILY'
		 WHEN PCARR_DESC IN ('USAA GENERAL IND CO','USAA CASUALTY INS CO','USAA','GARRISON PROP CAS','GAR') THEN 'USAA'																				
		 WHEN PCARR_DESC IN ('COLONIAL COUNTY MUT','ALLIED PROP CAS','ALLIED P C INS CO','COLONIAL COUNTY MUT','CRESTBROOK INSURANCE','DEPOSITORS INS CO',	
		 'NATIONWIDE AG INS CO', 'NATIONWIDE AGRI INS','NATIONWIDE GENERAL','NATIONWIDE INS CO','NATIONWIDE INS OF AM',	'NATIONWIDE MUT FIRE','NATIONWIDE MUT INS','NATIONWIDE MUTUAL IN',
		 'VICTORIA FIRE CASU',	'VICTORIA SPECIALTY I',	'AMCO INSURANCE CO',	'ALLIED P C INS CO',	'HARLEYSVILLEIC',	'HARLEYSVILLEPREFIC',	'NATIONWIDE AFFINITY',
		 'NATIONWIDE GENERAL',	'NATIONWIDE MUTUAL',	'NATIONWIDE PROP CAS',	'VICTORIA SELECT INSU') THEN 'Nationwide'
		 WHEN PCARR_DESC IS NULL  THEN 'NULL' 																								
		 WHEN PCARR_DESC IN ('NOT PRESENTLY INSURED', 'UNKNOWN', 'NO PRIOR INSURANCE') THEN PCARR_DESC
		 WHEN PCARR_DESC IN ( 'OTHER-NON STANDARD' , 'OTHER - NON STANDARD' ) THEN 'OTHER - NON STANDARD'
		 ELSE 'OTHER' END)) as Key,
	{query_2_body} {query_2_from}
),

ins_age AS -- Segmenting indicator counts by State insured age
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'Insured Age Group',(																								
	CASE WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 49 THEN 'a. 25-49'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 54 THEN 'b. 50-54'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 59 THEN 'c. 55-59'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 64 THEN 'd. 60-64'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 69 THEN 'e. 65-69'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 74 THEN 'f. 70-74'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 79 THEN 'g. 75-79'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 120 THEN 'h. 80+'																								
		 ELSE 'i. null' END)) as Key,
	{query_2_body} {query_2_from}
),

ppv_count AS -- Segmenting indicator counts by State PPV count
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'POL PPV CNT',(																								
	CASE WHEN POL_PPV_CNT = 1 THEN 'b 1' 																								
		 WHEN POL_PPV_CNT = 2 THEN 'c 2' 																								
		 WHEN POL_PPV_CNT = 3 THEN 'd 3' 																								
		 WHEN POL_PPV_CNT = 4 THEN 'e 4' 																								
		 WHEN POL_PPV_CNT = 5 THEN 'f 5+' 																								
		 ELSE 'g Unknown' END)) as Key,
	{query_2_body} {query_2_from}
),

telematics AS -- Segmenting indicator counts by State telematics indicator
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'Pol drvr tlmtc enrol ind',Pol_drvr_tlmtc_enrol_ind) as Key,
	{query_2_body} {query_2_from}
),

youthful AS -- Segmenting indicator counts by State youthful indicator
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'YOUTH OPER DRVR IND',YOUTH_OPER_DRVR_IND) as Key,
	{query_2_body} {query_2_from}
),

garage_sub AS -- Segmenting indicator counts by State garage substate
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'GARAGE SUBSTATE',(																								
	CASE WHEN LEFT(RISK_ST_ABBR,2) = LEFT(GARAGE_SUBSTATE,2) THEN GARAGE_SUBSTATE
	ELSE 'Out of State Mail' END)) as Key,
	{query_2_body}
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW" t1
    LEFT JOIN (SELECT AUTO_QTE_TRANS_ID,EXPECTED_LOSS_RATIO FROM "DSC_PLDS_DB"."APP_AUTOMATA_PRD"."PREVAIL_AUTO_ULM_QUOTE_POL_AGG_NB_QUALITY")
    USING (AUTO_QTE_TRANS_ID)
    LEFT JOIN (SELECT state, TO_CHAR(TO_DATE(TRANS_DT, 'YYYY-MM-DD'),'MM-01-YYYY') AS TRANSDT2, AVG(expected_loss_ratio) AS state_avg_lr
               FROM "DSC_PLDS_DB"."APP_AUTOMATA_PRD"."PREVAIL_AUTO_ULM_QUOTE_POL_AGG_NB_QUALITY" WHERE COMPL_QTE_IND =  'Y' GROUP BY state, TRANSDT2) AS t4
	ON t1.RISK_ST_ABBR = t4.state AND TO_CHAR(TRANS_DT,'MM-01-YYYY') = t4.TRANSDT2
	LEFT JOIN (SELECT ZIP_CODE, STATE_ABBR, GARAGE_SUBSTATE FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."STATE_PRODUCT_SUBSTATE") t7
	ON t7.ZIP_CODE = LEFT(MAIL_ADDR_FULL_ZIP_CD,5)	
    WHERE COMPL_QTE_IND = 'Y'
    GROUP BY Key
),

prior_bi AS -- Segmenting indicator counts by State prior BI per person
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'Prior BI Per Person',(																								
	CASE WHEN PCARR_BI_PERS_LMT_CD < 50 THEN 'a. Less than 50'																								
		 WHEN PCARR_BI_PERS_LMT_CD < 100 THEN 'b. 50-99'																								
		 WHEN PCARR_BI_PERS_LMT_CD < 250 THEN 'c. 100-249' 																								
		 WHEN PCARR_BI_PERS_LMT_CD < 3000 THEN 'd. 250+' 																								
		 WHEN PCARR_BI_PERS_LMT_CD IS NULL THEN 'f. Unknown/No Insurance'
		 ELSE 'e. other' END)) AS Key,
	{query_2_body} {query_2_from}
),

aarp_member AS -- Segmenting indicator counts by State AARP member status
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'AARP Member Status',AARP_MEMB_STAT_DESC) as Key,
	{query_2_body} {query_2_from}
),

aqd_group AS -- Segmenting indicator counts by State AQD group
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'Advanced Quote Discount',(																								
    CASE WHEN ADV_QTE_DAY_CNT IN (000,0,999) THEN 'A. 0 Days'																								
		 WHEN ADV_QTE_DAY_CNT <= 13 THEN 'B. 1-13'																								
		 WHEN ADV_QTE_DAY_CNT <= 60 THEN 'C. 14-60'																								
		 WHEN ADV_QTE_DAY_CNT <= 75 THEN 'D. 61-75'																								
		 WHEN ADV_QTE_DAY_CNT <= 99999 THEN 'E. 76+'																								
		 ELSE 'F. Unknown' END)) as Key,
	{query_2_body} {query_2_from}
),

renew_uw AS -- Segmenting indicator counts by State renewal UW tier
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'RENEW UW TIER',(																								
	CASE WHEN RENEW_UW_TIER_CD IN ('BB','BC','BD','BE','BF','BG','BH','BI','BJ','BK') THEN RENEW_UW_TIER_CD
		 ELSE 'Other' END)) as Key,
	{query_2_body} {query_2_from}
),

min_rating_factor AS -- Segmenting indicator counts by State min rating factor
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'Min Rating Factor',POL_MIN_RATE_FCTR_IND) as Key,
	{query_2_body} {query_2_from}
),

max_veh_age AS -- Segmenting indicator counts by State max vehicle age
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'Max Veh Age',(																								
	CASE WHEN POL_MAX_VEH_AGE <= 3 THEN '0-3'
		 WHEN POL_MAX_VEH_AGE <= 6 THEN '4-6'
		 WHEN POL_MAX_VEH_AGE <= 9 THEN '7-9'
		 WHEN POL_MAX_VEH_AGE <= 14 THEN '10-14' 																								
		 WHEN POL_MAX_VEH_AGE <= 19 THEN '15-19'
		 WHEN POL_MAX_VEH_AGE <= 24 THEN '20-24' 																								
		 WHEN POL_MAX_VEH_AGE > 24 THEN '25+'  																								
		 ELSE 'Other' END)) as Key,
	{query_2_body} {query_2_from}
),

pol_mult_single AS -- Segmenting indicator counts by State multi-single vehicle code
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'POL MULTI SNGL VEH CD',(																								
	CASE WHEN POL_MULTI_SNGL_VEH_CD IS NULL THEN 'S'
		 ELSE POL_MULTI_SNGL_VEH_CD END)) as Key,
	{query_2_body} {query_2_from}
),

mature_driving_course AS -- Segmenting indicator counts by State mature driving indicator
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'Mature Driving Course',(																								
	CASE WHEN POL_MATUR_DRVNG_COURS_CR_IND = 'Y' THEN 'Y'
		 ELSE 'N' END)) as Key,
	{query_2_body} {query_2_from}
),

incident_free AS -- Segmenting indicator counts by State incident free year count
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'Incident Free Year Count',(																								
	CASE WHEN POL_INCID_FREE_YR_CNT = 0 THEN '0'																								
		 WHEN POL_INCID_FREE_YR_CNT = 1 THEN '1'																								
		 WHEN POL_INCID_FREE_YR_CNT = 2 THEN '2' 																								
		 WHEN POL_INCID_FREE_YR_CNT = 3 THEN '3' 																								
		 WHEN POL_INCID_FREE_YR_CNT = 4 THEN '4' 																								
		 WHEN POL_INCID_FREE_YR_CNT = 5 THEN '5' END)) as Key,
	{query_2_body} {query_2_from}
),

naf_accidents AS -- Segmenting indicator counts by State NAF accident count
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'NAF Accident Count',(																								
	CASE WHEN PCARR_POL_NAF_ACCID_CNT = 0 THEN '0'																								
		 WHEN PCARR_POL_NAF_ACCID_CNT = 1 THEN '1'																								
		 WHEN PCARR_POL_NAF_ACCID_CNT > 1 THEN '2+' END)) as Key,
	{query_2_body} {query_2_from}
),

incident_mo_age AS -- Segmenting indicator counts by State incident month age
(
	SELECT CONCAT(RISK_ST_ABBR,BU_ABBR,'Incident Month Age',(																								
	CASE WHEN POL_INCID_MO_AGE <= 12 THEN '1-12'																								
		 WHEN POL_INCID_MO_AGE <= 24 THEN '13-24'																								
		 WHEN POL_INCID_MO_AGE <= 36 THEN '25-36'																								
		 WHEN POL_INCID_MO_AGE <= 48 THEN '37-48'																								
		 WHEN POL_INCID_MO_AGE <= 60 THEN '49-60' 																								
		 WHEN POL_INCID_MO_AGE = 61 THEN '61' END)) as Key,
	{query_2_body} {query_2_from}
)

SELECT * FROM
(
	SELECT * FROM ins_score
	UNION ALL
	SELECT * FROM household_comp
	UNION ALL
	SELECT * FROM clean_dirty
	UNION ALL
	SELECT * FROM max_age_group
	UNION ALL
	SELECT * FROM min_age_group
	UNION ALL
	SELECT * FROM max_minus_min
	UNION ALL
	SELECT * FROM pcarr_yr_cnt
	UNION ALL
	SELECT * FROM account_credit
	UNION ALL
	SELECT * FROM pol_coverage_pkg
	UNION ALL
	SELECT * FROM pcarr
	UNION ALL
	SELECT * FROM ins_age
	UNION ALL
	SELECT * FROM ppv_count
	UNION ALL
	SELECT * FROM telematics
	UNION ALL
	SELECT * FROM youthful
	UNION ALL
	SELECT * FROM garage_sub
	UNION ALL
	SELECT * FROM prior_bi
	UNION ALL
	SELECT * FROM aarp_member
	UNION ALL
	SELECT * FROM aqd_group
	UNION ALL
	SELECT * FROM renew_uw
	UNION ALL
	SELECT * FROM min_rating_factor
	UNION ALL
	SELECT * FROM max_veh_age
	UNION ALL
	SELECT * FROM pol_mult_single
	UNION ALL
	SELECT * FROM mature_driving_course
	UNION ALL
	SELECT * FROM incident_free
	UNION ALL
	SELECT * FROM naf_accidents
	UNION ALL
	SELECT * FROM incident_mo_age
)
ORDER BY KEY
"""

In [7]:
cs = ctx.cursor()
try:
    cs.execute(auto_seg_state)
finally:
    cs.close()
print('Done')

Done


## 3. Auto Persistency Table Creation (DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_AUTO_PERSISTENCY_EXCEL)

In [8]:
auto_persistency = """
CREATE OR REPLACE TRANSIENT TABLE DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_AUTO_PERSISTENCY_EXCEL AS

WITH state_e_30_nb AS -- Persistency calculations for State E+30 NB
(
    SELECT CONCAT(RISK_STATE,BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+30 NB') as Key,
    perst_calc_scorecard(18, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "18",
    perst_calc_scorecard(17, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "17",
    perst_calc_scorecard(16, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "16",
    perst_calc_scorecard(15, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "15",
    perst_calc_scorecard(14, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "14",
    perst_calc_scorecard(13, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "13",
    perst_calc_scorecard(12, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "12",
    perst_calc_scorecard(11, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "11",
    perst_calc_scorecard(10, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "10",
    perst_calc_scorecard(9, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "9",
    perst_calc_scorecard(8, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "8",
    perst_calc_scorecard(7, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "7",
    perst_calc_scorecard(6, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "6",
    perst_calc_scorecard(5, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "5",
    perst_calc_scorecard(4, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "4",
    perst_calc_scorecard(3, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "3",
    perst_calc_scorecard(2, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "2",
    perst_calc_scorecard(1, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "1",
    perst_calc_scorecard(0, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
    WHERE POL_NEW_RENEW_CD = 'N' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
    GROUP BY Key
),

region_e_30_nb AS -- Persistency calculations for Region E+30 NB
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_STATE),BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+30 NB') as Key,
    perst_calc_scorecard(18, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "18",
    perst_calc_scorecard(17, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "17",
    perst_calc_scorecard(16, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "16",
    perst_calc_scorecard(15, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "15",
    perst_calc_scorecard(14, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "14",
    perst_calc_scorecard(13, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "13",
    perst_calc_scorecard(12, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "12",
    perst_calc_scorecard(11, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "11",
    perst_calc_scorecard(10, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "10",
    perst_calc_scorecard(9, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "9",
    perst_calc_scorecard(8, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "8",
    perst_calc_scorecard(7, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "7",
    perst_calc_scorecard(6, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "6",
    perst_calc_scorecard(5, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "5",
    perst_calc_scorecard(4, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "4",
    perst_calc_scorecard(3, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "3",
    perst_calc_scorecard(2, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "2",
    perst_calc_scorecard(1, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "1",
    perst_calc_scorecard(0, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
    WHERE POL_NEW_RENEW_CD = 'N' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
    GROUP BY Key
),

cw_e_30_nb AS -- Persistency calculations for Countrywide E+30 NB
(
	SELECT CONCAT('CW',BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+30 NB') as Key,
    perst_calc_scorecard(18, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "18",
    perst_calc_scorecard(17, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "17",
    perst_calc_scorecard(16, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "16",
    perst_calc_scorecard(15, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "15",
    perst_calc_scorecard(14, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "14",
    perst_calc_scorecard(13, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "13",
    perst_calc_scorecard(12, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "12",
    perst_calc_scorecard(11, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "11",
    perst_calc_scorecard(10, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "10",
    perst_calc_scorecard(9, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "9",
    perst_calc_scorecard(8, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "8",
    perst_calc_scorecard(7, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "7",
    perst_calc_scorecard(6, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "6",
    perst_calc_scorecard(5, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "5",
    perst_calc_scorecard(4, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "4",
    perst_calc_scorecard(3, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "3",
    perst_calc_scorecard(2, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "2",
    perst_calc_scorecard(1, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "1",
    perst_calc_scorecard(0, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
    WHERE POL_NEW_RENEW_CD = 'N' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
    GROUP BY Key
),

state_e_60_nb AS -- Persistency calculations for State E+60 NB
(
    SELECT CONCAT(RISK_STATE,BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+60 NB') as Key,
    perst_calc_scorecard(18, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "18",
    perst_calc_scorecard(17, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "17",
    perst_calc_scorecard(16, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "16",
    perst_calc_scorecard(15, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "15",
    perst_calc_scorecard(14, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "14",
    perst_calc_scorecard(13, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "13",
    perst_calc_scorecard(12, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "12",
    perst_calc_scorecard(11, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "11",
    perst_calc_scorecard(10, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "10",
    perst_calc_scorecard(9, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "9",
    perst_calc_scorecard(8, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "8",
    perst_calc_scorecard(7, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "7",
    perst_calc_scorecard(6, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "6",
    perst_calc_scorecard(5, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "5",
    perst_calc_scorecard(4, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "4",
    perst_calc_scorecard(3, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "3",
    perst_calc_scorecard(2, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "2",
    perst_calc_scorecard(1, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "1",
    perst_calc_scorecard(0, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
    WHERE POL_NEW_RENEW_CD = 'N' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
    GROUP BY Key
),

region_e_60_nb AS -- Persistency calculations for Region E+60 NB
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_STATE),BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+60 NB') as Key,
    perst_calc_scorecard(18, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "18",
    perst_calc_scorecard(17, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "17",
    perst_calc_scorecard(16, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "16",
    perst_calc_scorecard(15, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "15",
    perst_calc_scorecard(14, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "14",
    perst_calc_scorecard(13, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "13",
    perst_calc_scorecard(12, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "12",
    perst_calc_scorecard(11, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "11",
    perst_calc_scorecard(10, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "10",
    perst_calc_scorecard(9, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "9",
    perst_calc_scorecard(8, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "8",
    perst_calc_scorecard(7, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "7",
    perst_calc_scorecard(6, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "6",
    perst_calc_scorecard(5, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "5",
    perst_calc_scorecard(4, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "4",
    perst_calc_scorecard(3, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "3",
    perst_calc_scorecard(2, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "2",
    perst_calc_scorecard(1, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "1",
    perst_calc_scorecard(0, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
    WHERE POL_NEW_RENEW_CD = 'N' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
    GROUP BY Key
),

cw_e_60_nb AS -- Persistency calculations for Countrywide E+60 NB
(
	SELECT CONCAT('CW',BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+60 NB') as Key,
    perst_calc_scorecard(18, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "18",
    perst_calc_scorecard(17, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "17",
    perst_calc_scorecard(16, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "16",
    perst_calc_scorecard(15, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "15",
    perst_calc_scorecard(14, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "14",
    perst_calc_scorecard(13, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "13",
    perst_calc_scorecard(12, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "12",
    perst_calc_scorecard(11, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "11",
    perst_calc_scorecard(10, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "10",
    perst_calc_scorecard(9, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "9",
    perst_calc_scorecard(8, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "8",
    perst_calc_scorecard(7, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "7",
    perst_calc_scorecard(6, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "6",
    perst_calc_scorecard(5, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "5",
    perst_calc_scorecard(4, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "4",
    perst_calc_scorecard(3, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "3",
    perst_calc_scorecard(2, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "2",
    perst_calc_scorecard(1, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "1",
    perst_calc_scorecard(0, POL_EFF_DT, CANCEL_E_60, ATR_E_60) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
    WHERE POL_NEW_RENEW_CD = 'N' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
    GROUP BY Key
),

state_e_90_nb AS -- Persistency calculations for State E+90 NB
(
    SELECT CONCAT(RISK_STATE,BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+90 NB') as Key,
    perst_calc_scorecard(18, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "18",
    perst_calc_scorecard(17, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "17",
    perst_calc_scorecard(16, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "16",
    perst_calc_scorecard(15, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "15",
    perst_calc_scorecard(14, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "14",
    perst_calc_scorecard(13, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "13",
    perst_calc_scorecard(12, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "12",
    perst_calc_scorecard(11, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "11",
    perst_calc_scorecard(10, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "10",
    perst_calc_scorecard(9, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "9",
    perst_calc_scorecard(8, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "8",
    perst_calc_scorecard(7, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "7",
    perst_calc_scorecard(6, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "6",
    perst_calc_scorecard(5, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "5",
    perst_calc_scorecard(4, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "4",
    perst_calc_scorecard(3, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "3",
    perst_calc_scorecard(2, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "2",
    perst_calc_scorecard(1, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "1",
    perst_calc_scorecard(0, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
    WHERE POL_NEW_RENEW_CD = 'N' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
    GROUP BY Key
),

region_e_90_nb AS -- Persistency calculations for Region E+90 NB
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_STATE),BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+90 NB') as Key,
    perst_calc_scorecard(18, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "18",
    perst_calc_scorecard(17, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "17",
    perst_calc_scorecard(16, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "16",
    perst_calc_scorecard(15, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "15",
    perst_calc_scorecard(14, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "14",
    perst_calc_scorecard(13, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "13",
    perst_calc_scorecard(12, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "12",
    perst_calc_scorecard(11, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "11",
    perst_calc_scorecard(10, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "10",
    perst_calc_scorecard(9, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "9",
    perst_calc_scorecard(8, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "8",
    perst_calc_scorecard(7, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "7",
    perst_calc_scorecard(6, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "6",
    perst_calc_scorecard(5, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "5",
    perst_calc_scorecard(4, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "4",
    perst_calc_scorecard(3, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "3",
    perst_calc_scorecard(2, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "2",
    perst_calc_scorecard(1, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "1",
    perst_calc_scorecard(0, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
    WHERE POL_NEW_RENEW_CD = 'N' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
    GROUP BY Key
),

cw_e_90_nb AS -- Persistency calculations for Countrywide E+90 NB
(
	SELECT CONCAT('CW',BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+90 NB') as Key,
    perst_calc_scorecard(18, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "18",
    perst_calc_scorecard(17, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "17",
    perst_calc_scorecard(16, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "16",
    perst_calc_scorecard(15, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "15",
    perst_calc_scorecard(14, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "14",
    perst_calc_scorecard(13, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "13",
    perst_calc_scorecard(12, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "12",
    perst_calc_scorecard(11, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "11",
    perst_calc_scorecard(10, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "10",
    perst_calc_scorecard(9, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "9",
    perst_calc_scorecard(8, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "8",
    perst_calc_scorecard(7, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "7",
    perst_calc_scorecard(6, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "6",
    perst_calc_scorecard(5, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "5",
    perst_calc_scorecard(4, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "4",
    perst_calc_scorecard(3, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "3",
    perst_calc_scorecard(2, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "2",
    perst_calc_scorecard(1, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "1",
    perst_calc_scorecard(0, POL_EFF_DT, CANCEL_E_90, ATR_E_90) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
    WHERE POL_NEW_RENEW_CD = 'N' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
    GROUP BY Key
),

state_pol_count AS -- Policy Inforce Counts by State
(
	SELECT CONCAT(STATE_ABBR,BUS_UNIT_ABBR,NEWCO_IND,'Inforce') as Key,
	zero_cond_scorecard(18, ME_DT, POL_INFORCE_CNT) AS "18",
	zero_cond_scorecard(17, ME_DT, POL_INFORCE_CNT) AS "17",
	zero_cond_scorecard(16, ME_DT, POL_INFORCE_CNT) AS "16",
	zero_cond_scorecard(15, ME_DT, POL_INFORCE_CNT) AS "15",
	zero_cond_scorecard(14, ME_DT, POL_INFORCE_CNT) AS "14",
	zero_cond_scorecard(13, ME_DT, POL_INFORCE_CNT) AS "13",
	zero_cond_scorecard(12, ME_DT, POL_INFORCE_CNT) AS "12",
	zero_cond_scorecard(11, ME_DT, POL_INFORCE_CNT) AS "11",
	zero_cond_scorecard(10, ME_DT, POL_INFORCE_CNT) AS "10",
	zero_cond_scorecard(9, ME_DT, POL_INFORCE_CNT) AS "9",
	zero_cond_scorecard(8, ME_DT, POL_INFORCE_CNT) AS "8",
	zero_cond_scorecard(7, ME_DT, POL_INFORCE_CNT) AS "7",
	zero_cond_scorecard(6, ME_DT, POL_INFORCE_CNT) AS "6",
	zero_cond_scorecard(5, ME_DT, POL_INFORCE_CNT) AS "5",
	zero_cond_scorecard(4, ME_DT, POL_INFORCE_CNT) AS "4",
	zero_cond_scorecard(3, ME_DT, POL_INFORCE_CNT) AS "3",
	zero_cond_scorecard(2, ME_DT, POL_INFORCE_CNT) AS "2",
	zero_cond_scorecard(1, ME_DT, POL_INFORCE_CNT) AS "1",
	zero_cond_scorecard(0, ME_DT, POL_INFORCE_CNT) AS "0",
	TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND"
	WHERE BUS_UNIT_ABBR IN ('AARP','AGCY') 
	AND CONCAT(STATE_ABBR,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG','MDHDC1',
	'MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1','NJNJRP',
	'NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
),

state_wp AS -- Written Premium by State
(
	SELECT CONCAT(STATE_ABBR,BUS_UNIT_ABBR,NEWCO_IND,'Written Premium') as Key,
	zero_cond_scorecard(18, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "18",
	zero_cond_scorecard(17, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "17",
	zero_cond_scorecard(16, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "16",
	zero_cond_scorecard(15, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "15",
	zero_cond_scorecard(14, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "14",
	zero_cond_scorecard(13, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "13",
	zero_cond_scorecard(12, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "12",
	zero_cond_scorecard(11, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "11",
	zero_cond_scorecard(10, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "10",
	zero_cond_scorecard(9, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "9",
	zero_cond_scorecard(8, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "8",
	zero_cond_scorecard(7, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "7",
	zero_cond_scorecard(6, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "6",
	zero_cond_scorecard(5, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "5",
	zero_cond_scorecard(4, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "4",
	zero_cond_scorecard(3, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "3",
	zero_cond_scorecard(2, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "2",
	zero_cond_scorecard(1, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "1",
	zero_cond_scorecard(0, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "0",
	TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND"
	WHERE BUS_UNIT_ABBR IN ('AARP','AGCY') 
	AND CONCAT(STATE_ABBR,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG','MDHDC1',
	'MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1','NJNJRP',
	'NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
),

cw_pol_count AS -- CW Policy Inforce Counts
(
	SELECT CONCAT('CW',BUS_UNIT_ABBR,NEWCO_IND,'Inforce') as Key,
	zero_cond_scorecard(18, ME_DT, POL_INFORCE_CNT) AS "18",
	zero_cond_scorecard(17, ME_DT, POL_INFORCE_CNT) AS "17",
	zero_cond_scorecard(16, ME_DT, POL_INFORCE_CNT) AS "16",
	zero_cond_scorecard(15, ME_DT, POL_INFORCE_CNT) AS "15",
	zero_cond_scorecard(14, ME_DT, POL_INFORCE_CNT) AS "14",
	zero_cond_scorecard(13, ME_DT, POL_INFORCE_CNT) AS "13",
	zero_cond_scorecard(12, ME_DT, POL_INFORCE_CNT) AS "12",
	zero_cond_scorecard(11, ME_DT, POL_INFORCE_CNT) AS "11",
	zero_cond_scorecard(10, ME_DT, POL_INFORCE_CNT) AS "10",
	zero_cond_scorecard(9, ME_DT, POL_INFORCE_CNT) AS "9",
	zero_cond_scorecard(8, ME_DT, POL_INFORCE_CNT) AS "8",
	zero_cond_scorecard(7, ME_DT, POL_INFORCE_CNT) AS "7",
	zero_cond_scorecard(6, ME_DT, POL_INFORCE_CNT) AS "6",
	zero_cond_scorecard(5, ME_DT, POL_INFORCE_CNT) AS "5",
	zero_cond_scorecard(4, ME_DT, POL_INFORCE_CNT) AS "4",
	zero_cond_scorecard(3, ME_DT, POL_INFORCE_CNT) AS "3",
	zero_cond_scorecard(2, ME_DT, POL_INFORCE_CNT) AS "2",
	zero_cond_scorecard(1, ME_DT, POL_INFORCE_CNT) AS "1",
	zero_cond_scorecard(0, ME_DT, POL_INFORCE_CNT) AS "0",
	TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND"
	WHERE BUS_UNIT_ABBR IN ('AARP','AGCY') 
	AND CONCAT(STATE_ABBR,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG','MDHDC1',
	'MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1','NJNJRP',
	'NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
),

cw_wp AS -- CW Written Premium
(
	SELECT CONCAT('CW',BUS_UNIT_ABBR,NEWCO_IND,'Written Premium') as Key,
	zero_cond_scorecard(18, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "18",
	zero_cond_scorecard(17, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "17",
	zero_cond_scorecard(16, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "16",
	zero_cond_scorecard(15, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "15",
	zero_cond_scorecard(14, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "14",
	zero_cond_scorecard(13, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "13",
	zero_cond_scorecard(12, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "12",
	zero_cond_scorecard(11, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "11",
	zero_cond_scorecard(10, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "10",
	zero_cond_scorecard(9, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "9",
	zero_cond_scorecard(8, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "8",
	zero_cond_scorecard(7, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "7",
	zero_cond_scorecard(6, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "6",
	zero_cond_scorecard(5, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "5",
	zero_cond_scorecard(4, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "4",
	zero_cond_scorecard(3, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "3",
	zero_cond_scorecard(2, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "2",
	zero_cond_scorecard(1, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "1",
	zero_cond_scorecard(0, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "0",
	TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND"
	WHERE BUS_UNIT_ABBR IN ('AARP','AGCY') 
	AND CONCAT(STATE_ABBR,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG','MDHDC1',
	'MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1','NJNJRP',
	'NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
),

region_pol_count AS -- Regional Policy Inforce Counts
(
	SELECT CONCAT(GET_REGION_SCORECARD(STATE_ABBR),BUS_UNIT_ABBR,NEWCO_IND,'Inforce') as Key,
	zero_cond_scorecard(18, ME_DT, POL_INFORCE_CNT) AS "18",
	zero_cond_scorecard(17, ME_DT, POL_INFORCE_CNT) AS "17",
	zero_cond_scorecard(16, ME_DT, POL_INFORCE_CNT) AS "16",
	zero_cond_scorecard(15, ME_DT, POL_INFORCE_CNT) AS "15",
	zero_cond_scorecard(14, ME_DT, POL_INFORCE_CNT) AS "14",
	zero_cond_scorecard(13, ME_DT, POL_INFORCE_CNT) AS "13",
	zero_cond_scorecard(12, ME_DT, POL_INFORCE_CNT) AS "12",
	zero_cond_scorecard(11, ME_DT, POL_INFORCE_CNT) AS "11",
	zero_cond_scorecard(10, ME_DT, POL_INFORCE_CNT) AS "10",
	zero_cond_scorecard(9, ME_DT, POL_INFORCE_CNT) AS "9",
	zero_cond_scorecard(8, ME_DT, POL_INFORCE_CNT) AS "8",
	zero_cond_scorecard(7, ME_DT, POL_INFORCE_CNT) AS "7",
	zero_cond_scorecard(6, ME_DT, POL_INFORCE_CNT) AS "6",
	zero_cond_scorecard(5, ME_DT, POL_INFORCE_CNT) AS "5",
	zero_cond_scorecard(4, ME_DT, POL_INFORCE_CNT) AS "4",
	zero_cond_scorecard(3, ME_DT, POL_INFORCE_CNT) AS "3",
	zero_cond_scorecard(2, ME_DT, POL_INFORCE_CNT) AS "2",
	zero_cond_scorecard(1, ME_DT, POL_INFORCE_CNT) AS "1",
	zero_cond_scorecard(0, ME_DT, POL_INFORCE_CNT) AS "0",
	TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND"
	WHERE BUS_UNIT_ABBR IN ('AARP','AGCY') 
	AND CONCAT(STATE_ABBR,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG','MDHDC1',
	'MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1','NJNJRP',
	'NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
),

region_wp AS -- Regional Written Premium
(
	SELECT CONCAT(GET_REGION_SCORECARD(STATE_ABBR),BUS_UNIT_ABBR,NEWCO_IND,'Written Premium') as Key,
	zero_cond_scorecard(18, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "18",
	zero_cond_scorecard(17, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "17",
	zero_cond_scorecard(16, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "16",
	zero_cond_scorecard(15, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "15",
	zero_cond_scorecard(14, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "14",
	zero_cond_scorecard(13, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "13",
	zero_cond_scorecard(12, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "12",
	zero_cond_scorecard(11, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "11",
	zero_cond_scorecard(10, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "10",
	zero_cond_scorecard(9, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "9",
	zero_cond_scorecard(8, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "8",
	zero_cond_scorecard(7, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "7",
	zero_cond_scorecard(6, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "6",
	zero_cond_scorecard(5, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "5",
	zero_cond_scorecard(4, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "4",
	zero_cond_scorecard(3, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "3",
	zero_cond_scorecard(2, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "2",
	zero_cond_scorecard(1, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "1",
	zero_cond_scorecard(0, ME_DT, POL_ANNL_TTL_BILL_PREM_AMT) AS "0",
	TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
	FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND"
	WHERE BUS_UNIT_ABBR IN ('AARP','AGCY') 
	AND CONCAT(STATE_ABBR,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG','MDHDC1',
	'MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1','NJNJRP',
	'NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
),

state_e_30_r AS -- Persistency calculations for State E+30 R
(
    SELECT CONCAT(RISK_STATE,BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+30 R') as Key,
    perst_calc_scorecard(18, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "18",
    perst_calc_scorecard(17, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "17",
    perst_calc_scorecard(16, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "16",
    perst_calc_scorecard(15, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "15",
    perst_calc_scorecard(14, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "14",
    perst_calc_scorecard(13, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "13",
    perst_calc_scorecard(12, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "12",
    perst_calc_scorecard(11, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "11",
    perst_calc_scorecard(10, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "10",
    perst_calc_scorecard(9, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "9",
    perst_calc_scorecard(8, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "8",
    perst_calc_scorecard(7, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "7",
    perst_calc_scorecard(6, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "6",
    perst_calc_scorecard(5, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "5",
    perst_calc_scorecard(4, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "4",
    perst_calc_scorecard(3, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "3",
    perst_calc_scorecard(2, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "2",
    perst_calc_scorecard(1, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "1",
    perst_calc_scorecard(0, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
	WHERE POL_NEW_RENEW_CD = 'R' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
	AND CONCAT(RISK_STATE,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG',
	'MDHDC1','MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1',
	'NJNJRP','NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
),

state_e_0_r AS -- Persistency calculations for State E+0 R
(
    SELECT CONCAT(RISK_STATE,BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+0 R') as Key,
    perst_calc_scorecard(18, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "18",
    perst_calc_scorecard(17, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "17",
    perst_calc_scorecard(16, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "16",
    perst_calc_scorecard(15, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "15",
    perst_calc_scorecard(14, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "14",
    perst_calc_scorecard(13, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "13",
    perst_calc_scorecard(12, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "12",
    perst_calc_scorecard(11, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "11",
    perst_calc_scorecard(10, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "10",
    perst_calc_scorecard(9, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "9",
    perst_calc_scorecard(8, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "8",
    perst_calc_scorecard(7, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "7",
    perst_calc_scorecard(6, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "6",
    perst_calc_scorecard(5, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "5",
    perst_calc_scorecard(4, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "4",
    perst_calc_scorecard(3, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "3",
    perst_calc_scorecard(2, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "2",
    perst_calc_scorecard(1, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "1",
    perst_calc_scorecard(0, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
	WHERE POL_NEW_RENEW_CD = 'R' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
	AND CONCAT(RISK_STATE,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG',
	'MDHDC1','MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1',
	'NJNJRP','NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
),

state_e_ult_r AS -- Persistency calculations for State E+Ult R
(
    SELECT CONCAT(RISK_STATE,BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+Ult R') as Key,
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 18 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 18 THEN ULT_ATR_CNT ELSE 0 END)) AS "18",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 17 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 17 THEN ULT_ATR_CNT ELSE 0 END)) AS "17",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 16 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 16 THEN ULT_ATR_CNT ELSE 0 END)) AS "16",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 15 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 15 THEN ULT_ATR_CNT ELSE 0 END)) AS "15",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 14 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 14 THEN ULT_ATR_CNT ELSE 0 END)) AS "14",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 13 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 13 THEN ULT_ATR_CNT ELSE 0 END)) AS "13",					 
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 12 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 12 THEN ULT_ATR_CNT ELSE 0 END)) AS "12",			 
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 11 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 11 THEN ULT_ATR_CNT ELSE 0 END)) AS "11",			 
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 10 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 10 THEN ULT_ATR_CNT ELSE 0 END)) AS "10",			 
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 9 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 9 THEN ULT_ATR_CNT ELSE 0 END)) AS "9",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 8 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 8 THEN ULT_ATR_CNT ELSE 0 END)) AS "8",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 7 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 7 THEN ULT_ATR_CNT ELSE 0 END)) AS "7",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 6 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 6 THEN ULT_ATR_CNT ELSE 0 END)) AS "6",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 5 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 5 THEN ULT_ATR_CNT ELSE 0 END)) AS "5",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 4 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 4 THEN ULT_ATR_CNT ELSE 0 END)) AS "4",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 3 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 3 THEN ULT_ATR_CNT ELSE 0 END)) AS "3",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 2 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 2 THEN ULT_ATR_CNT ELSE 0 END)) AS "2",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 1 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 1 THEN ULT_ATR_CNT ELSE 0 END)) AS "1",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 0 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 0 THEN ULT_ATR_CNT ELSE 0 END)) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
	WHERE POL_NEW_RENEW_CD = 'R' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
	AND CONCAT(RISK_STATE,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG',
	'MDHDC1','MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1',
	'NJNJRP','NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
),

state_prem_change AS -- Premium Changes by State
(
    SELECT CONCAT(RISK_STATE,BUS_UNIT_ABBR,NEWCO_INDICATOR,'PremiumChange') as Key,
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 18 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 18 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "18",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 17 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 17 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "17",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 16 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 16 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "16",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 15 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 15 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "15",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 14 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 14 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "14",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 13 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 13 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "13",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 12 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 12 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "12",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 11 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 11 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "11",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 10 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 10 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "10",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 9 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 9 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "9",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 8 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 8 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "8",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 7 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 7 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "7",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 6 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 6 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "6",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 5 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 5 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "5",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 4 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 4 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "4",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 3 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 3 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "3",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 2 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 2 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "2",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 1 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 1 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "1",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 0 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 0 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
	WHERE POL_NEW_RENEW_CD = 'R' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
	AND CONCAT(RISK_STATE,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG',
	'MDHDC1','MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1',
	'NJNJRP','NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
),

cw_e_30_r AS -- Persistency calculations for CW E+30 R
(
    SELECT CONCAT('CW',BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+30 R') as Key,
    perst_calc_scorecard(18, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "18",
    perst_calc_scorecard(17, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "17",
    perst_calc_scorecard(16, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "16",
    perst_calc_scorecard(15, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "15",
    perst_calc_scorecard(14, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "14",
    perst_calc_scorecard(13, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "13",
    perst_calc_scorecard(12, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "12",
    perst_calc_scorecard(11, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "11",
    perst_calc_scorecard(10, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "10",
    perst_calc_scorecard(9, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "9",
    perst_calc_scorecard(8, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "8",
    perst_calc_scorecard(7, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "7",
    perst_calc_scorecard(6, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "6",
    perst_calc_scorecard(5, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "5",
    perst_calc_scorecard(4, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "4",
    perst_calc_scorecard(3, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "3",
    perst_calc_scorecard(2, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "2",
    perst_calc_scorecard(1, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "1",
    perst_calc_scorecard(0, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
	WHERE POL_NEW_RENEW_CD = 'R' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
	AND CONCAT(RISK_STATE,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG',
	'MDHDC1','MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1',
	'NJNJRP','NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
),

cw_e_0_r AS -- Persistency calculations for CW E+0 R
(
    SELECT CONCAT('CW',BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+0 R') as Key,
    perst_calc_scorecard(18, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "18",
    perst_calc_scorecard(17, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "17",
    perst_calc_scorecard(16, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "16",
    perst_calc_scorecard(15, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "15",
    perst_calc_scorecard(14, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "14",
    perst_calc_scorecard(13, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "13",
    perst_calc_scorecard(12, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "12",
    perst_calc_scorecard(11, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "11",
    perst_calc_scorecard(10, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "10",
    perst_calc_scorecard(9, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "9",
    perst_calc_scorecard(8, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "8",
    perst_calc_scorecard(7, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "7",
    perst_calc_scorecard(6, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "6",
    perst_calc_scorecard(5, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "5",
    perst_calc_scorecard(4, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "4",
    perst_calc_scorecard(3, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "3",
    perst_calc_scorecard(2, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "2",
    perst_calc_scorecard(1, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "1",
    perst_calc_scorecard(0, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
	WHERE POL_NEW_RENEW_CD = 'R' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
	AND CONCAT(RISK_STATE,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG',
	'MDHDC1','MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1',
	'NJNJRP','NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
),

cw_e_ult_r AS -- Persistency calculations for CW E+Ult R
(
    SELECT CONCAT('CW',BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+Ult R') as Key,
    1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 18 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 18 THEN ULT_ATR_CNT ELSE 0 END)) AS "18",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 17 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 17 THEN ULT_ATR_CNT ELSE 0 END)) AS "17",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 16 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 16 THEN ULT_ATR_CNT ELSE 0 END)) AS "16",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 15 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 15 THEN ULT_ATR_CNT ELSE 0 END)) AS "15",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 14 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 14 THEN ULT_ATR_CNT ELSE 0 END)) AS "14",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 13 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 13 THEN ULT_ATR_CNT ELSE 0 END)) AS "13",					 
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 12 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 12 THEN ULT_ATR_CNT ELSE 0 END)) AS "12",			 
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 11 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 11 THEN ULT_ATR_CNT ELSE 0 END)) AS "11",			 
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 10 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 10 THEN ULT_ATR_CNT ELSE 0 END)) AS "10",			 
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 9 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 9 THEN ULT_ATR_CNT ELSE 0 END)) AS "9",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 8 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 8 THEN ULT_ATR_CNT ELSE 0 END)) AS "8",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 7 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 7 THEN ULT_ATR_CNT ELSE 0 END)) AS "7",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 6 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 6 THEN ULT_ATR_CNT ELSE 0 END)) AS "6",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 5 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 5 THEN ULT_ATR_CNT ELSE 0 END)) AS "5",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 4 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 4 THEN ULT_ATR_CNT ELSE 0 END)) AS "4",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 3 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 3 THEN ULT_ATR_CNT ELSE 0 END)) AS "3",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 2 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 2 THEN ULT_ATR_CNT ELSE 0 END)) AS "2",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 1 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 1 THEN ULT_ATR_CNT ELSE 0 END)) AS "1",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 0 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 0 THEN ULT_ATR_CNT ELSE 0 END)) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
	WHERE POL_NEW_RENEW_CD = 'R' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
	AND CONCAT(RISK_STATE,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG',
	'MDHDC1','MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1',
	'NJNJRP','NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
),

cw_prem_change AS -- CW Premium Changes
(
    SELECT CONCAT('CW',BUS_UNIT_ABBR,NEWCO_INDICATOR,'PremiumChange') as Key,
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 18 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 18 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "18",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 17 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 17 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "17",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 16 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 16 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "16",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 15 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 15 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "15",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 14 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 14 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "14",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 13 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 13 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "13",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 12 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 12 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "12",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 11 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 11 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "11",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 10 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 10 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "10",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 9 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 9 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "9",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 8 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 8 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "8",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 7 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 7 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "7",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 6 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 6 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "6",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 5 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 5 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "5",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 4 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 4 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "4",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 3 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 3 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "3",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 2 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 2 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "2",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 1 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 1 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "1",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 0 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 0 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
	WHERE POL_NEW_RENEW_CD = 'R' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
	AND CONCAT(RISK_STATE,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG',
	'MDHDC1','MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1',
	'NJNJRP','NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
),

region_e_30_r AS -- Persistency calculations for Regional E+30 R
(
    SELECT CONCAT(GET_REGION_SCORECARD(RISK_STATE),BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+30 R') as Key,
    perst_calc_scorecard(18, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "18",
    perst_calc_scorecard(17, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "17",
    perst_calc_scorecard(16, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "16",
    perst_calc_scorecard(15, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "15",
    perst_calc_scorecard(14, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "14",
    perst_calc_scorecard(13, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "13",
    perst_calc_scorecard(12, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "12",
    perst_calc_scorecard(11, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "11",
    perst_calc_scorecard(10, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "10",
    perst_calc_scorecard(9, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "9",
    perst_calc_scorecard(8, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "8",
    perst_calc_scorecard(7, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "7",
    perst_calc_scorecard(6, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "6",
    perst_calc_scorecard(5, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "5",
    perst_calc_scorecard(4, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "4",
    perst_calc_scorecard(3, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "3",
    perst_calc_scorecard(2, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "2",
    perst_calc_scorecard(1, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "1",
    perst_calc_scorecard(0, POL_EFF_DT, CANCEL_E_30, ATR_E_30) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
	WHERE POL_NEW_RENEW_CD = 'R' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
	AND CONCAT(RISK_STATE,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG',
	'MDHDC1','MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1',
	'NJNJRP','NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
),

region_e_0_r AS -- Persistency calculations for Regional E+0 R
(
    SELECT CONCAT(GET_REGION_SCORECARD(RISK_STATE),BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+0 R') as Key,
    perst_calc_scorecard(18, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "18",
    perst_calc_scorecard(17, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "17",
    perst_calc_scorecard(16, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "16",
    perst_calc_scorecard(15, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "15",
    perst_calc_scorecard(14, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "14",
    perst_calc_scorecard(13, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "13",
    perst_calc_scorecard(12, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "12",
    perst_calc_scorecard(11, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "11",
    perst_calc_scorecard(10, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "10",
    perst_calc_scorecard(9, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "9",
    perst_calc_scorecard(8, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "8",
    perst_calc_scorecard(7, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "7",
    perst_calc_scorecard(6, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "6",
    perst_calc_scorecard(5, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "5",
    perst_calc_scorecard(4, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "4",
    perst_calc_scorecard(3, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "3",
    perst_calc_scorecard(2, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "2",
    perst_calc_scorecard(1, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "1",
    perst_calc_scorecard(0, POL_EFF_DT, CANCEL_E_FLAT, ATR_E_FLAT) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
	WHERE POL_NEW_RENEW_CD = 'R' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
	AND CONCAT(RISK_STATE,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG',
	'MDHDC1','MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1',
	'NJNJRP','NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
),

region_e_ult_r AS -- Persistency calculations for Regional E+Ult R
(
    SELECT CONCAT(GET_REGION_SCORECARD(RISK_STATE),BUS_UNIT_ABBR,NEWCO_INDICATOR,'E+Ult R') as Key,
    1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 18 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 18 THEN ULT_ATR_CNT ELSE 0 END)) AS "18",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 17 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 17 THEN ULT_ATR_CNT ELSE 0 END)) AS "17",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 16 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 16 THEN ULT_ATR_CNT ELSE 0 END)) AS "16",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 15 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 15 THEN ULT_ATR_CNT ELSE 0 END)) AS "15",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 14 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 14 THEN ULT_ATR_CNT ELSE 0 END)) AS "14",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 13 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 13 THEN ULT_ATR_CNT ELSE 0 END)) AS "13",					 
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 12 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 12 THEN ULT_ATR_CNT ELSE 0 END)) AS "12",			 
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 11 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 11 THEN ULT_ATR_CNT ELSE 0 END)) AS "11",			 
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 10 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 10 THEN ULT_ATR_CNT ELSE 0 END)) AS "10",			 
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 9 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 9 THEN ULT_ATR_CNT ELSE 0 END)) AS "9",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 8 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 8 THEN ULT_ATR_CNT ELSE 0 END)) AS "8",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 7 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 7 THEN ULT_ATR_CNT ELSE 0 END)) AS "7",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 6 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 6 THEN ULT_ATR_CNT ELSE 0 END)) AS "6",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 5 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 5 THEN ULT_ATR_CNT ELSE 0 END)) AS "5",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 4 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 4 THEN ULT_ATR_CNT ELSE 0 END)) AS "4",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 3 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 3 THEN ULT_ATR_CNT ELSE 0 END)) AS "3",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 2 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 2 THEN ULT_ATR_CNT ELSE 0 END)) AS "2",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 1 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 1 THEN ULT_ATR_CNT ELSE 0 END)) AS "1",
	1 - DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 0 THEN ULT_CANC_CNT ELSE 0 END),
				 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 0 THEN ULT_ATR_CNT ELSE 0 END)) AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
	WHERE POL_NEW_RENEW_CD = 'R' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
	AND CONCAT(RISK_STATE,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG',
	'MDHDC1','MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1',
	'NJNJRP','NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
),

region_prem_change AS -- Premium Changes by Region
(
    SELECT CONCAT(GET_REGION_SCORECARD(RISK_STATE),BUS_UNIT_ABBR,NEWCO_INDICATOR,'PremiumChange') as Key,
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 18 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 18 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "18",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 17 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 17 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "17",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 16 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 16 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "16",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 15 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 15 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "15",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 14 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 14 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "14",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 13 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 13 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "13",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 12 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 12 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "12",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 11 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 11 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "11",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 10 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 10 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "10",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 9 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 9 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "9",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 8 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 8 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "8",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 7 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 7 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "7",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 6 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 6 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "6",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 5 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 5 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "5",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 4 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 4 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "4",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 3 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 3 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "3",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 2 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 2 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "2",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 1 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 1 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "1",
	DIV0NULL(SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 0 THEN FIRST_TRANS_PREM ELSE 0 END),
			 SUM(CASE WHEN DATEDIFF(MONTH, POL_EFF_DT, SYSDATE()) = 0 THEN PRIOR_TERM_LATEST_PREM ELSE 0 END)) - 1 AS "0",
    TO_CHAR(sysdate(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
	WHERE POL_NEW_RENEW_CD = 'R' AND POL_EFF_DT >= '2023-01-01' AND BUS_UNIT_ABBR <> 'CDRT' AND LOB = 'AUTO' AND ATR_E_30 = 1
	AND CONCAT(RISK_STATE,RATE_PLAN_CD) IN ('AKHDC1','AKPL84','ALHDC1','ALPNG','ARHDC1','ARPNG','AZHDC1','AZPNG','CACARP','CAHDC1','COHDC1','COPNG','CTHDC1','CTPNG','DCHDC1','DCPNG','DEHDC1','DEPNG',
	'FLHDC1','FLPNG','GAHDC1','GAPNG','HIHDC1','HIPL84','IAHDC1','IAPNG','IDHDC1','IDPNG','ILHDC1','ILPNG','INHDC1','INPNG','KSHDC1','KSPNG','KYHDC1','KYPNG','LAHDC1','LAPNG','MAHDC1','MAPNG',
	'MDHDC1','MDPNG','MEHDC1','MEPNG','MIHDC1','MIPNG','MNHDC1','MNPNG','MOHDC1','MOPNG','MSHDC1','MSPNG','MTHDC1','MTPNG','NCHDC1','NCNCRP','NDHDC1','NDPNG','NEHDC1','NEPNG','NHHDC1','NHPNG','NJHDC1',
	'NJNJRP','NMHDC1','NMPNG','NVHDC1','NVPNG','NYHDC1','NYPNG','OHHDC1','OHPNG','OKHDC1','OKPNG','ORHDC1','ORPNG','PAHDC1','PAPNG','RIHDC1','RIPNG','SCHDC1','SCPNG','SDHDC1','SDPNG','TNHDC1','TNPNG','TXHDC1',
	'TXPNG','UTHDC1','UTPNG','VAHDC1','VAPNG','VTHDC1','VTPNG','WAHDC1','WAPNG','WIHDC1','WIPNG','WVHDC1','WVPNG','WYHDC1','WYPNG')
	GROUP BY Key
)

SELECT * FROM
(
	SELECT * FROM state_e_30_nb
	UNION ALL
	SELECT * FROM region_e_30_nb
	UNION ALL
	SELECT * FROM cw_e_30_nb
	UNION ALL
	SELECT * FROM state_e_60_nb
	UNION ALL
	SELECT * FROM region_e_60_nb
	UNION ALL
	SELECT * FROM cw_e_60_nb
	UNION ALL
	SELECT * FROM state_e_90_nb
	UNION ALL
	SELECT * FROM region_e_90_nb
	UNION ALL
	SELECT * FROM cw_e_90_nb 
	UNION ALL
	SELECT * FROM state_pol_count
	UNION ALL
	SELECT * FROM state_wp
	UNION ALL
	SELECT * FROM cw_pol_count
	UNION ALL
	SELECT * FROM cw_wp
	UNION ALL
	SELECT * FROM region_pol_count
	UNION ALL
	SELECT * FROM region_wp
	UNION ALL
	SELECT * FROM state_e_30_r
	UNION ALL
	SELECT * FROM state_e_0_r
	UNION ALL
	SELECT * FROM state_e_ult_r
	UNION ALL
	SELECT * FROM state_prem_change
	UNION ALL
	SELECT * FROM cw_e_30_r
	UNION ALL
	SELECT * FROM cw_e_0_r
	UNION ALL
	SELECT * FROM cw_e_ult_r
	UNION ALL
	SELECT * FROM cw_prem_change
	UNION ALL
	SELECT * FROM region_e_30_r
	UNION ALL
	SELECT * FROM region_e_0_r
	UNION ALL
	SELECT * FROM region_e_ult_r
	UNION ALL
	SELECT * FROM region_prem_change
)
ORDER BY KEY;
"""

In [9]:
cs = ctx.cursor()
try:
    cs.execute(auto_persistency)
finally:
    cs.close()
print('Done')

Done


## 4. Auto Seg CW Table Creation (DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_AUTO_SEG_CW)

In [10]:
query_4_body = """
    -- Total Completed Quotes
    zero_cond_scorecard(12, TRANS_DT, COMPL_QTE_IND_CNT) AS "12 Total CQ",
    zero_cond_scorecard(11, TRANS_DT, COMPL_QTE_IND_CNT) AS "11 Total CQ",
    zero_cond_scorecard(10, TRANS_DT, COMPL_QTE_IND_CNT) AS "10 Total CQ",
    zero_cond_scorecard(9, TRANS_DT, COMPL_QTE_IND_CNT) AS "9 Total CQ",
    zero_cond_scorecard(8, TRANS_DT, COMPL_QTE_IND_CNT) AS "8 Total CQ",
    zero_cond_scorecard(7, TRANS_DT, COMPL_QTE_IND_CNT) AS "7 Total CQ",
    zero_cond_scorecard(6, TRANS_DT, COMPL_QTE_IND_CNT) AS "6 Total CQ",
    zero_cond_scorecard(5, TRANS_DT, COMPL_QTE_IND_CNT) AS "5 Total CQ",
    zero_cond_scorecard(4, TRANS_DT, COMPL_QTE_IND_CNT) AS "4 Total CQ",
    zero_cond_scorecard(3, TRANS_DT, COMPL_QTE_IND_CNT) AS "3 Total CQ",
    zero_cond_scorecard(2, TRANS_DT, COMPL_QTE_IND_CNT) AS "2 Total CQ",
    zero_cond_scorecard(1, TRANS_DT, COMPL_QTE_IND_CNT) AS "1 Total CQ",
    zero_cond_scorecard(0, TRANS_DT, COMPL_QTE_IND_CNT) AS "0 Total CQ",
    -- Total Issues
    one_cond_scorecard(12, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "12 Total I",
    one_cond_scorecard(11, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "11 Total I",
    one_cond_scorecard(10, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "10 Total I",
    one_cond_scorecard(9, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "9 Total I",
    one_cond_scorecard(8, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "8 Total I",
    one_cond_scorecard(7, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "7 Total I",
    one_cond_scorecard(6, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "6 Total I",
    one_cond_scorecard(5, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "5 Total I",
    one_cond_scorecard(4, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "4 Total I",
    one_cond_scorecard(3, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "3 Total I",
    one_cond_scorecard(2, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "2 Total I",
    one_cond_scorecard(1, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "1 Total I",
    one_cond_scorecard(0, TRANS_DT, ISSUE_IND, 'Y', ISSUE_IND_CNT) AS "0 Total I",
    -- Total AQP
    zero_cond_scorecard(12, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "12 Total AQP",
    zero_cond_scorecard(11, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "11 Total AQP",
    zero_cond_scorecard(10, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "10 Total AQP",
    zero_cond_scorecard(9, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "9 Total AQP",
    zero_cond_scorecard(8, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "8 Total AQP",
    zero_cond_scorecard(7, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "7 Total AQP",
    zero_cond_scorecard(6, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "6 Total AQP",
    zero_cond_scorecard(5, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "5 Total AQP",
    zero_cond_scorecard(4, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "4 Total AQP",
    zero_cond_scorecard(3, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "3 Total AQP",
    zero_cond_scorecard(2, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "2 Total AQP",
    zero_cond_scorecard(1, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "1 Total AQP",
    zero_cond_scorecard(0, TRANS_DT, POL_TTL_BILL_PREM_AMT * 2) AS "0 Total AQP",
    -- Total AIP
    one_cond_scorecard(12, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "12 Total AIP",
    one_cond_scorecard(11, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "11 Total AIP",
    one_cond_scorecard(10, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "10 Total AIP",
    one_cond_scorecard(9, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "9 Total AIP",
    one_cond_scorecard(8, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "8 Total AIP",
    one_cond_scorecard(7, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "7 Total AIP",
    one_cond_scorecard(6, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "6 Total AIP",
    one_cond_scorecard(5, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "5 Total AIP",
    one_cond_scorecard(4, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "4 Total AIP",
    one_cond_scorecard(3, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "3 Total AIP",
    one_cond_scorecard(2, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "2 Total AIP",
    one_cond_scorecard(1, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "1 Total AIP",
    one_cond_scorecard(0, TRANS_DT, ISSUE_IND, 'Y', POL_TTL_BILL_PREM_AMT * 2) AS "0 Total AIP",
    -- Total ULM
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 12 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "12 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 11 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "11 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 10 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "10 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 9 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "9 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 8 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "8 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 7 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "7 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 6 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "6 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 5 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "5 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 4 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "4 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 3 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "3 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 2 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "2 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 1 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "1 Total ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 0 THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "0 Total ULM",
    -- Phone HIGSRP
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())=12 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT 
    ELSE NULL END)) AS "12 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())=11 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "11 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())=10 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "10 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 9 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "9 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 8 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "8 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 7 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "7 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 6 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "6 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 5 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "5 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 4 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "4 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 3 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "3 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 2 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "2 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 1 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "1 Phone HIGSRP",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE())= 0 and COMPL_QTE_IND_CNT = 1 AND PCARR_POL_TERM_MO_CNT = 6 AND PCARR_POL_ANNL_PREM_AMT > 249
    AND PCARR_POL_ANNL_PREM_AMT < 6000 AND POL_ANNL_TTL_BILL_PREM_AMT > 200 AND CNTCT_METH_CD = 'P' THEN POL_ANNL_TTL_BILL_PREM_AMT/PCARR_POL_ANNL_PREM_AMT
    ELSE NULL END)) AS "0 Phone HIGSRP",																								
    -- Phone CQ
    one_cond_scorecard(12, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "12 Phone CQ",
    one_cond_scorecard(11, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "11 Phone CQ",
    one_cond_scorecard(10, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "10 Phone CQ",
    one_cond_scorecard(9, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "9 Phone CQ",
    one_cond_scorecard(8, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "8 Phone CQ",
    one_cond_scorecard(7, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "7 Phone CQ",
    one_cond_scorecard(6, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "6 Phone CQ",
    one_cond_scorecard(5, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "5 Phone CQ",
    one_cond_scorecard(4, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "4 Phone CQ",
    one_cond_scorecard(3, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "3 Phone CQ",
    one_cond_scorecard(2, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "2 Phone CQ",
    one_cond_scorecard(1, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "1 Phone CQ",
    one_cond_scorecard(0, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', COMPL_QTE_IND_CNT) AS "0 Phone CQ",
    -- Phone Issues
    two_cond_scorecard(12, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "12 Phone I",
    two_cond_scorecard(11, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "11 Phone I",
    two_cond_scorecard(10, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "10 Phone I",
    two_cond_scorecard(9, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "9 Phone I",
    two_cond_scorecard(8, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "8 Phone I",
    two_cond_scorecard(7, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "7 Phone I",
    two_cond_scorecard(6, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "6 Phone I",
    two_cond_scorecard(5, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "5 Phone I",
    two_cond_scorecard(4, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "4 Phone I",
    two_cond_scorecard(3, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "3 Phone I",
    two_cond_scorecard(2, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "2 Phone I",
    two_cond_scorecard(1, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "1 Phone I",
    two_cond_scorecard(0, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', ISSUE_IND_CNT) AS "0 Phone I",
    -- Phone AQP
    one_cond_scorecard(12, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "12 Phone AQP",
    one_cond_scorecard(11, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "11 Phone AQP",
    one_cond_scorecard(10, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "10 Phone AQP",
    one_cond_scorecard(9, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "9 Phone AQP",
    one_cond_scorecard(8, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "8 Phone AQP",
    one_cond_scorecard(7, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "7 Phone AQP",
    one_cond_scorecard(6, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "6 Phone AQP",
    one_cond_scorecard(5, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "5 Phone AQP",
    one_cond_scorecard(4, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "4 Phone AQP",
    one_cond_scorecard(3, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "3 Phone AQP",
    one_cond_scorecard(2, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "2 Phone AQP",
    one_cond_scorecard(1, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "1 Phone AQP",
    one_cond_scorecard(0, TRANS_DT, ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "0 Phone AQP",
    -- Phone AIP
    two_cond_scorecard(12, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "12 Phone AIP",
    two_cond_scorecard(11, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "11 Phone AIP",
    two_cond_scorecard(10, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "10 Phone AIP",
    two_cond_scorecard(9, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "9 Phone AIP",
    two_cond_scorecard(8, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "8 Phone AIP",
    two_cond_scorecard(7, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "7 Phone AIP",
    two_cond_scorecard(6, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "6 Phone AIP",
    two_cond_scorecard(5, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "5 Phone AIP",
    two_cond_scorecard(4, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "4 Phone AIP",
    two_cond_scorecard(3, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "3 Phone AIP",
    two_cond_scorecard(2, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "2 Phone AIP",
    two_cond_scorecard(1, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "1 Phone AIP",
    two_cond_scorecard(0, TRANS_DT, ISSUE_IND, 'Y', ORIG_QTE_CNTCT_METH_CD, 'P', POL_TTL_BILL_PREM_AMT * 2) AS "0 Phone AIP",
    -- Phone ULM
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 12 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "12 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 11 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "11 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 10 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "10 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 9 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "9 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 8 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "8 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 7 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "7 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 6 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "6 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 5 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "5 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 4 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "4 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 3 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "3 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 2 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "2 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 1 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "1 Phone ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 0 and ORIG_QTE_CNTCT_METH_CD = 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "0 Phone ULM",
    -- Internet ULM 
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 12 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "12 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 11 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "11 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 10 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "10 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 9 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "9 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 8 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "8 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 7 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "7 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 6 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "6 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 5 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "5 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 4 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "4 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 3 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "3 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 2 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "2 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 1 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "1 Internet ULM",
    NULLIFZERO(AVG(CASE WHEN DATEDIFF(MONTH,TRANS_DT,SYSDATE()) = 0 and ORIG_QTE_CNTCT_METH_CD <> 'P' THEN EXPECTED_LOSS_RATIO/state_avg_lr ELSE NULL END)) AS "0 Internet ULM",
    TO_CHAR(SYSDATE(),'MM-DD-YYYY') AS AS_OF
"""

query_4_from = """
    FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW" t1
    LEFT JOIN (SELECT AUTO_QTE_TRANS_ID,EXPECTED_LOSS_RATIO FROM "DSC_PLDS_DB"."APP_AUTOMATA_PRD"."PREVAIL_AUTO_ULM_QUOTE_POL_AGG_NB_QUALITY")
    USING (AUTO_QTE_TRANS_ID)
    LEFT JOIN (SELECT state, TO_CHAR(TO_DATE(TRANS_DT, 'YYYY-MM-DD'),'MM-01-YYYY') AS TRANSDT2, AVG(expected_loss_ratio) AS state_avg_lr
               FROM "DSC_PLDS_DB"."APP_AUTOMATA_PRD"."PREVAIL_AUTO_ULM_QUOTE_POL_AGG_NB_QUALITY" WHERE COMPL_QTE_IND =  'Y' GROUP BY state, TRANSDT2) AS t4
	ON t1.RISK_ST_ABBR = t4.state AND TO_CHAR(TRANS_DT,'MM-01-YYYY') = t4.TRANSDT2
	WHERE COMPL_QTE_IND = 'Y'
	GROUP BY Key
"""

In [11]:
auto_seg_cw = f"""
CREATE OR REPLACE TRANSIENT TABLE DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_AUTO_SEG_CW AS

WITH ins_score AS -- Segmenting indicator counts by CW insurance score
(
	SELECT CONCAT('CW',BU_ABBR,'Ins Score Decile',(																								
    CASE WHEN INS_SCR_CD in (000,0,999)  then 'NHNS'																								
         WHEN INS_SCR_CD <= 386 then 'Decile 10 Worst'																								
         WHEN INS_SCR_CD <= 460 then 'Decile 09'																								
         WHEN INS_SCR_CD <= 539 then 'Decile 08'																								
         WHEN INS_SCR_CD <= 617 then 'Decile 07'																								
         WHEN INS_SCR_CD <= 679 then 'Decile 06'																								
         WHEN INS_SCR_CD <= 724 then 'Decile 05'																								
         WHEN INS_SCR_CD <= 756 then 'Decile 04'																								
         WHEN INS_SCR_CD <= 782 then 'Decile 03'																								
         WHEN INS_SCR_CD <= 809 then 'Decile 02'																								
         WHEN INS_SCR_CD <= 980 then 'Decile 01 Best' else 'NHNS' end)) AS Key,
    {query_4_body} {query_4_from}
),

household_comp AS -- Segmenting indicator counts by CW household comp
(
	SELECT CONCAT('CW',BU_ABBR,'HH Comp',(																								
	CASE WHEN HH_COMPOS_PPV_CD IN ('SC1D') THEN 'a. single vehicle, 1 driver'																								
		 WHEN  HH_COMPOS_PPV_CD IN ('MC C=D') THEN 'd. multi vehicle, vehicle = driver'																								
		 WHEN  HH_COMPOS_PPV_CD IN ('SCMT1D') THEN 'b. single vehicle, more than 1 driver' 																								
		 WHEN  HH_COMPOS_PPV_CD IN ('MCMCTD') THEN 'c. multi vehicle, vehicle > drivers' 																								
		 WHEN  HH_COMPOS_PPV_CD IN ('MCMDTC') THEN 'e. multi vehicle, vehicle < drivers' 																								
		 WHEN  HH_COMPOS_PPV_CD IN ('0 CARS') THEN 'f. 0 vehicle'																								
		 ELSE 'g. other' END)) as Key,
	{query_4_body} {query_4_from}
),

clean_dirty AS -- Segmenting indicator counts by CW Clean/Dirty
(
	SELECT CONCAT('CW',BU_ABBR,'CLEAN DIRTY DESC',CLEAN_DIRTY_DESC) as Key,
	{query_4_body} {query_4_from}
),

max_age_group AS -- Segmenting indicator counts by CW max age
(
	SELECT CONCAT('CW',BU_ABBR,'Max Age Group',(																								
	CASE WHEN POL_RATE_DRVR_MAX_AGE < 50 THEN '50'																								
		 WHEN POL_RATE_DRVR_MAX_AGE <= 59 THEN '50-59' 																								
		 WHEN POL_RATE_DRVR_MAX_AGE <= 69 THEN '50-69' 																								
		 WHEN POL_RATE_DRVR_MAX_AGE <= 79 THEN '70-79' 																								
		 WHEN POL_RATE_DRVR_MAX_AGE <= 130 THEN '80+'
		 ELSE 'Unknown' END)) as Key,
	{query_4_body} {query_4_from}
),

min_age_group AS -- Segmenting indicator counts by CW min age
(
	SELECT CONCAT('CW',BU_ABBR,'Min Age Group',(																								
	CASE WHEN POL_RATE_DRVR_MIN_AGE < 50 THEN '50'																								
		 WHEN POL_RATE_DRVR_MIN_AGE <= 59 THEN '50-59' 																								
		 WHEN POL_RATE_DRVR_MIN_AGE <= 69 THEN '50-69' 																								
		 WHEN POL_RATE_DRVR_MIN_AGE <= 79 THEN '70-79' 																								
		 WHEN POL_RATE_DRVR_MIN_AGE <= 130 THEN '80+'
		 ELSE 'Unknown' END)) as Key,
	{query_4_body} {query_4_from}
),

max_minus_min AS -- Segmenting indicator counts by CW max-min age
(
	SELECT CONCAT('CW',BU_ABBR,'Max Minus Min Age Group',(																								
	CASE WHEN POL_RATE_DRVR_MAX_AGE - POL_RATE_DRVR_MIN_AGE = 0 THEN 'a. 0'
		 WHEN POL_RATE_DRVR_MAX_AGE - POL_RATE_DRVR_MIN_AGE BETWEEN 1 AND 2 THEN 'b. 1-2'
		 WHEN POL_RATE_DRVR_MAX_AGE - POL_RATE_DRVR_MIN_AGE BETWEEN 3 AND 6 THEN 'c. 3-6'
		 WHEN POL_RATE_DRVR_MAX_AGE - POL_RATE_DRVR_MIN_AGE BETWEEN 7 AND 20 THEN 'd. 7-20'
		 WHEN POL_RATE_DRVR_MAX_AGE - POL_RATE_DRVR_MIN_AGE >= 21 THEN 'e. 21+'																							
		 ELSE 'f. n/a' END)) as Key,
	{query_4_body} {query_4_from}	 
),

pcarr_yr_cnt AS -- Segmenting indicator counts by CW pcarr year count
(
	SELECT CONCAT('CW',BU_ABBR,'PCARR YR CNT',(																								
	CASE WHEN PCARR_YR_CNT = 0.5 THEN 'a 0.5'																								
		 WHEN PCARR_YR_CNT = 1.0 THEN 'b 1' 																								
		 WHEN PCARR_YR_CNT = 2.0 THEN 'c 2' 																								
		 WHEN PCARR_YR_CNT = 3.0 THEN 'd 3' 																								
		 WHEN PCARR_YR_CNT = 4.0 THEN 'e 4' 																								
		 WHEN PCARR_YR_CNT = 5.0 THEN 'f 5+' 																								
		 ELSE 'g Unknown' END)) as Key,
	{query_4_body} {query_4_from}
),

account_credit AS -- Segmenting indicator counts by CW account credit
(
	SELECT CONCAT('CW',BU_ABBR,'POL ACCT CR IND', POL_ACCT_CR_IND) as Key,
	{query_4_body} {query_4_from}
),

pol_coverage_pkg AS -- Segmenting indicator counts by CW policy coverage package
(
	SELECT CONCAT('CW',BU_ABBR,'POL COV PKG CD',(																								
	CASE WHEN POL_COV_PKG_CD IS NULL OR POL_COV_PKG_CD = 'NO COV' THEN 'Unknown'
		 ELSE POL_COV_PKG_CD END)) as Key,
	{query_4_body} {query_4_from}
),

pcarr AS -- Segmenting indicator counts by CW prior carrier
(
	SELECT CONCAT('CW',BU_ABBR,'Prior Carrier',(																								
    CASE WHEN PCARR_DESC IN ('STATE FARM INSURANCE') THEN 'STATE FARM'
		 WHEN PCARR_DESC IN ('IMPERIAL FIRE AND CA','SAFE AUTO INS CO','ALLSTATE CTY MUTUAL','ALLSTATE FIRE CAS IN','ALLSTATE INDEMNITY C','ALLSTATE INSURANCE C','ALLSTATE OF NEW JERS','ALLSTATE PROPERTY',
		 'ANJP C INS CO','CENTURY NATIONAL INS','CENTURY-NATIONAL INS','DIRECT GENERAL INS','DIRECT GENERAL INSUR','DIRECT INSURANCE CO','DIRECT INSURANCE COM','DIRECT NATIONAL INSU','ENCOM H A INS COMP',
		 'ENCOM INDEPENDENT IN','ENCOM INS CO OF NJ','ENCOM P C COMP','ENCOMPASS INDEMNITY','ENCOMPASS INS OF AM','ENCOMPASS INSURANCE','ENCOMPASS P C OF NJ','ESURANCE INS CO','ESURANCE INS CO NJ','ESURANCE PROP CAS',
		 'IMPERIAL FIRE CASU','INTEGON CASUALTY INS','INTEGON GENERAL INSU','INTEGON INDEMNITY CO','INTEGON NATIONAL','INTEGON NATIONAL INS','INTEGON PREFERRED IN','MIC GENERAL INSURANC','MS-DIRECT GENERAL IN',
		 'NATIONAL FARMERS UNI','NATIONAL GENERAL ASS','NATIONAL GENERAL INS','NATIONAL GENERAL PRE','NEW SOUTH INSURANCE','NORTHBROOK INSURANCE','STANDARD PROPERTY AN') THEN 'ALLSTATE'
		 WHEN PCARR_DESC IN ('GEICO ADVANTAGE INS','GEICO CASUALTY COMP','GEICO CHOICE INS CO','GEICO COUNTY MTL','GEICO GENERAL INS CO','GEICO INDEMNITY COMP','GEICO SECURE INS CO','GEICOTXCOUNTYMUTUAL',
		 'GOVERNMENT EMP INSCO','SOUTHERN COUNTY') THEN 'GEICO'
		 WHEN PCARR_DESC IN ('PROGRESSIVE GROUPS') THEN 'PROGRESSIVE'
		 WHEN PCARR_DESC IN ('EMPLOYERS INSURANCE','FIRST NAT INS CO AME','GENERAL INS CO AMER','LIB MUTUAL PERSONAL','LIBERTY COUNTY MUTUA','LIBERTY INSURANCE CO','LIBERTY MUTUAL FIRE','LIBERTY MUTUAL INSUR',
		 'LIBERTY MUTUAL MID A','LIBERTY PERSONAL INS','LM GENERAL INSURANCE','LM INSURANCE CORPORA','MERIDIAN SECURITY','MERIDIAN SECURITY IN','MILBANK INSURANCE CO','MONTGOMERY MUTUAL IN','PATRONS MUTUAL INS C',
		 'PEERLESS INDEMNITY I','SAFECO INS CO AMER','SAFECO INSUR CO ILL','SAFECO INSUR CO IND','SAFECO INSUR CO ORE','SAFECO LLOYDS INS CO','SAFECO NAT INSUR CO','SAIC OF OHIO','SAIC OF WISCONSIN','STATE AUTO MUTUAL',
		 'STATE AUTO P C INS','STATE AUTO PROP C','STATE AUTOMOBILE MUT','THE FIRST LIBERTY IN','THE NETHERLANDS INSU','WAUSAU UNDERWRITERS','AMER ECONOMY INS CO','AMER STATES INSUR CO','AMER STATES PREF INS',
		 'AMERICA FIRST INSURA','CONSOLIDATED INSURAN') THEN 'LIBERTY'
		 WHEN PCARR_DESC IN ('FID GUAR INS UNDRS','PHOENIX INSURANCE CO','ST PAUL PROTECTIVE I','STANDARD FIRE INS CO','TRAVCO INSURANCE CO','TRAVELERS CAS CONN','TRAVELERS COMMERCIAL','TRAVELERS HOME MAR',
		 'TRAVELERS INDEM AMER','TRAVELERS INDEM CONN','TRAVELERS INDEMNITY','TRAVELERS P C INS CO','TRAVELERS P C OF AM','TRAVELERS PERSON SEC','TRAVELERS PERSONAL','AUTO INS CO OF HARTF',
		 'CHARTER OAK FIRE INS','FIRST FLORIDIAN A H') THEN 'TRAVELERS'
		 WHEN PCARR_DESC IN ('AMFAM CONNECT INS', 'AMFAM CONNECT PC', 'AMFAM INS CO', 'AMER FAMILY MUTUAL', 'AMERICAN STD INS OH', 'THE GENERAL AUTOMOBI', 'MIDVALE INDEMNITY', 'AFMICSI', 'AMER STANDARD INS',
		 'AMERICAN FAM INS OH',	'AMERIPRISE INSURANCE',	'HOMESITE INSURANCE C',	'IDS PROPERTY CASUALT',	'MSAA',	'MSAA INSURANCE CO','MSAPIC','NGM INSURANCE CO','ODIC') THEN 'AMERICAN FAMILY'
		 WHEN PCARR_DESC IN ('USAA GENERAL IND CO','USAA CASUALTY INS CO','USAA','GARRISON PROP CAS','GAR') THEN 'USAA'																				
		 WHEN PCARR_DESC IN ('COLONIAL COUNTY MUT','ALLIED PROP CAS','ALLIED P C INS CO','COLONIAL COUNTY MUT','CRESTBROOK INSURANCE','DEPOSITORS INS CO',	
		 'NATIONWIDE AG INS CO', 'NATIONWIDE AGRI INS','NATIONWIDE GENERAL','NATIONWIDE INS CO','NATIONWIDE INS OF AM',	'NATIONWIDE MUT FIRE','NATIONWIDE MUT INS','NATIONWIDE MUTUAL IN',
		 'VICTORIA FIRE CASU',	'VICTORIA SPECIALTY I',	'AMCO INSURANCE CO',	'ALLIED P C INS CO',	'HARLEYSVILLEIC',	'HARLEYSVILLEPREFIC',	'NATIONWIDE AFFINITY',
		 'NATIONWIDE GENERAL',	'NATIONWIDE MUTUAL',	'NATIONWIDE PROP CAS',	'VICTORIA SELECT INSU') THEN 'Nationwide'
		 WHEN PCARR_DESC IS NULL  THEN 'NULL' 																								
		 WHEN PCARR_DESC IN ('NOT PRESENTLY INSURED', 'UNKNOWN', 'NO PRIOR INSURANCE') THEN PCARR_DESC
		 WHEN PCARR_DESC IN ( 'OTHER-NON STANDARD' , 'OTHER - NON STANDARD' ) THEN 'OTHER - NON STANDARD'
		 ELSE 'OTHER' END)) as Key,
	{query_4_body} {query_4_from}
),

ins_age AS -- Segmenting indicator counts by CW insured age
(
	SELECT CONCAT('CW',BU_ABBR,'Insured Age Group',(																								
	CASE WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 49 THEN 'a. 25-49'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 54 THEN 'b. 50-54'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 59 THEN 'c. 55-59'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 64 THEN 'd. 60-64'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 69 THEN 'e. 65-69'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 74 THEN 'f. 70-74'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 79 THEN 'g. 75-79'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 120 THEN 'h. 80+'																								
		 ELSE 'i. null' END)) as Key,
	{query_4_body} {query_4_from}
),

ppv_count AS -- Segmenting indicator counts by CW PPV count
(
	SELECT CONCAT('CW',BU_ABBR,'POL PPV CNT',(																								
	CASE WHEN POL_PPV_CNT = 1 THEN 'b 1' 																								
		 WHEN POL_PPV_CNT = 2 THEN 'c 2' 																								
		 WHEN POL_PPV_CNT = 3 THEN 'd 3' 																								
		 WHEN POL_PPV_CNT = 4 THEN 'e 4' 																								
		 WHEN POL_PPV_CNT = 5 THEN 'f 5+' 																								
		 ELSE 'g Unknown' END)) as Key,
	{query_4_body} {query_4_from}
),

telematics AS -- Segmenting indicator counts by CW telematics indicator
(
	SELECT CONCAT('CW',BU_ABBR,'Pol drvr tlmtc enrol ind',Pol_drvr_tlmtc_enrol_ind) as Key,
	{query_4_body} {query_4_from}
),

youthful AS -- Segmenting indicator counts by CW youthful indicator
(
	SELECT CONCAT('CW',BU_ABBR,'YOUTH OPER DRVR IND',YOUTH_OPER_DRVR_IND) as Key,
	{query_4_body} {query_4_from}
),

garage_sub AS -- Segmenting indicator counts by CW garage substate
(
	SELECT CONCAT('CW',BU_ABBR,'GARAGE SUBSTATE',(																								
	CASE WHEN LEFT(RISK_ST_ABBR,2) = LEFT(GARAGE_SUBSTATE,2) THEN GARAGE_SUBSTATE
	ELSE 'Out of State Mail' END)) as Key,
	{query_4_body}
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW" t1
    LEFT JOIN (SELECT AUTO_QTE_TRANS_ID,EXPECTED_LOSS_RATIO FROM "DSC_PLDS_DB"."APP_AUTOMATA_PRD"."PREVAIL_AUTO_ULM_QUOTE_POL_AGG_NB_QUALITY")
    USING (AUTO_QTE_TRANS_ID)
    LEFT JOIN (SELECT state, TO_CHAR(TO_DATE(TRANS_DT, 'YYYY-MM-DD'),'MM-01-YYYY') AS TRANSDT2, AVG(expected_loss_ratio) AS state_avg_lr
               FROM "DSC_PLDS_DB"."APP_AUTOMATA_PRD"."PREVAIL_AUTO_ULM_QUOTE_POL_AGG_NB_QUALITY" WHERE COMPL_QTE_IND =  'Y' GROUP BY state, TRANSDT2) AS t4
	ON t1.RISK_ST_ABBR = t4.state AND TO_CHAR(TRANS_DT,'MM-01-YYYY') = t4.TRANSDT2
	LEFT JOIN (SELECT ZIP_CODE, STATE_ABBR, GARAGE_SUBSTATE FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."STATE_PRODUCT_SUBSTATE") t7
	ON t7.ZIP_CODE = LEFT(MAIL_ADDR_FULL_ZIP_CD,5)	
    WHERE COMPL_QTE_IND = 'Y'
    GROUP BY Key
),

prior_bi AS -- Segmenting indicator counts by CW prior BI per person
(
	SELECT CONCAT('CW',BU_ABBR,'Prior BI Per Person',(																								
	CASE WHEN PCARR_BI_PERS_LMT_CD < 50 THEN 'a. Less than 50'																								
		 WHEN PCARR_BI_PERS_LMT_CD < 100 THEN 'b. 50-99'																								
		 WHEN PCARR_BI_PERS_LMT_CD < 250 THEN 'c. 100-249' 																								
		 WHEN PCARR_BI_PERS_LMT_CD < 3000 THEN 'd. 250+' 																								
		 WHEN PCARR_BI_PERS_LMT_CD IS NULL THEN 'f. Unknown/No Insurance'
		 ELSE 'e. other' END)) AS Key,
	{query_4_body} {query_4_from}
),

aarp_member AS -- Segmenting indicator counts by CW AARP member status
(
	SELECT CONCAT('CW',BU_ABBR,'AARP Member Status',AARP_MEMB_STAT_DESC) as Key,
	{query_4_body} {query_4_from}
),

aqd_group AS -- Segmenting indicator counts by CW AQD group
(
	SELECT CONCAT('CW',BU_ABBR,'Advanced Quote Discount',(																								
    CASE WHEN ADV_QTE_DAY_CNT IN (000,0,999) THEN 'A. 0 Days'																								
		 WHEN ADV_QTE_DAY_CNT <= 13 THEN 'B. 1-13'																								
		 WHEN ADV_QTE_DAY_CNT <= 60 THEN 'C. 14-60'																								
		 WHEN ADV_QTE_DAY_CNT <= 75 THEN 'D. 61-75'																								
		 WHEN ADV_QTE_DAY_CNT <= 99999 THEN 'E. 76+'																								
		 ELSE 'F. Unknown' END)) as Key,
	{query_4_body} {query_4_from}
),

renew_uw AS -- Segmenting indicator counts by CW renewal UW tier
(
	SELECT CONCAT('CW',BU_ABBR,'RENEW UW TIER',(																								
	CASE WHEN RENEW_UW_TIER_CD IN ('BB','BC','BD','BE','BF','BG','BH','BI','BJ','BK') THEN RENEW_UW_TIER_CD
		 ELSE 'Other' END)) as Key,
	{query_4_body} {query_4_from}
),

min_rating_factor AS -- Segmenting indicator counts by CW min rating factor
(
	SELECT CONCAT('CW',BU_ABBR,'Min Rating Factor',POL_MIN_RATE_FCTR_IND) as Key,
	{query_4_body} {query_4_from}
),

max_veh_age AS -- Segmenting indicator counts by CW max vehicle age
(
	SELECT CONCAT('CW',BU_ABBR,'Max Veh Age',(																								
	CASE WHEN POL_MAX_VEH_AGE <= 3 THEN '0-3'
		 WHEN POL_MAX_VEH_AGE <= 6 THEN '4-6'
		 WHEN POL_MAX_VEH_AGE <= 9 THEN '7-9'
		 WHEN POL_MAX_VEH_AGE <= 14 THEN '10-14' 																								
		 WHEN POL_MAX_VEH_AGE <= 19 THEN '15-19'
		 WHEN POL_MAX_VEH_AGE <= 24 THEN '20-24' 																								
		 WHEN POL_MAX_VEH_AGE > 24 THEN '25+'  																								
		 ELSE 'Other' END)) as Key,
	{query_4_body} {query_4_from}
),

pol_mult_single AS -- Segmenting indicator counts by CW multi-single vehicle code
(
	SELECT CONCAT('CW',BU_ABBR,'POL MULTI SNGL VEH CD',(																								
	CASE WHEN POL_MULTI_SNGL_VEH_CD IS NULL THEN 'S'
		 ELSE POL_MULTI_SNGL_VEH_CD END)) as Key,
	{query_4_body} {query_4_from}
),

mature_driving_course AS -- Segmenting indicator counts by CW mature driving indicator
(
	SELECT CONCAT('CW',BU_ABBR,'Mature Driving Course',(																								
	CASE WHEN POL_MATUR_DRVNG_COURS_CR_IND = 'Y' THEN 'Y'
		 ELSE 'N' END)) as Key,
	{query_4_body} {query_4_from}
),

incident_free AS -- Segmenting indicator counts by CW incident free year count
(
	SELECT CONCAT('CW',BU_ABBR,'Incident Free Year Count',(																								
	CASE WHEN POL_INCID_FREE_YR_CNT = 0 THEN '0'																								
		 WHEN POL_INCID_FREE_YR_CNT = 1 THEN '1'																								
		 WHEN POL_INCID_FREE_YR_CNT = 2 THEN '2' 																								
		 WHEN POL_INCID_FREE_YR_CNT = 3 THEN '3' 																								
		 WHEN POL_INCID_FREE_YR_CNT = 4 THEN '4' 																								
		 WHEN POL_INCID_FREE_YR_CNT = 5 THEN '5' END)) as Key,
	{query_4_body} {query_4_from}
),

naf_accidents AS -- Segmenting indicator counts by CW NAF accident count
(
	SELECT CONCAT('CW',BU_ABBR,'NAF Accident Count',(																								
	CASE WHEN PCARR_POL_NAF_ACCID_CNT = 0 THEN '0'																								
		 WHEN PCARR_POL_NAF_ACCID_CNT = 1 THEN '1'																								
		 WHEN PCARR_POL_NAF_ACCID_CNT > 1 THEN '2+' END)) as Key,
	{query_4_body} {query_4_from}
),

incident_mo_age AS -- Segmenting indicator counts by CW incident month age
(
	SELECT CONCAT('CW',BU_ABBR,'Incident Month Age',(																								
	CASE WHEN POL_INCID_MO_AGE <= 12 THEN '1-12'																								
		 WHEN POL_INCID_MO_AGE <= 24 THEN '13-24'																								
		 WHEN POL_INCID_MO_AGE <= 36 THEN '25-36'																								
		 WHEN POL_INCID_MO_AGE <= 48 THEN '37-48'																								
		 WHEN POL_INCID_MO_AGE <= 60 THEN '49-60' 																								
		 WHEN POL_INCID_MO_AGE = 61 THEN '61' END)) as Key,
	{query_4_body} {query_4_from}
),

region_ins_score AS -- Segmenting indicator counts by region insurance score
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'Ins Score Decile',(																								
    CASE WHEN INS_SCR_CD IN (000,0,999) THEN 'NHNS'																								
         WHEN INS_SCR_CD <= 386 THEN 'Decile 10 Worst'																								
         WHEN INS_SCR_CD <= 460 THEN 'Decile 09'																								
         WHEN INS_SCR_CD <= 539 THEN 'Decile 08'																								
         WHEN INS_SCR_CD <= 617 THEN 'Decile 07'																								
         WHEN INS_SCR_CD <= 679 THEN 'Decile 06'																								
         WHEN INS_SCR_CD <= 724 THEN 'Decile 05'																								
         WHEN INS_SCR_CD <= 756 THEN 'Decile 04'																								
         WHEN INS_SCR_CD <= 782 THEN 'Decile 03'																								
         WHEN INS_SCR_CD <= 809 THEN 'Decile 02'																								
         WHEN INS_SCR_CD <= 980 THEN 'Decile 01 Best'
		 ELSE 'NHNS' END)) as Key,
	{query_4_body} {query_4_from}
),

region_hh_comp AS -- Segmenting indicator counts by region household comp
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'HH Comp',(																								
	CASE WHEN HH_COMPOS_PPV_CD IN ('SC1D') THEN 'a. single vehicle, 1 driver'																								
		 WHEN HH_COMPOS_PPV_CD IN ('MC C=D') THEN 'd. multi vehicle, vehicle = driver'																								
		 WHEN HH_COMPOS_PPV_CD IN ('SCMT1D') THEN 'b. single vehicle, more than 1 driver' 																								
		 WHEN HH_COMPOS_PPV_CD IN ('MCMCTD') THEN 'c. multi vehicle, vehicle > drivers' 																								
		 WHEN HH_COMPOS_PPV_CD IN ('MCMDTC') THEN 'e. multi vehicle, vehicle < drivers' 																								
		 WHEN HH_COMPOS_PPV_CD IN ('0 CARS') THEN 'f. 0 vehicle'							
		 ELSE 'g. other' END)) as Key,
	{query_4_body} {query_4_from}
),

region_clean_dirty AS -- Segmenting indicator counts by region clean dirty code
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'CLEAN DIRTY DESC', CLEAN_DIRTY_DESC) as Key,
	{query_4_body} {query_4_from}
),

region_max_age_group AS -- Segmenting indicator counts by region max age
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'Max Age Group',(																								
	CASE WHEN POL_RATE_DRVR_MAX_AGE < 50 THEN '50'																								
		 WHEN POL_RATE_DRVR_MAX_AGE <= 59 THEN '50-59' 																								
		 WHEN POL_RATE_DRVR_MAX_AGE <= 69 THEN '50-69' 																								
		 WHEN POL_RATE_DRVR_MAX_AGE <= 79 THEN '70-79' 																								
		 WHEN POL_RATE_DRVR_MAX_AGE <= 130 THEN '80+'
		 ELSE 'Unknown' END)) as Key,
	{query_4_body} {query_4_from}
),

region_min_age_group AS -- Segmenting indicator counts by region min age
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'Min Age Group',(																								
	CASE WHEN POL_RATE_DRVR_MIN_AGE < 50 THEN '50'																								
		 WHEN POL_RATE_DRVR_MIN_AGE <= 59 THEN '50-59' 																								
		 WHEN POL_RATE_DRVR_MIN_AGE <= 69 THEN '50-69' 																								
		 WHEN POL_RATE_DRVR_MIN_AGE <= 79 THEN '70-79' 																								
		 WHEN POL_RATE_DRVR_MIN_AGE <= 130 THEN '80+'
		 ELSE 'Unknown' END)) as Key,
	{query_4_body} {query_4_from}
),

region_max_minus_min AS -- Segmenting indicator counts by region max-min age
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'Max Minus Min Age Group',(																								
	CASE WHEN POL_RATE_DRVR_MAX_AGE - POL_RATE_DRVR_MIN_AGE = 0 THEN 'a. 0'
		 WHEN POL_RATE_DRVR_MAX_AGE - POL_RATE_DRVR_MIN_AGE BETWEEN 1 AND 2 THEN 'b. 1-2'
		 WHEN POL_RATE_DRVR_MAX_AGE - POL_RATE_DRVR_MIN_AGE BETWEEN 3 AND 6 THEN 'c. 3-6'
		 WHEN POL_RATE_DRVR_MAX_AGE - POL_RATE_DRVR_MIN_AGE BETWEEN 7 AND 20 THEN 'd. 7-20'
		 WHEN POL_RATE_DRVR_MAX_AGE - POL_RATE_DRVR_MIN_AGE >= 21 THEN 'e. 21+'																							
		 ELSE 'f. n/a' END)) as Key,
	{query_4_body} {query_4_from}	 
),

region_pcarr_yr_cnt AS -- Segmenting indicator counts by region pcarr year count
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'PCARR YR CNT',(																								
	CASE WHEN PCARR_YR_CNT = 0.5 THEN 'a 0.5'																								
		 WHEN PCARR_YR_CNT = 1.0 THEN 'b 1' 																								
		 WHEN PCARR_YR_CNT = 2.0 THEN 'c 2' 																								
		 WHEN PCARR_YR_CNT = 3.0 THEN 'd 3' 																								
		 WHEN PCARR_YR_CNT = 4.0 THEN 'e 4' 																								
		 WHEN PCARR_YR_CNT = 5.0 THEN 'f 5+' 																								
		 ELSE 'g Unknown' END)) as Key,
	{query_4_body} {query_4_from}
),

region_account_credit AS -- Segmenting indicator counts by region account credit
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'POL ACCT CR IND', POL_ACCT_CR_IND) as Key,
	{query_4_body} {query_4_from}
),

region_pol_coverage_pkg AS -- Segmenting indicator counts by region policy coverage package
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'POL COV PKG CD',(																								
	CASE WHEN POL_COV_PKG_CD IS NULL OR POL_COV_PKG_CD = 'NO COV' THEN 'Unknown'
		 ELSE POL_COV_PKG_CD END)) as Key,
	{query_4_body} {query_4_from}
),

region_pcarr AS -- Segmenting indicator counts by region prior carrier
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'Prior Carrier',(																								
    CASE WHEN PCARR_DESC IN ('STATE FARM INSURANCE') THEN 'STATE FARM'
		 WHEN PCARR_DESC IN ('IMPERIAL FIRE AND CA','SAFE AUTO INS CO','ALLSTATE CTY MUTUAL','ALLSTATE FIRE CAS IN','ALLSTATE INDEMNITY C','ALLSTATE INSURANCE C','ALLSTATE OF NEW JERS','ALLSTATE PROPERTY',
		 'ANJP C INS CO','CENTURY NATIONAL INS','CENTURY-NATIONAL INS','DIRECT GENERAL INS','DIRECT GENERAL INSUR','DIRECT INSURANCE CO','DIRECT INSURANCE COM','DIRECT NATIONAL INSU','ENCOM H A INS COMP',
		 'ENCOM INDEPENDENT IN','ENCOM INS CO OF NJ','ENCOM P C COMP','ENCOMPASS INDEMNITY','ENCOMPASS INS OF AM','ENCOMPASS INSURANCE','ENCOMPASS P C OF NJ','ESURANCE INS CO','ESURANCE INS CO NJ','ESURANCE PROP CAS',
		 'IMPERIAL FIRE CASU','INTEGON CASUALTY INS','INTEGON GENERAL INSU','INTEGON INDEMNITY CO','INTEGON NATIONAL','INTEGON NATIONAL INS','INTEGON PREFERRED IN','MIC GENERAL INSURANC','MS-DIRECT GENERAL IN',
		 'NATIONAL FARMERS UNI','NATIONAL GENERAL ASS','NATIONAL GENERAL INS','NATIONAL GENERAL PRE','NEW SOUTH INSURANCE','NORTHBROOK INSURANCE','STANDARD PROPERTY AN') THEN 'ALLSTATE'
		 WHEN PCARR_DESC IN ('GEICO ADVANTAGE INS','GEICO CASUALTY COMP','GEICO CHOICE INS CO','GEICO COUNTY MTL','GEICO GENERAL INS CO','GEICO INDEMNITY COMP','GEICO SECURE INS CO','GEICOTXCOUNTYMUTUAL',
		 'GOVERNMENT EMP INSCO','SOUTHERN COUNTY') THEN 'GEICO'
		 WHEN PCARR_DESC IN ('PROGRESSIVE GROUPS') THEN 'PROGRESSIVE'
		 WHEN PCARR_DESC IN ('EMPLOYERS INSURANCE','FIRST NAT INS CO AME','GENERAL INS CO AMER','LIB MUTUAL PERSONAL','LIBERTY COUNTY MUTUA','LIBERTY INSURANCE CO','LIBERTY MUTUAL FIRE','LIBERTY MUTUAL INSUR',
		 'LIBERTY MUTUAL MID A','LIBERTY PERSONAL INS','LM GENERAL INSURANCE','LM INSURANCE CORPORA','MERIDIAN SECURITY','MERIDIAN SECURITY IN','MILBANK INSURANCE CO','MONTGOMERY MUTUAL IN','PATRONS MUTUAL INS C',
		 'PEERLESS INDEMNITY I','SAFECO INS CO AMER','SAFECO INSUR CO ILL','SAFECO INSUR CO IND','SAFECO INSUR CO ORE','SAFECO LLOYDS INS CO','SAFECO NAT INSUR CO','SAIC OF OHIO','SAIC OF WISCONSIN','STATE AUTO MUTUAL',
		 'STATE AUTO P C INS','STATE AUTO PROP C','STATE AUTOMOBILE MUT','THE FIRST LIBERTY IN','THE NETHERLANDS INSU','WAUSAU UNDERWRITERS','AMER ECONOMY INS CO','AMER STATES INSUR CO','AMER STATES PREF INS',
		 'AMERICA FIRST INSURA','CONSOLIDATED INSURAN') THEN 'LIBERTY'
		 WHEN PCARR_DESC IN ('FID GUAR INS UNDRS','PHOENIX INSURANCE CO','ST PAUL PROTECTIVE I','STANDARD FIRE INS CO','TRAVCO INSURANCE CO','TRAVELERS CAS CONN','TRAVELERS COMMERCIAL','TRAVELERS HOME MAR',
		 'TRAVELERS INDEM AMER','TRAVELERS INDEM CONN','TRAVELERS INDEMNITY','TRAVELERS P C INS CO','TRAVELERS P C OF AM','TRAVELERS PERSON SEC','TRAVELERS PERSONAL','AUTO INS CO OF HARTF',
		 'CHARTER OAK FIRE INS','FIRST FLORIDIAN A H') THEN 'TRAVELERS'
		 WHEN PCARR_DESC IN ('AMFAM CONNECT INS', 'AMFAM CONNECT PC', 'AMFAM INS CO', 'AMER FAMILY MUTUAL', 'AMERICAN STD INS OH', 'THE GENERAL AUTOMOBI', 'MIDVALE INDEMNITY', 'AFMICSI', 'AMER STANDARD INS',
		 'AMERICAN FAM INS OH',	'AMERIPRISE INSURANCE',	'HOMESITE INSURANCE C',	'IDS PROPERTY CASUALT',	'MSAA',	'MSAA INSURANCE CO','MSAPIC','NGM INSURANCE CO','ODIC') THEN 'AMERICAN FAMILY'
		 WHEN PCARR_DESC IN ('USAA GENERAL IND CO','USAA CASUALTY INS CO','USAA','GARRISON PROP CAS','GAR') THEN 'USAA'																				
		 WHEN PCARR_DESC IN ('COLONIAL COUNTY MUT','ALLIED PROP CAS','ALLIED P C INS CO','COLONIAL COUNTY MUT','CRESTBROOK INSURANCE','DEPOSITORS INS CO',	
		 'NATIONWIDE AG INS CO', 'NATIONWIDE AGRI INS','NATIONWIDE GENERAL','NATIONWIDE INS CO','NATIONWIDE INS OF AM',	'NATIONWIDE MUT FIRE','NATIONWIDE MUT INS','NATIONWIDE MUTUAL IN',
		 'VICTORIA FIRE CASU',	'VICTORIA SPECIALTY I',	'AMCO INSURANCE CO',	'ALLIED P C INS CO',	'HARLEYSVILLEIC',	'HARLEYSVILLEPREFIC',	'NATIONWIDE AFFINITY',
		 'NATIONWIDE GENERAL',	'NATIONWIDE MUTUAL',	'NATIONWIDE PROP CAS',	'VICTORIA SELECT INSU') THEN 'Nationwide'
		 WHEN PCARR_DESC IS NULL  THEN 'NULL' 																								
		 WHEN PCARR_DESC IN ('NOT PRESENTLY INSURED', 'UNKNOWN', 'NO PRIOR INSURANCE') THEN PCARR_DESC
		 WHEN PCARR_DESC IN ( 'OTHER-NON STANDARD' , 'OTHER - NON STANDARD' ) THEN 'OTHER - NON STANDARD'
		 ELSE 'OTHER' END)) as Key,
	{query_4_body} {query_4_from}
),

region_ins_age AS -- Segmenting indicator counts by region insured age
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'Insured Age Group',(																								
	CASE WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 49 THEN 'a. 25-49'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 54 THEN 'b. 50-54'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 59 THEN 'c. 55-59'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 64 THEN 'd. 60-64'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 69 THEN 'e. 65-69'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 74 THEN 'f. 70-74'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 79 THEN 'g. 75-79'																								
		 WHEN DATEDIFF(YEAR,ACCT_DOB,POL_EFF_DT) -1 <= 120 THEN 'h. 80+'																								
		 ELSE 'i. null' END)) as Key,
	{query_4_body} {query_4_from}
),

region_ppv_count AS -- Segmenting indicator counts by region PPV count
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'POL PPV CNT',(																								
	CASE WHEN POL_PPV_CNT = 1 THEN 'b 1' 																								
		 WHEN POL_PPV_CNT = 2 THEN 'c 2' 																								
		 WHEN POL_PPV_CNT = 3 THEN 'd 3' 																								
		 WHEN POL_PPV_CNT = 4 THEN 'e 4' 																								
		 WHEN POL_PPV_CNT = 5 THEN 'f 5+' 																								
		 ELSE 'g Unknown' END)) as Key,
	{query_4_body} {query_4_from}
),

region_telematics AS -- Segmenting indicator counts by region telematics indicator
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'Pol drvr tlmtc enrol ind',Pol_drvr_tlmtc_enrol_ind) as Key,
	{query_4_body} {query_4_from}
),

region_youthful AS -- Segmenting indicator counts by region youthful indicator
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'YOUTH OPER DRVR IND',YOUTH_OPER_DRVR_IND) as Key,
	{query_4_body} {query_4_from}
),

region_garage_sub AS -- Segmenting indicator counts by region garage substate
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'GARAGE SUBSTATE',(																								
	CASE WHEN LEFT(RISK_ST_ABBR,2) = LEFT(GARAGE_SUBSTATE,2) THEN GARAGE_SUBSTATE
	ELSE 'Out of State Mail' END)) as Key,
	{query_4_body}
	FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW" t1
    LEFT JOIN (SELECT AUTO_QTE_TRANS_ID,EXPECTED_LOSS_RATIO FROM "DSC_PLDS_DB"."APP_AUTOMATA_PRD"."PREVAIL_AUTO_ULM_QUOTE_POL_AGG_NB_QUALITY")
    USING (AUTO_QTE_TRANS_ID)
    LEFT JOIN (SELECT state, TO_CHAR(TO_DATE(TRANS_DT, 'YYYY-MM-DD'),'MM-01-YYYY') AS TRANSDT2, AVG(expected_loss_ratio) AS state_avg_lr
               FROM "DSC_PLDS_DB"."APP_AUTOMATA_PRD"."PREVAIL_AUTO_ULM_QUOTE_POL_AGG_NB_QUALITY" WHERE COMPL_QTE_IND =  'Y' GROUP BY state, TRANSDT2) AS t4
	ON t1.RISK_ST_ABBR = t4.state AND TO_CHAR(TRANS_DT,'MM-01-YYYY') = t4.TRANSDT2
	LEFT JOIN (SELECT ZIP_CODE, STATE_ABBR, GARAGE_SUBSTATE FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."STATE_PRODUCT_SUBSTATE") t7
	ON t7.ZIP_CODE = LEFT(MAIL_ADDR_FULL_ZIP_CD,5)	
    WHERE COMPL_QTE_IND = 'Y'
    GROUP BY Key
),

region_prior_bi AS -- Segmenting indicator counts by region prior BI per person
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'Prior BI Per Person',(																								
	CASE WHEN PCARR_BI_PERS_LMT_CD < 50 THEN 'a. Less than 50'																								
		 WHEN PCARR_BI_PERS_LMT_CD < 100 THEN 'b. 50-99'																								
		 WHEN PCARR_BI_PERS_LMT_CD < 250 THEN 'c. 100-249' 																								
		 WHEN PCARR_BI_PERS_LMT_CD < 3000 THEN 'd. 250+' 																								
		 WHEN PCARR_BI_PERS_LMT_CD IS NULL THEN 'f. Unknown/No Insurance'
		 ELSE 'e. other' END)) AS Key,
	{query_4_body} {query_4_from}
),

region_aarp_member AS -- Segmenting indicator counts by region AARP member status
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'AARP Member Status',AARP_MEMB_STAT_DESC) as Key,
	{query_4_body} {query_4_from}
),

region_aqd_group AS -- Segmenting indicator counts by region AQD group
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'Advanced Quote Discount',(																								
    CASE WHEN ADV_QTE_DAY_CNT IN (000,0,999) THEN 'A. 0 Days'																								
		 WHEN ADV_QTE_DAY_CNT <= 13 THEN 'B. 1-13'																								
		 WHEN ADV_QTE_DAY_CNT <= 60 THEN 'C. 14-60'																								
		 WHEN ADV_QTE_DAY_CNT <= 75 THEN 'D. 61-75'																								
		 WHEN ADV_QTE_DAY_CNT <= 99999 THEN 'E. 76+'																								
		 ELSE 'F. Unknown' END)) as Key,
	{query_4_body} {query_4_from}
),

region_renew_uw AS -- Segmenting indicator counts by region renewal UW tier
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'RENEW UW TIER',(																								
	CASE WHEN RENEW_UW_TIER_CD IN ('BB','BC','BD','BE','BF','BG','BH','BI','BJ','BK') THEN RENEW_UW_TIER_CD
		 ELSE 'Other' END)) as Key,
	{query_4_body} {query_4_from}
),

region_min_rating_factor AS -- Segmenting indicator counts by region min rating factor
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'Min Rating Factor',POL_MIN_RATE_FCTR_IND) as Key,
	{query_4_body} {query_4_from}
),

region_max_veh_age AS -- Segmenting indicator counts by region max vehicle age
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'Max Veh Age',(																								
	CASE WHEN POL_MAX_VEH_AGE <= 3 THEN '0-3'
		 WHEN POL_MAX_VEH_AGE <= 6 THEN '4-6'
		 WHEN POL_MAX_VEH_AGE <= 9 THEN '7-9'
		 WHEN POL_MAX_VEH_AGE <= 14 THEN '10-14' 																								
		 WHEN POL_MAX_VEH_AGE <= 19 THEN '15-19'
		 WHEN POL_MAX_VEH_AGE <= 24 THEN '20-24' 																								
		 WHEN POL_MAX_VEH_AGE > 24 THEN '25+'  																								
		 ELSE 'Other' END)) as Key,
	{query_4_body} {query_4_from}
),

region_pol_mult_single AS -- Segmenting indicator counts by region multi-single vehicle code
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'POL MULTI SNGL VEH CD',(																								
	CASE WHEN POL_MULTI_SNGL_VEH_CD IS NULL THEN 'S'
		 ELSE POL_MULTI_SNGL_VEH_CD END)) as Key,
	{query_4_body} {query_4_from}
),

region_mature_driving_course AS -- Segmenting indicator counts by region mature driving indicator
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'Mature Driving Course',(																								
	CASE WHEN POL_MATUR_DRVNG_COURS_CR_IND = 'Y' THEN 'Y'
		 ELSE 'N' END)) as Key,
	{query_4_body} {query_4_from}
),

region_incident_free AS -- Segmenting indicator counts by region incident free year count
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'Incident Free Year Count',(																								
	CASE WHEN POL_INCID_FREE_YR_CNT = 0 THEN '0'																								
		 WHEN POL_INCID_FREE_YR_CNT = 1 THEN '1'																								
		 WHEN POL_INCID_FREE_YR_CNT = 2 THEN '2' 																								
		 WHEN POL_INCID_FREE_YR_CNT = 3 THEN '3' 																								
		 WHEN POL_INCID_FREE_YR_CNT = 4 THEN '4' 																								
		 WHEN POL_INCID_FREE_YR_CNT = 5 THEN '5' END)) as Key,
	{query_4_body} {query_4_from}
),

region_naf_accidents AS -- Segmenting indicator counts by region NAF accident count
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'NAF Accident Count',(																								
	CASE WHEN PCARR_POL_NAF_ACCID_CNT = 0 THEN '0'																								
		 WHEN PCARR_POL_NAF_ACCID_CNT = 1 THEN '1'																								
		 WHEN PCARR_POL_NAF_ACCID_CNT > 1 THEN '2+' END)) as Key,
	{query_4_body} {query_4_from}
),

region_incident_mo_age AS -- Segmenting indicator counts by region incident month age
(
	SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR),'Incident Month Age',(																								
	CASE WHEN POL_INCID_MO_AGE <= 12 THEN '1-12'																								
		 WHEN POL_INCID_MO_AGE <= 24 THEN '13-24'																								
		 WHEN POL_INCID_MO_AGE <= 36 THEN '25-36'																								
		 WHEN POL_INCID_MO_AGE <= 48 THEN '37-48'																								
		 WHEN POL_INCID_MO_AGE <= 60 THEN '49-60' 																								
		 WHEN POL_INCID_MO_AGE = 61 THEN '61' END)) as Key,
	{query_4_body} {query_4_from}
)

SELECT * FROM
(
	SELECT * FROM ins_score
	UNION ALL
	SELECT * FROM household_comp
	UNION ALL
	SELECT * FROM clean_dirty
	UNION ALL
	SELECT * FROM max_age_group
	UNION ALL
	SELECT * FROM min_age_group
	UNION ALL
	SELECT * FROM max_minus_min
	UNION ALL
	SELECT * FROM pcarr_yr_cnt
	UNION ALL
	SELECT * FROM account_credit
	UNION ALL
	SELECT * FROM pol_coverage_pkg
	UNION ALL
	SELECT * FROM pcarr
	UNION ALL
	SELECT * FROM ins_age
	UNION ALL
	SELECT * FROM ppv_count
	UNION ALL
	SELECT * FROM telematics
	UNION ALL
	SELECT * FROM youthful
	UNION ALL
	SELECT * FROM garage_sub
	UNION ALL
	SELECT * FROM prior_bi
	UNION ALL
	SELECT * FROM aarp_member
	UNION ALL
	SELECT * FROM aqd_group
	UNION ALL
	SELECT * FROM renew_uw
	UNION ALL
	SELECT * FROM min_rating_factor
	UNION ALL
	SELECT * FROM max_veh_age
	UNION ALL
	SELECT * FROM pol_mult_single
	UNION ALL
	SELECT * FROM mature_driving_course
	UNION ALL
	SELECT * FROM incident_free
	UNION ALL
	SELECT * FROM naf_accidents
	UNION ALL
	SELECT * FROM incident_mo_age
	UNION ALL
	SELECT * FROM region_ins_score
	UNION ALL
	SELECT * FROM region_hh_comp
	UNION ALL
	SELECT * FROM region_clean_dirty
	UNION ALL
	SELECT * FROM region_max_age_group
	UNION ALL
	SELECT * FROM region_min_age_group
	UNION ALL
	SELECT * FROM region_max_minus_min
	UNION ALL
	SELECT * FROM region_pcarr_yr_cnt
	UNION ALL
	SELECT * FROM region_account_credit
	UNION ALL
	SELECT * FROM region_pol_coverage_pkg
	UNION ALL
	SELECT * FROM region_pcarr
	UNION ALL
	SELECT * FROM region_ins_age
	UNION ALL
	SELECT * FROM region_ppv_count
	UNION ALL
	SELECT * FROM region_telematics
	UNION ALL
	SELECT * FROM region_youthful
	UNION ALL
	SELECT * FROM region_garage_sub
	UNION ALL
	SELECT * FROM region_prior_bi
	UNION ALL
	SELECT * FROM region_aarp_member
	UNION ALL
	SELECT * FROM region_aqd_group
	UNION ALL
	SELECT * FROM region_renew_uw
	UNION ALL
	SELECT * FROM region_min_rating_factor
	UNION ALL
	SELECT * FROM region_max_veh_age
	UNION ALL
	SELECT * FROM region_pol_mult_single
	UNION ALL
	SELECT * FROM region_mature_driving_course
	UNION ALL
	SELECT * FROM region_incident_free
	UNION ALL
	SELECT * FROM region_naf_accidents
	UNION ALL
	SELECT * FROM region_incident_mo_age
)
ORDER BY KEY
"""

In [12]:
cs = ctx.cursor()
try:
    cs.execute(auto_seg_cw)
finally:
    cs.close()
print('Done')

Done


## 5. NowCo Auto Seg Table Creation (DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_AUTO_SEG_NOWCO)

In [13]:
query_5_body = """
    -- Completed Quote Total
    ind_funnel_scorecard(12, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "12 CQ Tot",
    ind_funnel_scorecard(11, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "11 CQ Tot",
    ind_funnel_scorecard(10, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "10 CQ Tot",
    ind_funnel_scorecard(9, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "9 CQ Tot",
    ind_funnel_scorecard(8, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "8 CQ Tot",
    ind_funnel_scorecard(7, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "7 CQ Tot",
    ind_funnel_scorecard(6, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "6 CQ Tot",
    ind_funnel_scorecard(5, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "5 CQ Tot",
    ind_funnel_scorecard(4, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "4 CQ Tot",
    ind_funnel_scorecard(3, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "3 CQ Tot",
    ind_funnel_scorecard(2, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "2 CQ Tot",
    ind_funnel_scorecard(1, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "1 CQ Tot",
    ind_funnel_scorecard(0, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT) AS "0 CQ Tot",
    -- Issue Total
    ind_funnel_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT) AS "12 Iss Tot",
    ind_funnel_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT) AS "11 Iss Tot",
    ind_funnel_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT) AS "10 Iss Tot",
    ind_funnel_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT) AS "9 Iss Tot",
    ind_funnel_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT) AS "8 Iss Tot",
    ind_funnel_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT) AS "7 Iss Tot",
    ind_funnel_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT) AS "6 Iss Tot",
    ind_funnel_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT) AS "5 Iss Tot",
    ind_funnel_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT) AS "4 Iss Tot",
    ind_funnel_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT) AS "3 Iss Tot",
    ind_funnel_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT) AS "2 Iss Tot",
    ind_funnel_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT) AS "1 Iss Tot",
    ind_funnel_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND_CNT) AS "0 Iss Tot",
    -- AQP Total
    one_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "12 AQP Tot",
    one_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "11 AQP Tot",
    one_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "10 AQP Tot",
    one_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "9 AQP Tot",
    one_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "8 AQP Tot",
    one_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "7 AQP Tot",
    one_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "6 AQP Tot",
    one_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "5 AQP Tot",
    one_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "4 AQP Tot",
    one_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "3 AQP Tot",
    one_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "2 AQP Tot",
    one_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "1 AQP Tot",
    one_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "0 AQP Tot",
    -- AIP Total
    one_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "12 AIP Tot",
    one_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "11 AIP Tot",
    one_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "10 AIP Tot",
    one_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "9 AIP Tot",
    one_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "8 AIP Tot",
    one_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "7 AIP Tot",
    one_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "6 AIP Tot",
    one_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "5 AIP Tot",
    one_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "4 AIP Tot",
    one_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "3 AIP Tot",
    one_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "2 AIP Tot",
    one_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "1 AIP Tot",
    one_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "0 AIP Tot",
    -- HIGSRP
    AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 12 AND BUS_UNIT_ABBR = 'AARP' AND COMPL_QUOTE_IND_CNT = 1 AND PRIOR_CARR_POL_ANNUAL_PREM_AMT >= 250
    AND ANNUALIZED_PREM_AMT > 300 AND ORIG_QUOTE_PROCESS_METH_CD = 'T' THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) AS "12 HIGSRP",
    AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 11 AND BUS_UNIT_ABBR = 'AARP' AND COMPL_QUOTE_IND_CNT = 1 AND PRIOR_CARR_POL_ANNUAL_PREM_AMT >= 250
    AND ANNUALIZED_PREM_AMT > 300 AND ORIG_QUOTE_PROCESS_METH_CD = 'T' THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) AS "11 HIGSRP",
    AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 10 AND BUS_UNIT_ABBR = 'AARP' AND COMPL_QUOTE_IND_CNT = 1 AND PRIOR_CARR_POL_ANNUAL_PREM_AMT >= 250
    AND ANNUALIZED_PREM_AMT > 300 AND ORIG_QUOTE_PROCESS_METH_CD = 'T' THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) AS "10 HIGSRP",
    AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 9 AND BUS_UNIT_ABBR = 'AARP' AND COMPL_QUOTE_IND_CNT = 1 AND PRIOR_CARR_POL_ANNUAL_PREM_AMT >= 250
    AND ANNUALIZED_PREM_AMT > 300 AND ORIG_QUOTE_PROCESS_METH_CD = 'T' THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) AS "9 HIGSRP",
    AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 8 AND BUS_UNIT_ABBR = 'AARP' AND COMPL_QUOTE_IND_CNT = 1 AND PRIOR_CARR_POL_ANNUAL_PREM_AMT >= 250
    AND ANNUALIZED_PREM_AMT > 300 AND ORIG_QUOTE_PROCESS_METH_CD = 'T' THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) AS "8 HIGSRP",
    AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 7 AND BUS_UNIT_ABBR = 'AARP' AND COMPL_QUOTE_IND_CNT = 1 AND PRIOR_CARR_POL_ANNUAL_PREM_AMT >= 250
    AND ANNUALIZED_PREM_AMT > 300 AND ORIG_QUOTE_PROCESS_METH_CD = 'T' THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) AS "7 HIGSRP",
    AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 6 AND BUS_UNIT_ABBR = 'AARP' AND COMPL_QUOTE_IND_CNT = 1 AND PRIOR_CARR_POL_ANNUAL_PREM_AMT >= 250
    AND ANNUALIZED_PREM_AMT > 300 AND ORIG_QUOTE_PROCESS_METH_CD = 'T' THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) AS "6 HIGSRP",
    AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 5 AND BUS_UNIT_ABBR = 'AARP' AND COMPL_QUOTE_IND_CNT = 1 AND PRIOR_CARR_POL_ANNUAL_PREM_AMT >= 250
    AND ANNUALIZED_PREM_AMT > 300 AND ORIG_QUOTE_PROCESS_METH_CD = 'T' THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) AS "5 HIGSRP",
    AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 4 AND BUS_UNIT_ABBR = 'AARP' AND COMPL_QUOTE_IND_CNT = 1 AND PRIOR_CARR_POL_ANNUAL_PREM_AMT >= 250
    AND ANNUALIZED_PREM_AMT > 300 AND ORIG_QUOTE_PROCESS_METH_CD = 'T' THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) AS "4 HIGSRP",
    AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 3 AND BUS_UNIT_ABBR = 'AARP' AND COMPL_QUOTE_IND_CNT = 1 AND PRIOR_CARR_POL_ANNUAL_PREM_AMT >= 250
    AND ANNUALIZED_PREM_AMT > 300 AND ORIG_QUOTE_PROCESS_METH_CD = 'T' THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) AS "3 HIGSRP",
    AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 2 AND BUS_UNIT_ABBR = 'AARP' AND COMPL_QUOTE_IND_CNT = 1 AND PRIOR_CARR_POL_ANNUAL_PREM_AMT >= 250
    AND ANNUALIZED_PREM_AMT > 300 AND ORIG_QUOTE_PROCESS_METH_CD = 'T' THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) AS "2 HIGSRP",
    AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 1 AND BUS_UNIT_ABBR = 'AARP' AND COMPL_QUOTE_IND_CNT = 1 AND PRIOR_CARR_POL_ANNUAL_PREM_AMT >= 250
    AND ANNUALIZED_PREM_AMT > 300 AND ORIG_QUOTE_PROCESS_METH_CD = 'T' THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) AS "1 HIGSRP",
    AVG(CASE WHEN DATEDIFF(MONTH, QUOTE_LAST_TRANSACTION_DT, SYSDATE()) = 0 AND BUS_UNIT_ABBR = 'AARP' AND COMPL_QUOTE_IND_CNT = 1 AND PRIOR_CARR_POL_ANNUAL_PREM_AMT >= 250
    AND ANNUALIZED_PREM_AMT > 300 AND ORIG_QUOTE_PROCESS_METH_CD = 'T' THEN ANNUALIZED_PREM_AMT / PRIOR_CARR_POL_ANNUAL_PREM_AMT ELSE NULL END) AS "0 HIGSRP",
    -- CQ Phone
    one_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT) AS "12 CQ Phone",
    one_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT) AS "11 CQ Phone",
    one_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT) AS "10 CQ Phone",
    one_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT) AS "9 CQ Phone",
    one_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT) AS "8 CQ Phone",
    one_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT) AS "7 CQ Phone",
    one_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT) AS "6 CQ Phone",
    one_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT) AS "5 CQ Phone",
    one_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT) AS "4 CQ Phone",
    one_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT) AS "3 CQ Phone",
    one_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT) AS "2 CQ Phone",
    one_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT) AS "1 CQ Phone",
    one_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT) AS "0 CQ Phone",
    -- Issue Phone
    one_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT) AS "12 Issue Phone",
    one_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT) AS "11 Issue Phone",
    one_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT) AS "10 Issue Phone",
    one_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT) AS "9 Issue Phone",
    one_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT) AS "8 Issue Phone",
    one_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT) AS "7 Issue Phone",
    one_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT) AS "6 Issue Phone",
    one_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT) AS "5 Issue Phone",
    one_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT) AS "4 Issue Phone",
    one_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT) AS "3 Issue Phone",
    one_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT) AS "2 Issue Phone",
    one_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT) AS "1 Issue Phone",
    one_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT) AS "0 Issue Phone",
    -- AQP Phone
    two_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "12 AQP Phone",
    two_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "11 AQP Phone",
    two_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "10 AQP Phone",
    two_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "9 AQP Phone",
    two_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "8 AQP Phone",
    two_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "7 AQP Phone",
    two_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "6 AQP Phone",
    two_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "5 AQP Phone",
    two_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "4 AQP Phone",
    two_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "3 AQP Phone",
    two_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "2 AQP Phone",
    two_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "1 AQP Phone",
    two_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT) AS "0 AQP Phone",
    -- AIP Phone
    two_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "12 AIP Phone",
    two_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "11 AIP Phone",
    two_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "10 AIP Phone",
    two_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "9 AIP Phone",
    two_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "8 AIP Phone",
    two_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "7 AIP Phone",
    two_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "6 AIP Phone",
    two_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "5 AIP Phone",
    two_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "4 AIP Phone",
    two_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "3 AIP Phone",
    two_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "2 AIP Phone",
    two_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "1 AIP Phone",
    two_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT) AS "0 AIP Phone",
    -- CQ Internet
    one_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, TRUE) AS "12 CQ Internet",
    one_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, TRUE) AS "11 CQ Internet",
    one_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, TRUE) AS "10 CQ Internet",
    one_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, TRUE) AS "9 CQ Internet",
    one_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, TRUE) AS "8 CQ Internet",
    one_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, TRUE) AS "7 CQ Internet",
    one_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, TRUE) AS "6 CQ Internet",
    one_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, TRUE) AS "5 CQ Internet",
    one_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, TRUE) AS "4 CQ Internet",
    one_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, TRUE) AS "3 CQ Internet",
    one_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, TRUE) AS "2 CQ Internet",
    one_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, TRUE) AS "1 CQ Internet",
    one_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, TRUE) AS "0 CQ Internet",
    -- Issue Internet
    one_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT, TRUE) AS "12 Issue Internet",
    one_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT, TRUE) AS "11 Issue Internet",
    one_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT, TRUE) AS "10 Issue Internet",
    one_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT, TRUE) AS "9 Issue Internet",
    one_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT, TRUE) AS "8 Issue Internet",
    one_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT, TRUE) AS "7 Issue Internet",
    one_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT, TRUE) AS "6 Issue Internet",
    one_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT, TRUE) AS "5 Issue Internet",
    one_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT, TRUE) AS "4 Issue Internet",
    one_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT, TRUE) AS "3 Issue Internet",
    one_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT, TRUE) AS "2 Issue Internet",
    one_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT, TRUE) AS "1 Issue Internet",
    one_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND_CNT, TRUE) AS "0 Issue Internet",
    -- AQP Internet
    two_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT, TRUE) AS "12 AQP Internet",
    two_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT, TRUE) AS "11 AQP Internet",
    two_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT, TRUE) AS "10 AQP Internet",
    two_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT, TRUE) AS "9 AQP Internet",
    two_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT, TRUE) AS "8 AQP Internet",
    two_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT, TRUE) AS "7 AQP Internet",
    two_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT, TRUE) AS "6 AQP Internet",
    two_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT, TRUE) AS "5 AQP Internet",
    two_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT, TRUE) AS "4 AQP Internet",
    two_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT, TRUE) AS "3 AQP Internet",
    two_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT, TRUE) AS "2 AQP Internet",
    two_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT, TRUE) AS "1 AQP Internet",
    two_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', COMPL_QUOTE_IND_CNT, '1', ANNUALIZED_PREM_AMT, TRUE) AS "0 AQP Internet",
    -- AIP Internet
    two_cond_scorecard(12, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT, TRUE) AS "12 AIP Internet",
    two_cond_scorecard(11, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT, TRUE) AS "11 AIP Internet",
    two_cond_scorecard(10, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT, TRUE) AS "10 AIP Internet",
    two_cond_scorecard(9, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT, TRUE) AS "9 AIP Internet",
    two_cond_scorecard(8, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT, TRUE) AS "8 AIP Internet",
    two_cond_scorecard(7, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT, TRUE) AS "7 AIP Internet",
    two_cond_scorecard(6, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT, TRUE) AS "6 AIP Internet",
    two_cond_scorecard(5, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT, TRUE) AS "5 AIP Internet",
    two_cond_scorecard(4, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT, TRUE) AS "4 AIP Internet",
    two_cond_scorecard(3, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT, TRUE) AS "3 AIP Internet",
    two_cond_scorecard(2, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT, TRUE) AS "2 AIP Internet",
    two_cond_scorecard(1, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT, TRUE) AS "1 AIP Internet",
    two_cond_scorecard(0, QUOTE_LAST_TRANSACTION_DT, ORIG_QUOTE_PROCESS_METH_CD, 'T', ISSUE_IND, 'Y', ANNUALIZED_PREM_AMT, TRUE) AS "0 AIP Internet",
    TO_CHAR(SYSDATE(),'MM-DD-YYYY') AS AS_OF
"""

query_5_from = """
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW" 																						
    WHERE BUS_UNIT_ABBR <> 'CDRT' AND COMPL_QUOTE_IND_CNT = 1 
    AND CONCAT(RISK_STATE_ABBR,BUS_UNIT_ABBR) NOT IN ('FLAARP','SDAARP','MEAARP','VTAARP','NEAARP','WVAARP','LAAARP','IAAARP','ARAARP','MSAARP','DEAARP','ALAARP',
    'KYAARP','TNAARP','WIAARP','GAAARP','MOAARP','UTAARP','INAARP','NJAARP','ORAARP','COAARP','OHAARP','ILAARP','AZAARP','TXAARP')
    GROUP BY Key
"""

In [14]:
auto_seg_nowco = f"""
CREATE OR REPLACE TRANSIENT TABLE DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_AUTO_SEG_NOWCO AS

WITH ins_decile AS -- Segmenting indicator counts by ins score decile
(
    SELECT CONCAT(RISK_STATE_ABBR, 'Ins Score Decile',
    CASE WHEN (INS_SCORE_CD) IN ('ERR','NUS','FAI','XXX','SUP','ORD') THEN 'NHNS'
         WHEN (INS_SCORE_CD) IN (000,0,999) THEN 'NHNS' 
         WHEN (INS_SCORE_CD) <= 571 THEN 'Decile 10 Worst'
         WHEN (INS_SCORE_CD) <= 630 THEN 'Decile 09'
         WHEN (INS_SCORE_CD) <= 698 THEN 'Decile 08'
         WHEN (INS_SCORE_CD) <= 753 THEN 'Decile 07'
         WHEN (INS_SCORE_CD) <= 790 THEN 'Decile 06'
         WHEN (INS_SCORE_CD) <= 813 THEN 'Decile 05'
         WHEN (INS_SCORE_CD) <= 831 THEN 'Decile 04'
         WHEN (INS_SCORE_CD) <= 856 THEN 'Decile 03'
         WHEN (INS_SCORE_CD) <= 891 THEN 'Decile 02'
         WHEN (INS_SCORE_CD) <= 979 THEN 'Decile 01 Best'
         ELSE 'NHNS' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

clean_dirty AS -- Segmenting indicator counts by clean/dirty code
(
    SELECT CONCAT(RISK_STATE_ABBR,'CLEAN DIRTY DESC', CLEAN_DIRTY_DESC, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

driver_age AS -- Segmenting indicator counts by driver age range
(
    SELECT CONCAT(RISK_STATE_ABBR,'DRIVER_AGE_RANGE',																						
    CASE WHEN DRIVER_AGE_RANGE IN ('16-20','21-29','< 30') THEN '< 30'
         WHEN DRIVER_AGE_RANGE IN ('30-49') THEN '30-49'
         WHEN DRIVER_AGE_RANGE IN ('50-54','55-59','60-64','50-64') THEN '50-64'
         WHEN DRIVER_AGE_RANGE IN ('65-69','70-74','65-74') THEN '65-74'
         WHEN DRIVER_AGE_RANGE IN ('75-79') THEN '75-79'
         WHEN DRIVER_AGE_RANGE IN ('80 +') THEN '80 +'
         ELSE 'other' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

driver_max AS -- Segmenting indicator counts by driver max age
(
    SELECT CONCAT(RISK_STATE_ABBR,'DRIVER_MAX_AGE',																						
    CASE WHEN DRIVER_MAX_AGE = 'UKWN' THEN 'Unknown'																						
         WHEN DRIVER_MAX_AGE < 50 THEN '50'																						
         WHEN DRIVER_MAX_AGE <= 59 THEN '50-59' 																						
         WHEN DRIVER_MAX_AGE <= 69 THEN '50-69' 																						
         WHEN DRIVER_MAX_AGE <= 79 THEN '70-79' 																						
         WHEN DRIVER_MAX_AGE <= 130 THEN '80+'
         ELSE 'Unknown' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

driver_min AS -- Segmenting indicator counts by driver min age
(
    SELECT CONCAT(RISK_STATE_ABBR,'DRIVER_MIN_AGE',																						
    CASE WHEN DRIVER_MIN_AGE = 'UKWN' THEN 'Unknown'																						
         WHEN DRIVER_MIN_AGE < 50 THEN '50'																						
         WHEN DRIVER_MIN_AGE <= 59 THEN '50-59' 																						
         WHEN DRIVER_MIN_AGE <= 69 THEN '50-69' 																						
         WHEN DRIVER_MIN_AGE <= 79 THEN '70-79' 																						
         WHEN DRIVER_MIN_AGE <= 130 THEN '80+'
         ELSE 'Unknown' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

max_minus_min AS -- Segmenting indicator counts by max - min age group
(
    SELECT CONCAT(RISK_STATE_ABBR,'Max Minus Min Age Group',																						
    CASE WHEN DRIVER_MAX_AGE = 'UKWN' OR DRIVER_MIN_AGE = 'UKWN' THEN 'f. n/a'																					
         WHEN DRIVER_MAX_AGE - DRIVER_MIN_AGE = 0 THEN 'a. 0' 
         WHEN DRIVER_MAX_AGE - DRIVER_MIN_AGE BETWEEN 1 AND 2 THEN 'b. 1-2'
         WHEN DRIVER_MAX_AGE - DRIVER_MIN_AGE BETWEEN 3 AND 6 THEN 'c. 3-6'
         WHEN DRIVER_MAX_AGE - DRIVER_MIN_AGE BETWEEN 7 AND 20 THEN 'd. 7-20'																					
         WHEN DRIVER_MAX_AGE - DRIVER_MIN_AGE >= 21 THEN 'e. 21+' 																						
         ELSE 'f. n/a' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

pcarr_yr AS -- Segmenting indicator counts by pcarr year count
(
    SELECT CONCAT(RISK_STATE_ABBR,'PCARR YR CNT',																						
    CASE WHEN PRIOR_CARR_YR_CNT = 0 THEN 'a 0'																						
         WHEN PRIOR_CARR_YR_CNT = 1 THEN 'b 1' 																						
         WHEN PRIOR_CARR_YR_CNT = 2 THEN 'c 2' 																						
         WHEN PRIOR_CARR_YR_CNT = 3 THEN 'd 3' 																						
         WHEN PRIOR_CARR_YR_CNT = 4 THEN 'e 4' 																						
         WHEN PRIOR_CARR_YR_CNT >= 5 THEN 'f 5+' 																						
         ELSE 'g Unknown' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

account_credit AS -- Segmenting indicator counts by account credit indicator
(
    SELECT CONCAT(RISK_STATE_ABBR,'ACCT CR IND', CASE WHEN ACCT_CR_IND = 'Y' THEN 'Y' ELSE 'N' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

pol_cov_pkg AS -- Segmenting indicator counts by policy coverage package code
(
    SELECT CONCAT(RISK_STATE_ABBR,'POL COV PACKAGE CD',																						
    CASE WHEN POL_COV_PACKAGE_CD IS NULL OR POL_COV_PACKAGE_CD = 'NO COV' THEN 'Unknown'
         ELSE POL_COV_PACKAGE_CD END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

pcarr AS -- Segmenting indicator counts by prior carrier
(
    SELECT CONCAT(RISK_STATE_ABBR,'Prior Carrier',																						
    CASE WHEN PRIOR_CARR_DESC IN ('STATE FARM','PROGRESSIVE','GEICO','ALLSTATE','LIBERTY MUTUAL','TRAVELERS',
    'USAA','AMERICAN FAMILY','NATIONWIDE','NOT PRESENTLY INSURED','HARTFORD') THEN PRIOR_CARR_DESC 
         WHEN PRIOR_CARR_DESC IN ('ALLSTATE INDEMNITY') THEN 'ALLSTATE' 
         ELSE 'OTHER' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

youthful AS -- Segmenting indicator counts by youthful indicator
(
    SELECT CONCAT(RISK_STATE_ABBR, 'POL YOUTHFUL OP IND', POL_YOUTHFUL_OP_IND, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

market_index AS -- Segmenting indicator counts by market index
(
    SELECT CONCAT(RISK_STATE_ABBR,'Market Index',																						
    CASE WHEN MKT_INDX_CTGY_CD IN ('A','B','C','D','E') THEN MKT_INDX_CTGY_CD 
         ELSE 'Unknown' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

vehicle_count AS 
(
    SELECT CONCAT(RISK_STATE_ABBR,'Vehicle Cnt',																						
    CASE WHEN veh_cnt IN ('0', '1', '2', '3', '4', '5') THEN TO_VARCHAR(veh_cnt)
         ELSE '0' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

garage_sub AS -- Segmenting indicator counts by garage substate
(
    SELECT CONCAT(RISK_STATE_ABBR,'Garage Substate',																						
    CASE WHEN LEFT(RISK_STATE_ABBR,2) = LEFT(GARAGE_SUBSTATE,2) THEN GARAGE_SUBSTATE
         ELSE 'Out of State Mail' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body}																							
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW" t1																						
	LEFT JOIN (SELECT ZIP_CODE, STATE_ABBR, GARAGE_SUBSTATE from "DSC_PLBI_DB"."APP_AUTO_PRD"."STATE_PRODUCT_SUBSTATE") t7 
    ON t7.ZIP_CODE = t1.CUST_MAILING_ZIP_CD
    WHERE BUS_UNIT_ABBR <> 'CDRT' AND COMPL_QUOTE_IND_CNT = 1 
    AND CONCAT(RISK_STATE_ABBR,BUS_UNIT_ABBR) NOT IN ('FLAARP','SDAARP','MEAARP','VTAARP','NEAARP','WVAARP','LAAARP','IAAARP','ARAARP','MSAARP','DEAARP','ALAARP',
    'KYAARP','TNAARP','WIAARP','GAAARP','MOAARP','UTAARP','INAARP','NJAARP','ORAARP','COAARP','OHAARP','ILAARP','AZAARP','TXAARP')
    GROUP BY Key
),

prior_bi AS -- Segmenting indicator counts by prior BI
(
    SELECT CONCAT(RISK_STATE_ABBR,'Prior BI Per Person',																						
    CASE WHEN PRIOR_CARR_BI_PERSON_LMT < 50 THEN 'a. Less than 50'																						
         WHEN PRIOR_CARR_BI_PERSON_LMT < 100 THEN 'b. 50-99'																						
         WHEN PRIOR_CARR_BI_PERSON_LMT < 250 THEN 'c. 100-249' 																						
         WHEN PRIOR_CARR_BI_PERSON_LMT < 3000 THEN 'd. 250+' 																						
         WHEN PRIOR_CARR_BI_PERSON_LMT IS NULL THEN 'f. Unknown/No Insurance'																						
         ELSE 'e. other' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

aarp_status AS -- Segmenting indicator counts by status
(
    SELECT CONCAT(RISK_STATE_ABBR,'AARP Member Status',																						
    CASE WHEN AARP_MEMB_STATUS_DESC IN ('AARP New Member') THEN 'New Member' 
         WHEN AARP_MEMB_STATUS_DESC IN ('AARP Tenured Member') THEN 'Tenured Member'
         ELSE 'Other' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

adv_qte_discount AS -- Segmenting indicator counts by advanced quote discount
(
    SELECT CONCAT(RISK_STATE_ABBR,'Advanced Quote Discount',																						
    CASE WHEN ADV_QT_DAY_CNT in (000,0,999) THEN 'A. 0 Days'																						
         WHEN ADV_QT_DAY_CNT <= 13 THEN 'B. 1-13'																						
         WHEN ADV_QT_DAY_CNT <= 60 THEN 'C. 14-60'																						
         WHEN ADV_QT_DAY_CNT <= 75 THEN 'D. 61-75'																						
         WHEN ADV_QT_DAY_CNT <= 99999 THEN 'E. 76+'																						
         ELSE 'F. Unknown' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

max_veh_age AS -- Segmenting indicator counts by max vehicle age
(
    SELECT CONCAT(RISK_STATE_ABBR,'Max Veh Age',																						
    CASE WHEN POL_MAX_VEH_AGE <= 3 THEN '0-3'
         WHEN POL_MAX_VEH_AGE <= 6 THEN '4-6'
         WHEN POL_MAX_VEH_AGE <= 9 THEN '7-9'
         WHEN POL_MAX_VEH_AGE <= 14 THEN '10-14' 																						
         WHEN POL_MAX_VEH_AGE <= 19 THEN '15-19'
         WHEN POL_MAX_VEH_AGE <= 24 THEN '20-24' 																						
         WHEN POL_MAX_VEH_AGE > 24 THEN '25+'  																						
         ELSE 'Other' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

pol_multi_single_veh AS -- Segmenting indicator counts by multi single veh pol code
(
    SELECT CONCAT(RISK_STATE_ABBR,'POL MULTI SNGL VEH CD', MULTI_SINGLE_VEH_POL_CD, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

household_comp AS -- Segmenting indicator counts by household comp
(
    SELECT CONCAT(RISK_STATE_ABBR, 'HH COMP', POLICY_CAR_DRIVER_CD, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

naf_accid AS -- Segmenting indicator counts by NAF accidents
(
    SELECT CONCAT(RISK_STATE_ABBR,'NAF Accident Count',																						
    CASE WHEN POL_NAF_ACCD_CNT = 0 OR POL_NAF_ACCD_CNT IS NULL THEN '0'
         WHEN POL_NAF_ACCD_CNT = 1 THEN '1'
         ELSE '2+' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

inc_month AS -- Segmenting indicator counts by incident month
(
    SELECT CONCAT(RISK_STATE_ABBR,'Incident Month Age',																						
    CASE WHEN POL_LATEST_INCID_MO_AGE <= 12 THEN '1-12'																						
         WHEN POL_LATEST_INCID_MO_AGE <= 24 THEN '13-24'																						
         WHEN POL_LATEST_INCID_MO_AGE <= 36 THEN '25-36'																						
         WHEN POL_LATEST_INCID_MO_AGE <= 48 THEN '37-48'																						
         WHEN POL_LATEST_INCID_MO_AGE <= 60 THEN '49-60' 																						
         WHEN POL_LATEST_INCID_MO_AGE = 99 THEN 'None' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
),

quote_vender AS
(
    SELECT CONCAT(RISK_STATE_ABBR,'Quote Vender Category',																						
    CASE WHEN QUOTE_VENDOR_CTGY_CD IN ('Comp Rater','Ez Lynx','QTI') THEN QUOTE_VENDOR_CTGY_CD
         ELSE 'Direct' END, BUS_UNIT_ABBR) AS Key,
    {query_5_body} {query_5_from}
)

SELECT * FROM
(
    SELECT * FROM ins_decile
    UNION ALL
    SELECT * FROM clean_dirty
    UNION ALL
    SELECT * FROM driver_age
    UNION ALL
    SELECT * FROM driver_max
    UNION ALL
    SELECT * FROM driver_min
    UNION ALL
    SELECT * FROM max_minus_min
    UNION ALL
    SELECT * FROM pcarr_yr
    UNION ALL
    SELECT * FROM account_credit
    UNION ALL
    SELECT * FROM pol_cov_pkg
    UNION ALL
    SELECT * FROM pcarr
    UNION ALL
    SELECT * FROM youthful
    UNION ALL
    SELECT * FROM market_index
    UNION ALL
    SELECT * FROM vehicle_count
    UNION ALL
    SELECT * FROM garage_sub
    UNION ALL
    SELECT * FROM prior_bi
    UNION ALL
    SELECT * FROM aarp_status
    UNION ALL
    SELECT * FROM adv_qte_discount
    UNION ALL
    SELECT * FROM max_veh_age
    UNION ALL
    SELECT * FROM pol_multi_single_veh
    UNION ALL
    SELECT * FROM household_comp
    UNION ALL
    SELECT * FROM naf_accid
    UNION ALL
    SELECT * FROM inc_month
    UNION ALL
    SELECT * FROM quote_vender
)
ORDER BY KEY
"""

In [15]:
cs = ctx.cursor()
try:
    cs.execute(auto_seg_nowco)
finally:
    cs.close()
print('Done')

Done


## 6. New Funnel Table Creation (DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_NEW_FUNNEL)

In [16]:
query_6_body = """
    -- Response
    ind_funnel_scorecard(1, QUOTE_DT, RESPONSE_IND) AS "01 RESPONSE",
    ind_funnel_scorecard(2, QUOTE_DT, RESPONSE_IND) AS "02 RESPONSE",
    ind_funnel_scorecard(3, QUOTE_DT, RESPONSE_IND) AS "03 RESPONSE",
    ind_funnel_scorecard(4, QUOTE_DT, RESPONSE_IND) AS "04 RESPONSE",
    ind_funnel_scorecard(5, QUOTE_DT, RESPONSE_IND) AS "05 RESPONSE",
    ind_funnel_scorecard(6, QUOTE_DT, RESPONSE_IND) AS "06 RESPONSE",
    ind_funnel_scorecard(7, QUOTE_DT, RESPONSE_IND) AS "07 RESPONSE",
    ind_funnel_scorecard(8, QUOTE_DT, RESPONSE_IND) AS "08 RESPONSE",
    ind_funnel_scorecard(9, QUOTE_DT, RESPONSE_IND) AS "09 RESPONSE",
    ind_funnel_scorecard(10, QUOTE_DT, RESPONSE_IND) AS "10 RESPONSE",
    ind_funnel_scorecard(11, QUOTE_DT, RESPONSE_IND) AS "11 RESPONSE",
    ind_funnel_scorecard(12, QUOTE_DT, RESPONSE_IND) AS "12 RESPONSE",
    ind_funnel_scorecard(13, QUOTE_DT, RESPONSE_IND) AS "13 RESPONSE",
    ind_funnel_scorecard(14, QUOTE_DT, RESPONSE_IND) AS "14 RESPONSE",
    ind_funnel_scorecard(15, QUOTE_DT, RESPONSE_IND) AS "15 RESPONSE",
    ind_funnel_scorecard(16, QUOTE_DT, RESPONSE_IND) AS "16 RESPONSE",
    ind_funnel_scorecard(17, QUOTE_DT, RESPONSE_IND) AS "17 RESPONSE",
    ind_funnel_scorecard(18, QUOTE_DT, RESPONSE_IND) AS "18 RESPONSE",
    ind_funnel_scorecard(19, QUOTE_DT, RESPONSE_IND) AS "19 RESPONSE",
    -- Success
    ind_funnel_scorecard(1, QUOTE_DT, SUCCESS_IND) AS "01 SUCCESS",
    ind_funnel_scorecard(2, QUOTE_DT, SUCCESS_IND) AS "02 SUCCESS",
    ind_funnel_scorecard(3, QUOTE_DT, SUCCESS_IND) AS "03 SUCCESS",
    ind_funnel_scorecard(4, QUOTE_DT, SUCCESS_IND) AS "04 SUCCESS",
    ind_funnel_scorecard(5, QUOTE_DT, SUCCESS_IND) AS "05 SUCCESS",
    ind_funnel_scorecard(6, QUOTE_DT, SUCCESS_IND) AS "06 SUCCESS",
    ind_funnel_scorecard(7, QUOTE_DT, SUCCESS_IND) AS "07 SUCCESS",
    ind_funnel_scorecard(8, QUOTE_DT, SUCCESS_IND) AS "08 SUCCESS",
    ind_funnel_scorecard(9, QUOTE_DT, SUCCESS_IND) AS "09 SUCCESS",
    ind_funnel_scorecard(10, QUOTE_DT, SUCCESS_IND) AS "10 SUCCESS",
    ind_funnel_scorecard(11, QUOTE_DT, SUCCESS_IND) AS "11 SUCCESS",
    ind_funnel_scorecard(12, QUOTE_DT, SUCCESS_IND) AS "12 SUCCESS",
    ind_funnel_scorecard(13, QUOTE_DT, SUCCESS_IND) AS "13 SUCCESS",
    ind_funnel_scorecard(14, QUOTE_DT, SUCCESS_IND) AS "14 SUCCESS",
    ind_funnel_scorecard(15, QUOTE_DT, SUCCESS_IND) AS "15 SUCCESS",
    ind_funnel_scorecard(16, QUOTE_DT, SUCCESS_IND) AS "16 SUCCESS",
    ind_funnel_scorecard(17, QUOTE_DT, SUCCESS_IND) AS "17 SUCCESS",
    ind_funnel_scorecard(18, QUOTE_DT, SUCCESS_IND) AS "18 SUCCESS",
    ind_funnel_scorecard(19, QUOTE_DT, SUCCESS_IND) AS "19 SUCCESS",
    -- Competitive
    ind_funnel_scorecard(1, QUOTE_DT, COMPETITIVE_IND) AS "01 COMPETITIVE",
    ind_funnel_scorecard(2, QUOTE_DT, COMPETITIVE_IND) AS "02 COMPETITIVE",
    ind_funnel_scorecard(3, QUOTE_DT, COMPETITIVE_IND) AS "03 COMPETITIVE",
    ind_funnel_scorecard(4, QUOTE_DT, COMPETITIVE_IND) AS "04 COMPETITIVE",
    ind_funnel_scorecard(5, QUOTE_DT, COMPETITIVE_IND) AS "05 COMPETITIVE",
    ind_funnel_scorecard(6, QUOTE_DT, COMPETITIVE_IND) AS "06 COMPETITIVE",
    ind_funnel_scorecard(7, QUOTE_DT, COMPETITIVE_IND) AS "07 COMPETITIVE",
    ind_funnel_scorecard(8, QUOTE_DT, COMPETITIVE_IND) AS "08 COMPETITIVE",
    ind_funnel_scorecard(9, QUOTE_DT, COMPETITIVE_IND) AS "09 COMPETITIVE",
    ind_funnel_scorecard(10, QUOTE_DT, COMPETITIVE_IND) AS "10 COMPETITIVE",
    ind_funnel_scorecard(11, QUOTE_DT, COMPETITIVE_IND) AS "11 COMPETITIVE",
    ind_funnel_scorecard(12, QUOTE_DT, COMPETITIVE_IND) AS "12 COMPETITIVE",
    ind_funnel_scorecard(13, QUOTE_DT, COMPETITIVE_IND) AS "13 COMPETITIVE",
    ind_funnel_scorecard(14, QUOTE_DT, COMPETITIVE_IND) AS "14 COMPETITIVE",
    ind_funnel_scorecard(15, QUOTE_DT, COMPETITIVE_IND) AS "15 COMPETITIVE",
    ind_funnel_scorecard(16, QUOTE_DT, COMPETITIVE_IND) AS "16 COMPETITIVE",
    ind_funnel_scorecard(17, QUOTE_DT, COMPETITIVE_IND) AS "17 COMPETITIVE",
    ind_funnel_scorecard(18, QUOTE_DT, COMPETITIVE_IND) AS "18 COMPETITIVE",
    ind_funnel_scorecard(19, QUOTE_DT, COMPETITIVE_IND) AS "19 COMPETITIVE",
    -- Win
    ind_funnel_scorecard(1, QUOTE_DT, WIN_IND) AS "01 WIN",
    ind_funnel_scorecard(2, QUOTE_DT, WIN_IND) AS "02 WIN",
    ind_funnel_scorecard(3, QUOTE_DT, WIN_IND) AS "03 WIN",
    ind_funnel_scorecard(4, QUOTE_DT, WIN_IND) AS "04 WIN",
    ind_funnel_scorecard(5, QUOTE_DT, WIN_IND) AS "05 WIN",
    ind_funnel_scorecard(6, QUOTE_DT, WIN_IND) AS "06 WIN",
    ind_funnel_scorecard(7, QUOTE_DT, WIN_IND) AS "07 WIN",
    ind_funnel_scorecard(8, QUOTE_DT, WIN_IND) AS "08 WIN",
    ind_funnel_scorecard(9, QUOTE_DT, WIN_IND) AS "09 WIN",
    ind_funnel_scorecard(10, QUOTE_DT, WIN_IND) AS "10 WIN",
    ind_funnel_scorecard(11, QUOTE_DT, WIN_IND) AS "11 WIN",
    ind_funnel_scorecard(12, QUOTE_DT, WIN_IND) AS "12 WIN",
    ind_funnel_scorecard(13, QUOTE_DT, WIN_IND) AS "13 WIN",
    ind_funnel_scorecard(14, QUOTE_DT, WIN_IND) AS "14 WIN",
    ind_funnel_scorecard(15, QUOTE_DT, WIN_IND) AS "15 WIN",
    ind_funnel_scorecard(16, QUOTE_DT, WIN_IND) AS "16 WIN",
    ind_funnel_scorecard(17, QUOTE_DT, WIN_IND) AS "17 WIN",
    ind_funnel_scorecard(18, QUOTE_DT, WIN_IND) AS "18 WIN",
    ind_funnel_scorecard(19, QUOTE_DT, WIN_IND) AS "19 WIN",
    -- Bridged
    ind_funnel_scorecard(1, QUOTE_DT, BRIDGED_IND) AS "01 BRIDGED",
    ind_funnel_scorecard(2, QUOTE_DT, BRIDGED_IND) AS "02 BRIDGED",
    ind_funnel_scorecard(3, QUOTE_DT, BRIDGED_IND) AS "03 BRIDGED",
    ind_funnel_scorecard(4, QUOTE_DT, BRIDGED_IND) AS "04 BRIDGED",
    ind_funnel_scorecard(5, QUOTE_DT, BRIDGED_IND) AS "05 BRIDGED",
    ind_funnel_scorecard(6, QUOTE_DT, BRIDGED_IND) AS "06 BRIDGED",
    ind_funnel_scorecard(7, QUOTE_DT, BRIDGED_IND) AS "07 BRIDGED",
    ind_funnel_scorecard(8, QUOTE_DT, BRIDGED_IND) AS "08 BRIDGED",
    ind_funnel_scorecard(9, QUOTE_DT, BRIDGED_IND) AS "09 BRIDGED",
    ind_funnel_scorecard(10, QUOTE_DT, BRIDGED_IND) AS "10 BRIDGED",
    ind_funnel_scorecard(11, QUOTE_DT, BRIDGED_IND) AS "11 BRIDGED",
    ind_funnel_scorecard(12, QUOTE_DT, BRIDGED_IND) AS "12 BRIDGED",
    ind_funnel_scorecard(13, QUOTE_DT, BRIDGED_IND) AS "13 BRIDGED",
    ind_funnel_scorecard(14, QUOTE_DT, BRIDGED_IND) AS "14 BRIDGED",
    ind_funnel_scorecard(15, QUOTE_DT, BRIDGED_IND) AS "15 BRIDGED",
    ind_funnel_scorecard(16, QUOTE_DT, BRIDGED_IND) AS "16 BRIDGED",
    ind_funnel_scorecard(17, QUOTE_DT, BRIDGED_IND) AS "17 BRIDGED",
    ind_funnel_scorecard(18, QUOTE_DT, BRIDGED_IND) AS "18 BRIDGED",
    ind_funnel_scorecard(19, QUOTE_DT, BRIDGED_IND) AS "19 BRIDGED",
    -- Completed Quote
    ind_funnel_scorecard(1, QUOTE_DT, COMPL_QUOTE_IND) AS "01 COMPL_QUOTE",
    ind_funnel_scorecard(2, QUOTE_DT, COMPL_QUOTE_IND) AS "02 COMPL_QUOTE",
    ind_funnel_scorecard(3, QUOTE_DT, COMPL_QUOTE_IND) AS "03 COMPL_QUOTE",
    ind_funnel_scorecard(4, QUOTE_DT, COMPL_QUOTE_IND) AS "04 COMPL_QUOTE",
    ind_funnel_scorecard(5, QUOTE_DT, COMPL_QUOTE_IND) AS "05 COMPL_QUOTE",
    ind_funnel_scorecard(6, QUOTE_DT, COMPL_QUOTE_IND) AS "06 COMPL_QUOTE",
    ind_funnel_scorecard(7, QUOTE_DT, COMPL_QUOTE_IND) AS "07 COMPL_QUOTE",
    ind_funnel_scorecard(8, QUOTE_DT, COMPL_QUOTE_IND) AS "08 COMPL_QUOTE",
    ind_funnel_scorecard(9, QUOTE_DT, COMPL_QUOTE_IND) AS "09 COMPL_QUOTE",
    ind_funnel_scorecard(10, QUOTE_DT, COMPL_QUOTE_IND) AS "10 COMPL_QUOTE",
    ind_funnel_scorecard(11, QUOTE_DT, COMPL_QUOTE_IND) AS "11 COMPL_QUOTE",
    ind_funnel_scorecard(12, QUOTE_DT, COMPL_QUOTE_IND) AS "12 COMPL_QUOTE",
    ind_funnel_scorecard(13, QUOTE_DT, COMPL_QUOTE_IND) AS "13 COMPL_QUOTE",
    ind_funnel_scorecard(14, QUOTE_DT, COMPL_QUOTE_IND) AS "14 COMPL_QUOTE",
    ind_funnel_scorecard(15, QUOTE_DT, COMPL_QUOTE_IND) AS "15 COMPL_QUOTE",
    ind_funnel_scorecard(16, QUOTE_DT, COMPL_QUOTE_IND) AS "16 COMPL_QUOTE",
    ind_funnel_scorecard(17, QUOTE_DT, COMPL_QUOTE_IND) AS "17 COMPL_QUOTE",
    ind_funnel_scorecard(18, QUOTE_DT, COMPL_QUOTE_IND) AS "18 COMPL_QUOTE",
    ind_funnel_scorecard(19, QUOTE_DT, COMPL_QUOTE_IND) AS "19 COMPL_QUOTE",
    -- Issue
    ind_funnel_scorecard(1, QUOTE_DT, ISSUE_IND) AS "01 ISSUE",
    ind_funnel_scorecard(2, QUOTE_DT, ISSUE_IND) AS "02 ISSUE",
    ind_funnel_scorecard(3, QUOTE_DT, ISSUE_IND) AS "03 ISSUE",
    ind_funnel_scorecard(4, QUOTE_DT, ISSUE_IND) AS "04 ISSUE",
    ind_funnel_scorecard(5, QUOTE_DT, ISSUE_IND) AS "05 ISSUE",
    ind_funnel_scorecard(6, QUOTE_DT, ISSUE_IND) AS "06 ISSUE",
    ind_funnel_scorecard(7, QUOTE_DT, ISSUE_IND) AS "07 ISSUE",
    ind_funnel_scorecard(8, QUOTE_DT, ISSUE_IND) AS "08 ISSUE",
    ind_funnel_scorecard(9, QUOTE_DT, ISSUE_IND) AS "09 ISSUE",
    ind_funnel_scorecard(10, QUOTE_DT, ISSUE_IND) AS "10 ISSUE",
    ind_funnel_scorecard(11, QUOTE_DT, ISSUE_IND) AS "11 ISSUE",
    ind_funnel_scorecard(12, QUOTE_DT, ISSUE_IND) AS "12 ISSUE",
    ind_funnel_scorecard(13, QUOTE_DT, ISSUE_IND) AS "13 ISSUE",
    ind_funnel_scorecard(14, QUOTE_DT, ISSUE_IND) AS "14 ISSUE",
    ind_funnel_scorecard(15, QUOTE_DT, ISSUE_IND) AS "15 ISSUE",
    ind_funnel_scorecard(16, QUOTE_DT, ISSUE_IND) AS "16 ISSUE",
    ind_funnel_scorecard(17, QUOTE_DT, ISSUE_IND) AS "17 ISSUE",
    ind_funnel_scorecard(18, QUOTE_DT, ISSUE_IND) AS "18 ISSUE",
    ind_funnel_scorecard(19, QUOTE_DT, ISSUE_IND) AS "19 ISSUE",
    -- HIGAVG
    AVG(div_ind_funnel_scorecard(1, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "01 HIGAVG",
    AVG(div_ind_funnel_scorecard(2, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "02 HIGAVG",
    AVG(div_ind_funnel_scorecard(3, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "03 HIGAVG",
    AVG(div_ind_funnel_scorecard(4, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "04 HIGAVG",
    AVG(div_ind_funnel_scorecard(5, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "05 HIGAVG",
    AVG(div_ind_funnel_scorecard(6, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "06 HIGAVG",
    AVG(div_ind_funnel_scorecard(7, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "07 HIGAVG",
    AVG(div_ind_funnel_scorecard(8, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "08 HIGAVG",
    AVG(div_ind_funnel_scorecard(9, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "09 HIGAVG",
    AVG(div_ind_funnel_scorecard(10, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "10 HIGAVG",
    AVG(div_ind_funnel_scorecard(11, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "11 HIGAVG",
    AVG(div_ind_funnel_scorecard(12, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "12 HIGAVG",
    AVG(div_ind_funnel_scorecard(13, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "13 HIGAVG",
    AVG(div_ind_funnel_scorecard(14, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "14 HIGAVG",
    AVG(div_ind_funnel_scorecard(15, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "15 HIGAVG",
    AVG(div_ind_funnel_scorecard(16, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "16 HIGAVG",
    AVG(div_ind_funnel_scorecard(17, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "17 HIGAVG",
    AVG(div_ind_funnel_scorecard(18, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "18 HIGAVG",
    AVG(div_ind_funnel_scorecard(19, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED, AVG_PREMIUM_XHIG, ANNUALIZEDPREMIUM)) AS "19 HIGAVG",
    TO_CHAR(SYSDATE(),'MM-DD-YYYY') AS AS_OF
    FROM "DSC_PLBI_DB"."APP_AGCY_PRD"."AGCY_COMP_RATER_FUNNEL_VW"
    GROUP BY KEY
"""

In [17]:
new_funnel = f"""
CREATE OR REPLACE TRANSIENT TABLE DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_NEW_FUNNEL AS

WITH state_lob AS
(
    SELECT CONCAT(RISK_ST_ABBR, LOB) as Key,
    {query_6_body}
),

cw_lob AS
(
    SELECT CONCAT('CW', LOB) as Key,
    {query_6_body}
),

region_lob AS
(
    SELECT CONCAT(GET_REGION_SCORECARD(RISK_ST_ABBR), LOB) as Key,
    {query_6_body}
)

SELECT * FROM
(
    SELECT * FROM state_lob
    UNION ALL
    SELECT * FROM cw_lob
    UNION ALL
    SELECT * FROM region_lob
)
ORDER BY KEY
"""

In [18]:
cs = ctx.cursor()
try:
    cs.execute(new_funnel)
finally:
    cs.close()
print('Done')

Done


## 7. Funnel Detail Table Creation (DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_FUNNEL_DETAIL)

In [19]:
query_7_body = """
    -- RC1
    rc1_funnel_scorecard(14, QUOTE_DT, QCN) AS "14 RC1",
    rc1_funnel_scorecard(13, QUOTE_DT, QCN) AS "13 RC1",
    rc1_funnel_scorecard(12, QUOTE_DT, QCN) AS "12 RC1",
    rc1_funnel_scorecard(11, QUOTE_DT, QCN) AS "11 RC1",
    rc1_funnel_scorecard(10, QUOTE_DT, QCN) AS "10 RC1",
    rc1_funnel_scorecard(9, QUOTE_DT, QCN) AS " 9 RC1",
    rc1_funnel_scorecard(8, QUOTE_DT, QCN) AS " 8 RC1",
    rc1_funnel_scorecard(7, QUOTE_DT, QCN) AS " 7 RC1",
    rc1_funnel_scorecard(6, QUOTE_DT, QCN) AS " 6 RC1",
    rc1_funnel_scorecard(5, QUOTE_DT, QCN) AS " 5 RC1",
    rc1_funnel_scorecard(4, QUOTE_DT, QCN) AS " 4 RC1",
    rc1_funnel_scorecard(3, QUOTE_DT, QCN) AS " 3 RC1",
    rc1_funnel_scorecard(2, QUOTE_DT, QCN) AS " 2 RC1",
    -- FAILS
    ind_funnel_scorecard(14, QUOTE_DT, FAILURE_IND) AS "14 FAIL",
    ind_funnel_scorecard(13, QUOTE_DT, FAILURE_IND) AS "13 FAIL",
    ind_funnel_scorecard(12, QUOTE_DT, FAILURE_IND) AS "12 FAIL",
    ind_funnel_scorecard(11, QUOTE_DT, FAILURE_IND) AS "11 FAIL",
    ind_funnel_scorecard(10, QUOTE_DT, FAILURE_IND) AS "10 FAIL",
    ind_funnel_scorecard(9, QUOTE_DT, FAILURE_IND) AS "9 FAIL",
    ind_funnel_scorecard(8, QUOTE_DT, FAILURE_IND) AS "8 FAIL",
    ind_funnel_scorecard(7, QUOTE_DT, FAILURE_IND) AS "7 FAIL",
    ind_funnel_scorecard(6, QUOTE_DT, FAILURE_IND) AS "6 FAIL",
    ind_funnel_scorecard(5, QUOTE_DT, FAILURE_IND) AS "5 FAIL",
    ind_funnel_scorecard(4, QUOTE_DT, FAILURE_IND) AS "4 FAIL",
    ind_funnel_scorecard(3, QUOTE_DT, FAILURE_IND) AS "3 FAIL",
    ind_funnel_scorecard(2, QUOTE_DT, FAILURE_IND) AS "2 FAIL",
    -- SUCCESSES
    ind_funnel_scorecard(14, QUOTE_DT, SUCCESS_IND) AS "14 SUCCESS",
    ind_funnel_scorecard(13, QUOTE_DT, SUCCESS_IND) AS "13 SUCCESS",
    ind_funnel_scorecard(12, QUOTE_DT, SUCCESS_IND) AS "12 SUCCESS",
    ind_funnel_scorecard(11, QUOTE_DT, SUCCESS_IND) AS "11 SUCCESS",
    ind_funnel_scorecard(10, QUOTE_DT, SUCCESS_IND) AS "10 SUCCESS",
    ind_funnel_scorecard(9, QUOTE_DT, SUCCESS_IND) AS "9 SUCCESS",
    ind_funnel_scorecard(8, QUOTE_DT, SUCCESS_IND) AS "8 SUCCESS",
    ind_funnel_scorecard(7, QUOTE_DT, SUCCESS_IND) AS "7 SUCCESS",
    ind_funnel_scorecard(6, QUOTE_DT, SUCCESS_IND) AS "6 SUCCESS",
    ind_funnel_scorecard(5, QUOTE_DT, SUCCESS_IND) AS "5 SUCCESS",
    ind_funnel_scorecard(4, QUOTE_DT, SUCCESS_IND) AS "4 SUCCESS",
    ind_funnel_scorecard(3, QUOTE_DT, SUCCESS_IND) AS "3 SUCCESS",
    ind_funnel_scorecard(2, QUOTE_DT, SUCCESS_IND) AS "2 SUCCESS",
    -- SUCCESSES 2+
    adv_ind_funnel_scorecard(14, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED) AS "14 SUCCESS_2+",
    adv_ind_funnel_scorecard(13, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED) AS "13 SUCCESS_2+",
    adv_ind_funnel_scorecard(12, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED) AS "12 SUCCESS_2+",
    adv_ind_funnel_scorecard(11, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED) AS "11 SUCCESS_2+",
    adv_ind_funnel_scorecard(10, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED) AS "10 SUCCESS_2+",
    adv_ind_funnel_scorecard(9, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED) AS "9 SUCCESS_2+",
    adv_ind_funnel_scorecard(8, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED) AS "8 SUCCESS_2+",
    adv_ind_funnel_scorecard(7, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED) AS "7 SUCCESS_2+",
    adv_ind_funnel_scorecard(6, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED) AS "6 SUCCESS_2+",
    adv_ind_funnel_scorecard(5, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED) AS "5 SUCCESS_2+",
    adv_ind_funnel_scorecard(4, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED) AS "4 SUCCESS_2+",
    adv_ind_funnel_scorecard(3, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED) AS "3 SUCCESS_2+",
    adv_ind_funnel_scorecard(2, QUOTE_DT, SUCCESS_IND, CARRIERSQUOTED) AS "2 SUCCESS_2+",
    -- COMPETITIVE
	ind_funnel_scorecard(14, QUOTE_DT, COMPETITIVE_IND) AS "14 COMPETITIVE",
    ind_funnel_scorecard(13, QUOTE_DT, COMPETITIVE_IND) AS "13 COMPETITIVE",
    ind_funnel_scorecard(12, QUOTE_DT, COMPETITIVE_IND) AS "12 COMPETITIVE",
    ind_funnel_scorecard(11, QUOTE_DT, COMPETITIVE_IND) AS "11 COMPETITIVE",
    ind_funnel_scorecard(10, QUOTE_DT, COMPETITIVE_IND) AS "10 COMPETITIVE",
    ind_funnel_scorecard(9, QUOTE_DT, COMPETITIVE_IND) AS "9 COMPETITIVE",
    ind_funnel_scorecard(8, QUOTE_DT, COMPETITIVE_IND) AS "8 COMPETITIVE",
    ind_funnel_scorecard(7, QUOTE_DT, COMPETITIVE_IND) AS "7 COMPETITIVE",
    ind_funnel_scorecard(6, QUOTE_DT, COMPETITIVE_IND) AS "6 COMPETITIVE",
    ind_funnel_scorecard(5, QUOTE_DT, COMPETITIVE_IND) AS "5 COMPETITIVE",
    ind_funnel_scorecard(4, QUOTE_DT, COMPETITIVE_IND) AS "4 COMPETITIVE",
    ind_funnel_scorecard(3, QUOTE_DT, COMPETITIVE_IND) AS "3 COMPETITIVE",
    ind_funnel_scorecard(2, QUOTE_DT, COMPETITIVE_IND) AS "2 COMPETITIVE",
	-- WIN
	ind_funnel_scorecard(14, QUOTE_DT, WIN_IND) AS "14 WIN",
    ind_funnel_scorecard(13, QUOTE_DT, WIN_IND) AS "13 WIN",
    ind_funnel_scorecard(12, QUOTE_DT, WIN_IND) AS "12 WIN",
    ind_funnel_scorecard(11, QUOTE_DT, WIN_IND) AS "11 WIN",
    ind_funnel_scorecard(10, QUOTE_DT, WIN_IND) AS "10 WIN",
    ind_funnel_scorecard(9, QUOTE_DT, WIN_IND) AS "9 WIN",
    ind_funnel_scorecard(8, QUOTE_DT, WIN_IND) AS "8 WIN",
    ind_funnel_scorecard(7, QUOTE_DT, WIN_IND) AS "7 WIN",
    ind_funnel_scorecard(6, QUOTE_DT, WIN_IND) AS "6 WIN",
    ind_funnel_scorecard(5, QUOTE_DT, WIN_IND) AS "5 WIN",
    ind_funnel_scorecard(4, QUOTE_DT, WIN_IND) AS "4 WIN",
    ind_funnel_scorecard(3, QUOTE_DT, WIN_IND) AS "3 WIN",
    ind_funnel_scorecard(2, QUOTE_DT, WIN_IND) AS "2 WIN",
	-- CQ
	ind_funnel_scorecard(14, QUOTE_DT, COMPL_QTE_IND_CNT) AS "14 CQ",
    ind_funnel_scorecard(13, QUOTE_DT, COMPL_QTE_IND_CNT) AS "13 CQ",
    ind_funnel_scorecard(12, QUOTE_DT, COMPL_QTE_IND_CNT) AS "12 CQ",
    ind_funnel_scorecard(11, QUOTE_DT, COMPL_QTE_IND_CNT) AS "11 CQ",
    ind_funnel_scorecard(10, QUOTE_DT, COMPL_QTE_IND_CNT) AS "10 CQ",
    ind_funnel_scorecard(9, QUOTE_DT, COMPL_QTE_IND_CNT) AS "9 CQ",
    ind_funnel_scorecard(8, QUOTE_DT, COMPL_QTE_IND_CNT) AS "8 CQ",
    ind_funnel_scorecard(7, QUOTE_DT, COMPL_QTE_IND_CNT) AS "7 CQ",
    ind_funnel_scorecard(6, QUOTE_DT, COMPL_QTE_IND_CNT) AS "6 CQ",
    ind_funnel_scorecard(5, QUOTE_DT, COMPL_QTE_IND_CNT) AS "5 CQ",
    ind_funnel_scorecard(4, QUOTE_DT, COMPL_QTE_IND_CNT) AS "4 CQ",
    ind_funnel_scorecard(3, QUOTE_DT, COMPL_QTE_IND_CNT) AS "3 CQ",
    ind_funnel_scorecard(2, QUOTE_DT, COMPL_QTE_IND_CNT) AS "2 CQ",
    -- Issue
    ind_funnel_scorecard(14, QUOTE_DT, ISS_IND_CNT) AS "14 ISSUE",
    ind_funnel_scorecard(13, QUOTE_DT, ISS_IND_CNT) AS "13 ISSUE",
    ind_funnel_scorecard(12, QUOTE_DT, ISS_IND_CNT) AS "12 ISSUE",
    ind_funnel_scorecard(11, QUOTE_DT, ISS_IND_CNT) AS "11 ISSUE",
    ind_funnel_scorecard(10, QUOTE_DT, ISS_IND_CNT) AS "10 ISSUE",
    ind_funnel_scorecard(9, QUOTE_DT, ISS_IND_CNT) AS "9 ISSUE",
    ind_funnel_scorecard(8, QUOTE_DT, ISS_IND_CNT) AS "8 ISSUE",
    ind_funnel_scorecard(7, QUOTE_DT, ISS_IND_CNT) AS "7 ISSUE",
    ind_funnel_scorecard(6, QUOTE_DT, ISS_IND_CNT) AS "6 ISSUE",
    ind_funnel_scorecard(5, QUOTE_DT, ISS_IND_CNT) AS "5 ISSUE",
    ind_funnel_scorecard(4, QUOTE_DT, ISS_IND_CNT) AS "4 ISSUE",
    ind_funnel_scorecard(3, QUOTE_DT, ISS_IND_CNT) AS "3 ISSUE",
    ind_funnel_scorecard(2, QUOTE_DT, ISS_IND_CNT) AS "2 ISSUE",
    -- Bridge
    ind_funnel_scorecard(14, QUOTE_DT, BRIDGED_IND_CNT) AS "14 BRIDGE",
    ind_funnel_scorecard(13, QUOTE_DT, BRIDGED_IND_CNT) AS "13 BRIDGE",
    ind_funnel_scorecard(12, QUOTE_DT, BRIDGED_IND_CNT) AS "12 BRIDGE",
    ind_funnel_scorecard(11, QUOTE_DT, BRIDGED_IND_CNT) AS "11 BRIDGE",
    ind_funnel_scorecard(10, QUOTE_DT, BRIDGED_IND_CNT) AS "10 BRIDGE",
    ind_funnel_scorecard(9, QUOTE_DT, BRIDGED_IND_CNT) AS "9 BRIDGE",
    ind_funnel_scorecard(8, QUOTE_DT, BRIDGED_IND_CNT) AS "8 BRIDGE",
    ind_funnel_scorecard(7, QUOTE_DT, BRIDGED_IND_CNT) AS "7 BRIDGE",
    ind_funnel_scorecard(6, QUOTE_DT, BRIDGED_IND_CNT) AS "6 BRIDGE",
    ind_funnel_scorecard(5, QUOTE_DT, BRIDGED_IND_CNT) AS "5 BRIDGE",
    ind_funnel_scorecard(4, QUOTE_DT, BRIDGED_IND_CNT) AS "4 BRIDGE",
    ind_funnel_scorecard(3, QUOTE_DT, BRIDGED_IND_CNT) AS "3 BRIDGE",
    ind_funnel_scorecard(2, QUOTE_DT, BRIDGED_IND_CNT) AS "2 BRIDGE",
    -- AVG
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE())= 14 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN AVG_PREMIUM_XHIG ELSE 0 END) AS "14 AVG", 
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE())= 13 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN AVG_PREMIUM_XHIG ELSE 0 END) AS "13 AVG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE())= 12 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN AVG_PREMIUM_XHIG ELSE 0 END) AS "12 AVG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE())= 11 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN AVG_PREMIUM_XHIG ELSE 0 END) AS "11 AVG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE())= 10 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN AVG_PREMIUM_XHIG ELSE 0 END) AS "10 AVG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE())= 9 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN AVG_PREMIUM_XHIG ELSE 0 END) AS "9 AVG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE())= 8 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN AVG_PREMIUM_XHIG ELSE 0 END) AS "8 AVG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE())= 7 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN AVG_PREMIUM_XHIG ELSE 0 END) AS "7 AVG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE())= 6 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN AVG_PREMIUM_XHIG ELSE 0 END) AS "6 AVG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE())= 5 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN AVG_PREMIUM_XHIG ELSE 0 END) AS "5 AVG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE())= 4 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN AVG_PREMIUM_XHIG ELSE 0 END) AS "4 AVG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE())= 3 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN AVG_PREMIUM_XHIG ELSE 0 END) AS "3 AVG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE())= 2 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN AVG_PREMIUM_XHIG ELSE 0 END) AS "2 AVG",
    -- HIG
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE()) = 14 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN ANNUALIZEDPREMIUM ELSE 0 END) AS "14 HIG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE()) = 13 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN ANNUALIZEDPREMIUM ELSE 0 END) AS "13 HIG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE()) = 12 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN ANNUALIZEDPREMIUM ELSE 0 END) AS "12 HIG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE()) = 11 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN ANNUALIZEDPREMIUM ELSE 0 END) AS "11 HIG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE()) = 10 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN ANNUALIZEDPREMIUM ELSE 0 END) AS "10 HIG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE()) = 9 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN ANNUALIZEDPREMIUM ELSE 0 END) AS "9 HIG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE()) = 8 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN ANNUALIZEDPREMIUM ELSE 0 END) AS "8 HIG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE()) = 7 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN ANNUALIZEDPREMIUM ELSE 0 END) AS "7 HIG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE()) = 6 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN ANNUALIZEDPREMIUM ELSE 0 END) AS "6 HIG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE()) = 5 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN ANNUALIZEDPREMIUM ELSE 0 END) AS "5 HIG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE()) = 4 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN ANNUALIZEDPREMIUM ELSE 0 END) AS "4 HIG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE()) = 3 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN ANNUALIZEDPREMIUM ELSE 0 END) AS "3 HIG",
    SUM(CASE WHEN DATEDIFF(MONTH,QUOTE_DT,SYSDATE()) = 2 AND SUCCESS_IND = '1' AND CARRIERSQUOTED > 2  THEN ANNUALIZEDPREMIUM ELSE 0 END) AS "2 HIG",
    TO_CHAR(SYSDATE(),'MM-DD-YYYY') as AS_OF
    FROM "DSC_PLBI_DB"."APP_AGCY_PRD"."AGCY_COMP_RATER_AUTO_QTE_VW"
    GROUP BY KEY
"""

In [20]:
funnel_detail = f"""
CREATE OR REPLACE TRANSIENT TABLE DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_FUNNEL_DETAIL AS

WITH market_index AS -- Segmenting funnel data by market index
(
    SELECT CONCAT(STATE_ABBR, 'Market Index', (CASE WHEN MKT_INDX_CTGY_CD IS NULL THEN 'unknown' else MKT_INDX_CTGY_CD END)) as Key,
    {query_7_body}
),

aarp_member AS -- Segmenting funnel data by AARP member status
(
	SELECT CONCAT(STATE_ABBR, 'AARP Member Status', AARP_MEMB_STTS_DESCR) as Key,
    {query_7_body}
),

aarp_thru AS -- Segmenting funnel data by AARP thru agent indicator
(
	SELECT CONCAT(STATE_ABBR, 'AARP Thru Agent', AARP_THRU_AGNT_IND) as Key,
    {query_7_body}
),

pcarr AS -- Segmenting funnel data by prior carrier
(
	SELECT CONCAT(STATE_ABBR, 'Prior Carrier',
           CASE WHEN PCARR_DESC in ('OTHER - STANDARD','NOT PRESENTLY INSURED','PROGRESSIVE','HARTFORD',
			   'OTHER - NON STANDARD','SAFECO','STATE FARM','TRAVELERS','ALLSTATE','AMERICAN NATIONAL','GEICO','NATIONWIDE') THEN PCARR_DESC 
			    ELSE 'OTHER - ALL OTHER' END) as Key,
    {query_7_body}
),

pcarr_yr AS -- Segmenting funnel data by prior carrier year count
(
	SELECT CONCAT(STATE_ABBR, 'Prior Carrier Year Count', PCARR_YR_CNT) as Key,
    {query_7_body}
),

full_pay AS --Segmenting funnel data by Full Pay Discount
(
	SELECT CONCAT(STATE_ABBR, 'Full Pay Discount', FULL_PAY_DISCOUNT_IND) as Key,
    {query_7_body}
),

aqd_group AS -- Segmenting funnel data by AQD group
(
	SELECT CONCAT(STATE_ABBR, 'AQD Group', AQD_GRP) as Key,
    {query_7_body}
),

acct_credit AS -- Segmenting funnel data by account credit indicator
(
	SELECT CONCAT(STATE_ABBR, 'Account Credit', ACCT_CR_IND) as Key,
    {query_7_body}
),

ins_age AS -- Segmenting funnel data by insured age group
(
	SELECT CONCAT(STATE_ABBR, 'Insured Age Group', INSURED_AGE_GRP) as Key,
    {query_7_body}
),

ins_score AS -- Segmenting funnel data by insurance score
(
	SELECT CONCAT(STATE_ABBR,'Insurance Score',
           CASE WHEN INS_SCORE_CD IN ('000','999','NUS','XXX','ERR') THEN 'NHNS' 
				WHEN INS_SCORE_CD IS NULL THEN 'NHNS'
                WHEN INS_SCORE_CD = '800' THEN '800'
				WHEN INS_SCORE_CD BETWEEN 1 AND 614 THEN '1 to 614'
				WHEN INS_SCORE_CD BETWEEN 615 AND 699 THEN '615 to 699'
				WHEN INS_SCORE_CD BETWEEN 700 AND 754 THEN '700 to 754'
				WHEN INS_SCORE_CD BETWEEN 755 AND 789 THEN '755 to 789'
				WHEN INS_SCORE_CD BETWEEN 790 AND 809 THEN '790 to 809'
				WHEN INS_SCORE_CD BETWEEN 810 AND 824 THEN '810 to 824'
				WHEN INS_SCORE_CD BETWEEN 825 AND 840 THEN '825 to 840'
				WHEN INS_SCORE_CD BETWEEN 841 AND 865 THEN '841 to 865'
				WHEN INS_SCORE_CD BETWEEN 866 AND 893 THEN '866 to 893'
				WHEN INS_SCORE_CD BETWEEN 894 AND 979 THEN '894+' else INS_SCORE_CD END) as Key,
    {query_7_body}
),

veh_cnt AS -- Segmenting funnel data by vehicle count
(
	SELECT CONCAT(STATE_ABBR, 'Veh Count', NUM_VEH) as Key,
    {query_7_body}
),

avg_mileage AS -- Segmenting funnel data by average mileage
(
	SELECT CONCAT(STATE_ABBR,'AVG Annual Mileage',
           CASE WHEN AVERAGE_ANNUAL_MILAGE < 7000 THEN 'Less than 7000' 
                WHEN AVERAGE_ANNUAL_MILAGE BETWEEN 7000 AND 8999 THEN '7000-8999' 
                WHEN AVERAGE_ANNUAL_MILAGE = 9000 THEN '9000'
                WHEN AVERAGE_ANNUAL_MILAGE BETWEEN 9001 AND 12499 THEN '9001-12499' 
                WHEN AVERAGE_ANNUAL_MILAGE >= 12500 THEN '12500+' else 'unknown' END) as Key,
    {query_7_body}
),

youthful AS -- Segmenting funnel data by youthful indicator
(
	SELECT CONCAT(STATE_ABBR, 'Youthful', YOUTHFUL_IND) as Key,
    {query_7_body}
),

household_comp AS -- Segmenting funnel data by household comp
(
	SELECT CONCAT(STATE_ABBR, 'Household Comp', HH_COMP) as Key,
    {query_7_body}
),

cov_pkg AS -- Segmenting funnel data by coverage package
(
	SELECT CONCAT(STATE_ABBR,'Coverage Package Code', POL_COV_PKG_CD) as Key,
    {query_7_body}
),

bi_limit AS -- Segmenting funnel data by BI limit
(
	SELECT CONCAT(STATE_ABBR,'Prior BI Limit',
           CASE WHEN PRIOR_CARR_BI_PERSON_LMT IN ('0010','0015','0020','0025','0030') THEN '10 to 30'
                WHEN PRIOR_CARR_BI_PERSON_LMT IN ('0050','0055','0060','0065','0070','0075') THEN '50 to 75'
                WHEN PRIOR_CARR_BI_PERSON_LMT IN ('0100','0150') THEN '100 to 150'
                WHEN PRIOR_CARR_BI_PERSON_LMT IN ('0200','0250','0300','0500') THEN '200 to 500' else 'unknown' END) as Key,
    {query_7_body}
),

garage_sub AS -- Segmenting funnel data by garage_sub
(
	SELECT CONCAT(STATE_ABBR, 'Garage Substate',
    CASE WHEN LEFT(STATE_ABBR,2) = LEFT(GARAGE_SUBSTATE,2) THEN GARAGE_SUBSTATE ELSE 'Out of State Mail' END) as Key,
    {query_7_body}
)

SELECT * FROM
(
	SELECT * FROM market_index
	UNION ALL
	SELECT * FROM aarp_member
	UNION ALL
	SELECT * FROM aarp_thru
	UNION ALL
	SELECT * FROM pcarr
	UNION ALL
	SELECT * FROM pcarr_yr
	UNION ALL
	SELECT * FROM full_pay
	UNION ALL
	SELECT * FROM aqd_group
	UNION ALL
	SELECT * FROM acct_credit
	UNION ALL
	SELECT * FROM ins_age
	UNION ALL
	SELECT * FROM ins_score
	UNION ALL
	SELECT * FROM veh_cnt
	UNION ALL
	SELECT * FROM avg_mileage
	UNION ALL
	SELECT * FROM youthful
	UNION ALL
	SELECT * FROM household_comp
	UNION ALL
	SELECT * FROM cov_pkg
	UNION ALL
	SELECT * FROM bi_limit
	UNION ALL
	SELECT * FROM garage_sub
)
ORDER BY KEY;
"""

In [21]:
cs = ctx.cursor()
try:
    cs.execute(funnel_detail)
finally:
    cs.close()
print('Done')

Done


## 8. Inforce Segmentation Table Creation (DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_INFORCE_SEGMENTATION)

In [22]:
inforce_segmentation = """
CREATE OR REPLACE TRANSIENT TABLE DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_INFORCE_SEGMENTATION AS

WITH ins_score AS -- Gets latest month's ins score decile PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'Ins Score Decile',(
    CASE WHEN NEWCO_INS_SCR_CD IN (000,0,999) THEN 'NHNS'
         WHEN NEWCO_INS_SCR_CD <= 386 THEN 'Decile 10 Worst'
         WHEN NEWCO_INS_SCR_CD <= 460 THEN 'Decile 09'
         WHEN NEWCO_INS_SCR_CD <= 539 THEN 'Decile 08'
         WHEN NEWCO_INS_SCR_CD <= 617 THEN 'Decile 07'
         WHEN NEWCO_INS_SCR_CD <= 679 THEN 'Decile 06'
         WHEN NEWCO_INS_SCR_CD <= 724 THEN 'Decile 05'
         WHEN NEWCO_INS_SCR_CD <= 756 THEN 'Decile 04'
         WHEN NEWCO_INS_SCR_CD <= 782 THEN 'Decile 03'
         WHEN NEWCO_INS_SCR_CD <= 809 THEN 'Decile 02'
         WHEN NEWCO_INS_SCR_CD <= 980 THEN 'Decile 01 Best' ELSE 'NHNS' END)) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1' 
    GROUP BY KEY
),

hh_comp AS -- Gets latest month's household comp PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'HH Comp',(
    CASE WHEN HH_COMP = 'SC1D' THEN 'a. single vehicle, 1 driver'
         WHEN HH_COMP = 'MC C=D' THEN 'd. multi vehicle, vehicle = driver'
         WHEN HH_COMP = 'SCMT1D' THEN 'b. single vehicle, more than 1 driver' 
         WHEN HH_COMP = 'MCMCTD' THEN 'c. multi vehicle, vehicle > drivers' 
         WHEN HH_COMP = 'MCMDTC' THEN 'e. multi vehicle, vehicle < drivers' 
         WHEN HH_COMP = '0 CARS' THEN 'f. 0 vehicle' ELSE 'g. other' END)) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1' 
    GROUP BY KEY
),

clean_dirty AS -- Gets latest month's Clean/Dirty PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'CLEAN DIRTY DESC', CLEAN_DIRTY_DESC) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1' 
    GROUP BY KEY
),

max_age AS -- Gets latest month's max age group PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'Max Age Group',(
    CASE WHEN POL_MAX_DRVR_AGE < 50 THEN '50'
         WHEN POL_MAX_DRVR_AGE <= 59 THEN '50-59' 
         WHEN POL_MAX_DRVR_AGE <= 69 THEN '50-69' 
         WHEN POL_MAX_DRVR_AGE <= 79 THEN '70-79' 
         WHEN POL_MAX_DRVR_AGE <= 130 THEN '80+' ELSE 'Unknown' END)) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1' 
    GROUP BY KEY
),

min_age AS -- Gets latest month's min age group PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'Min Age Group',(
    CASE WHEN POL_MIN_DRVR_AGE < 50 THEN '50'
         WHEN POL_MIN_DRVR_AGE <= 59 THEN '50-59' 
         WHEN POL_MIN_DRVR_AGE <= 69 THEN '50-69' 
         WHEN POL_MIN_DRVR_AGE <= 79 THEN '70-79' 
         WHEN POL_MIN_DRVR_AGE <= 130 THEN '80+' ELSE 'Unknown' END)) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1' 
    GROUP BY KEY
),

max_minus_min AS -- Gets latest month's (max-min) age group PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'Max Minus Min Age Group',(
    CASE WHEN (POL_MAX_DRVR_AGE - POL_MIN_DRVR_AGE) = 0 THEN 'a. 0'
         WHEN (POL_MAX_DRVR_AGE - POL_MIN_DRVR_AGE) BETWEEN 1 AND 2 THEN 'b. 1-2'
         WHEN (POL_MAX_DRVR_AGE - POL_MIN_DRVR_AGE) BETWEEN 3 AND 6 THEN 'c. 3-6'
         WHEN (POL_MAX_DRVR_AGE - POL_MIN_DRVR_AGE) BETWEEN 7 AND 20 THEN 'd. 7-20'
         WHEN (POL_MAX_DRVR_AGE - POL_MIN_DRVR_AGE) >= 21 THEN 'e. 21+' ELSE 'f. n/a' END)) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1' 
    GROUP BY KEY
),

pcarr_year_cnt AS -- Gets latest month's pcarr YR PIF count 
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'PCARR YR CNT',(
    CASE WHEN PCARR_YR_CNT = 0.5 THEN 'a 0.5'
         WHEN PCARR_YR_CNT = 1.0 THEN 'b 1' 
         WHEN PCARR_YR_CNT = 2.0 THEN 'c 2' 
         WHEN PCARR_YR_CNT = 3.0 THEN 'd 3' 
         WHEN PCARR_YR_CNT = 4.0 THEN 'e 4' 
         WHEN PCARR_YR_CNT = 5.0 THEN 'f 5+' ELSE 'g Unknown' END)) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1' 
    GROUP BY KEY
),

acct_credit AS -- Gets latest month's acct credit PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'POL ACCT CR IND', ACCT_CR_IND) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1' 
    GROUP BY KEY
),

coverage_pkg AS -- Gets latest month's coverage package PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'POL COV PKG CD',(
    CASE WHEN POL_COV_PKG_CD IS NULL OR POL_COV_PKG_CD = 'NO COV' THEN 'Unknown' 
         ELSE POL_COV_PKG_CD END)) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1' 
    GROUP BY KEY
),

pcarr AS -- Gets latest month's prior carrier PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'Prior Carrier',(
    CASE WHEN PCARR_DESC IN ('STATE FARM INSURANCE') THEN 'STATE FARM'
         WHEN PCARR_DESC IN ('IMPERIAL FIRE AND CA','SAFE AUTO INS CO','ALLSTATE CTY MUTUAL','ALLSTATE FIRE CAS IN','ALLSTATE INDEMNITY C','ALLSTATE INSURANCE C','ALLSTATE OF NEW JERS','ALLSTATE PROPERTY','ANJP C INS CO','CENTURY NATIONAL INS','CENTURY-NATIONAL INS','DIRECT GENERAL INS','DIRECT GENERAL INSUR','DIRECT INSURANCE CO','DIRECT INSURANCE COM','DIRECT NATIONAL INSU','ENCOM H A INS COMP','ENCOM INDEPENDENT IN','ENCOM INS CO OF NJ','ENCOM P C COMP','ENCOMPASS INDEMNITY','ENCOMPASS INS OF AM','ENCOMPASS INSURANCE','ENCOMPASS P C OF NJ','ESURANCE INS CO','ESURANCE INS CO NJ','ESURANCE PROP CAS','IMPERIAL FIRE CASU','INTEGON CASUALTY INS','INTEGON GENERAL INSU','INTEGON INDEMNITY CO','INTEGON NATIONAL','INTEGON NATIONAL INS','INTEGON PREFERRED IN','MIC GENERAL INSURANC','MS-DIRECT GENERAL IN','NATIONAL FARMERS UNI','NATIONAL GENERAL ASS','NATIONAL GENERAL INS','NATIONAL GENERAL PRE','NEW SOUTH INSURANCE','NORTHBROOK INSURANCE','STANDARD PROPERTY AN') THEN 'ALLSTATE'
         WHEN PCARR_DESC IN ('GEICO ADVANTAGE INS','GEICO CASUALTY COMP','GEICO CHOICE INS CO','GEICO COUNTY MTL','GEICO GENERAL INS CO','GEICO INDEMNITY COMP','GEICO SECURE INS CO','GEICOTXCOUNTYMUTUAL','GOVERNMENT EMP INSCO','SOUTHERN COUNTY') THEN 'GEICO'
         WHEN PCARR_DESC IN ('PROGRESSIVE GROUPS') THEN 'PROGRESSIVE'
         WHEN PCARR_DESC IN ('EMPLOYERS INSURANCE','FIRST NAT INS CO AME','GENERAL INS CO AMER','LIB MUTUAL PERSONAL','LIBERTY COUNTY MUTUA','LIBERTY INSURANCE CO','LIBERTY MUTUAL FIRE','LIBERTY MUTUAL INSUR','LIBERTY MUTUAL MID A','LIBERTY PERSONAL INS','LM GENERAL INSURANCE','LM INSURANCE CORPORA','MERIDIAN SECURITY','MERIDIAN SECURITY IN','MILBANK INSURANCE CO','MONTGOMERY MUTUAL IN','PATRONS MUTUAL INS C','PEERLESS INDEMNITY I','SAFECO INS CO AMER','SAFECO INSUR CO ILL','SAFECO INSUR CO IND','SAFECO INSUR CO ORE','SAFECO LLOYDS INS CO','SAFECO NAT INSUR CO','SAIC OF OHIO','SAIC OF WISCONSIN','STATE AUTO MUTUAL','STATE AUTO P C INS','STATE AUTO PROP C','STATE AUTOMOBILE MUT','THE FIRST LIBERTY IN','THE NETHERLANDS INSU','WAUSAU UNDERWRITERS','AMER ECONOMY INS CO','AMER STATES INSUR CO','AMER STATES PREF INS','AMERICA FIRST INSURA','CONSOLIDATED INSURAN') THEN 'LIBERTY'
         WHEN PCARR_DESC IN ('FID GUAR INS UNDRS','PHOENIX INSURANCE CO','ST PAUL PROTECTIVE I','STANDARD FIRE INS CO','TRAVCO INSURANCE CO','TRAVELERS CAS CONN','TRAVELERS COMMERCIAL','TRAVELERS HOME MAR','TRAVELERS INDEM AMER','TRAVELERS INDEM CONN','TRAVELERS INDEMNITY','TRAVELERS P C INS CO','TRAVELERS P C OF AM','TRAVELERS PERSON SEC','TRAVELERS PERSONAL','AUTO INS CO OF HARTF','CHARTER OAK FIRE INS','FIRST FLORIDIAN A H') THEN 'TRAVELERS'
         WHEN PCARR_DESC in ('AMFAM CONNECT INS', 'AMFAM CONNECT PC', 'AMFAM INS CO', 'AMER FAMILY MUTUAL', 'AMERICAN STD INS OH', 'THE GENERAL AUTOMOBI', 'MIDVALE INDEMNITY',     'AFMICSI', 'AMER STANDARD INS', 'AMERICAN FAM INS OH', 'AMERIPRISE INSURANCE', 'HOMESITE INSURANCE C', 'IDS PROPERTY CASUALT', 'MSAA', 'MSAA INSURANCE CO', 'MSAPIC', 'NGM INSURANCE CO', 'ODIC') THEN 'AMERICAN FAMILY'                                                                                                         
         WHEN PCARR_DESC in ('USAA GENERAL IND CO', 'USAA CASUALTY INS CO', 'USAA', 'GARRISON PROP CAS', 'GAR') THEN 'USAA'
         WHEN PCARR_DESC in ('COLONIAL COUNTY MUT', 'ALLIED PROP CAS', 'ALLIED P C INS CO', 'COLONIAL COUNTY MUT', 'CRESTBROOK INSURANCE', 'DEPOSITORS INS CO', 'NATIONWIDE AG INS CO', 'NATIONWIDE AGRI INS', 'NATIONWIDE GENERAL', 'NATIONWIDE INS CO', 'NATIONWIDE INS OF AM', 'NATIONWIDE MUT FIRE', 'NATIONWIDE MUT INS', 'NATIONWIDE MUTUAL IN', 'VICTORIA FIRE CASU','VICTORIA SPECIALTY I', 'AMCO INSURANCE CO', 'ALLIED P C INS CO', 'HARLEYSVILLEIC', 'HARLEYSVILLEPREFIC', 'NATIONWIDE AFFINITY', 'NATIONWIDE GENERAL', 'NATIONWIDE MUTUAL', 'NATIONWIDE PROP CAS', 'VICTORIA SELECT INSU') THEN 'Nationwide'
         WHEN PCARR_DESC IS NULL  THEN 'NULL'
         WHEN PCARR_DESC IN ('NOT PRESENTLY INSURED', 'UNKNOWN', 'NO PRIOR INSURANCE') THEN PCARR_DESC
         WHEN PCARR_DESC IN ('OTHER-NON STANDARD', 'OTHER - NON STANDARD') THEN 'OTHER - NON STANDARD'
         ELSE 'OTHER' END)) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1' 
    GROUP BY KEY
),

insured_age_seg AS -- Gets latest month's insured age group PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'Insured Age Group',(
    CASE WHEN INSURED_AGE <= 49 THEN 'a. 25-49'
         WHEN INSURED_AGE <= 54 THEN 'b. 50-54'
         WHEN INSURED_AGE <= 59 THEN 'c. 55-59'
         WHEN INSURED_AGE <= 64 THEN 'd. 60-64'
         WHEN INSURED_AGE <= 69 THEN 'e. 65-69'
         WHEN INSURED_AGE <= 74 THEN 'f. 70-74'
         WHEN INSURED_AGE <= 79 THEN 'g. 75-79'
         WHEN INSURED_AGE <= 120 THEN 'h. 80+' ELSE 'i. null' END)) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1' 
    GROUP BY KEY
),

ppv AS -- Gets latest month's PPV PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'POL PPV CNT',(
    CASE WHEN POL_PPV_CNT = 1 THEN 'b 1' 
         WHEN POL_PPV_CNT = 2 THEN 'c 2' 
         WHEN POL_PPV_CNT = 3 THEN 'd 3' 
         WHEN POL_PPV_CNT = 4 THEN 'e 4' 
         WHEN POL_PPV_CNT = 5 THEN 'f 5+' ELSE 'g Unknown' END)) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1'
    GROUP BY KEY
),

young_driver AS -- Gets latest month's young driver ind PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'YOUTH OPER DRVR IND', YOUTH_OPER_DRVR_IND) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1'
    GROUP BY KEY
),

garage AS -- Gets latest month's garage substate PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'GARAGE SUBSTATE',(
    CASE WHEN LEFT(STATE_ABBR,2) = LEFT(GARAGE_SUBSTATE,2) THEN GARAGE_SUBSTATE ELSE 'Out of State Mail' END)) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1' 
    GROUP BY KEY
),

bi_per_person AS -- Gets latest month's bi per person PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'Prior BI Per Person',(
    CASE WHEN PCARR_BI_PERS_LMT_CD < 50 THEN 'a. Less than 50'                                   
         WHEN PCARR_BI_PERS_LMT_CD < 100 THEN 'b. 50-99'                                      
         WHEN PCARR_BI_PERS_LMT_CD < 250 THEN 'c. 100-249'                                    
         WHEN PCARR_BI_PERS_LMT_CD < 3000 THEN 'd. 250+'                                        
         WHEN PCARR_BI_PERS_LMT_CD IS NULL THEN 'f. Unknown/No Insurance' ELSE 'e. other' END)) as Key,                                      
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1' 
    GROUP BY KEY
),

adv_qte_days AS -- Gets latest month's advanced quote days PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'Advanced Quote Discount',(
    CASE WHEN ADV_QTE_DAY_CNT in (000,0,999) THEN 'A. 0 Days'
         WHEN ADV_QTE_DAY_CNT <= 13 THEN 'B. 1-13'
         WHEN ADV_QTE_DAY_CNT <= 60 THEN 'C. 14-60'
         WHEN ADV_QTE_DAY_CNT <= 75 THEN 'D. 61-75'
         WHEN ADV_QTE_DAY_CNT <= 99999 THEN 'E. 76+' ELSE 'F. Unknown' END)) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1'
    GROUP BY KEY
),

min_rating_factor AS -- Gets latest month's min rating factor PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'Min Rating Factor', POL_MIN_RATE_FCTR_IND) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" LEFT JOIN 
    (SELECT AUTO_POL_TRANS_ID, POL_MIN_RATE_FCTR_IND FROM "PRD_PL_DB"."APP_DCPA_AUTO_CURATE"."AUTO_POLICY" GROUP BY AUTO_POL_TRANS_ID, POL_MIN_RATE_FCTR_IND)
    USING (AUTO_POL_TRANS_ID)
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1'
    GROUP BY KEY
),

renew_uw_tier AS -- Gets latest month's UW tier PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'RENEW UW TIER',(
    CASE WHEN RENEW_UW_TIER_CD in ('BB','BC','BD','BE','BF','BG','BH','BI','BJ','BK') THEN RENEW_UW_TIER_CD ELSE 'Other' END)) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1'
    GROUP BY KEY
),

max_veh_age AS -- Gets latest month's max vehicle age PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'Max Veh Age',(
    CASE WHEN POL_MAX_VEH_AGE <= 3 THEN '0-3'
         WHEN POL_MAX_VEH_AGE <= 6 THEN '4-6'
         WHEN POL_MAX_VEH_AGE <= 9 THEN '7-9'
         WHEN POL_MAX_VEH_AGE <= 14 THEN '10-14'
         WHEN POL_MAX_VEH_AGE <= 19 THEN '15-19'
         WHEN POL_MAX_VEH_AGE <= 24 THEN '20-24'
         WHEN POL_MAX_VEH_AGE > 24 THEN '25+' ELSE 'Other' END)) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1'
    GROUP BY KEY
),

single_vehicle AS -- Gets latest month's single vehicle code PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'POL MULTI SNGL VEH CD', POL_MULTI_SNGL_VEH_CD) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" 
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1'
    GROUP BY KEY
),

mature_driving AS -- Gets latest month's mature driving course ind PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'Mature Driving Course', POL_MATUR_DRVNG_COURS_CR_IND) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" LEFT JOIN 
    (SELECT AUTO_POL_TRANS_ID, POL_MATUR_DRVNG_COURS_CR_IND FROM "PRD_PL_DB"."APP_DCPA_AUTO_CURATE"."AUTO_POLICY" GROUP BY AUTO_POL_TRANS_ID, POL_MATUR_DRVNG_COURS_CR_IND)
    USING (AUTO_POL_TRANS_ID)
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1'
    GROUP BY KEY
),

incident_free_yr AS -- Gets latest month's incident free yr PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'Incident Free Year Count', POL_INCID_FREE_YR_CNT) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" LEFT JOIN
    (SELECT AUTO_POL_TRANS_ID, POL_INCID_FREE_YR_CNT FROM "PRD_PL_DB"."APP_DCPA_AUTO_CURATE"."AUTO_POLICY" GROUP BY AUTO_POL_TRANS_ID, POL_INCID_FREE_YR_CNT)
    USING (AUTO_POL_TRANS_ID)
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1'
    GROUP BY KEY
),

naf_acct_count AS -- Gets latest month's NAF accident PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'NAF Accident Count',(
    CASE WHEN POL_TTL_NAF_ACCID_CNT = 0 THEN '0'
         WHEN POL_TTL_NAF_ACCID_CNT = 1 THEN '1'
         WHEN POL_TTL_NAF_ACCID_CNT > 1 THEN '2+' END)) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND"
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1'
    GROUP BY KEY
),

incident_month_age AS -- Gets latest month's incident month age PIF count
(
    SELECT CONCAT(STATE_ABBR, RATE_PLAN_CD, 'Incident Month Age', POL_LATST_INCID_MO_AGE,(
    CASE WHEN POL_LATST_INCID_MO_AGE BETWEEN 1 AND 12 THEN '1-12'
         WHEN POL_LATST_INCID_MO_AGE BETWEEN 13 AND 24 THEN '13-24'
         WHEN POL_LATST_INCID_MO_AGE BETWEEN 25 AND 36 THEN '25-36'
         WHEN POL_LATST_INCID_MO_AGE BETWEEN 37 AND 48 THEN '37-48'
         WHEN POL_LATST_INCID_MO_AGE BETWEEN 49 AND 60 THEN '49-60'
         WHEN POL_LATST_INCID_MO_AGE = 61 THEN '61'
         WHEN POL_LATST_INCID_MO_AGE = 0 THEN '0' END)) as Key,
    COUNT(POLICY_ID) as PIF
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_INFORCE_MONTHEND" t1
    WHERE BUS_UNIT_ABBR = 'AARP' and LATEST_MONTH_IND = 'Y' and RATE_PLAN_CD = 'HDC1'
    GROUP BY KEY
)

SELECT * FROM
(
    SELECT * FROM ins_score         
    UNION ALL 
    SELECT * FROM hh_comp           
    UNION ALL 
    SELECT * FROM clean_dirty       
    UNION ALL
    SELECT * FROM max_age           
    UNION ALL
    SELECT * FROM min_age           
    UNION ALL
    SELECT * FROM max_minus_min     
    UNION ALL
    SELECT * FROM pcarr_year_cnt    
    UNION ALL
    SELECT * FROM acct_credit       
    UNION ALL
    SELECT * FROM coverage_pkg      
    UNION ALL
    SELECT * FROM pcarr             
    UNION ALL
    SELECT * FROM insured_age_seg   
    UNION ALL
    SELECT * FROM ppv               
    UNION ALL
    SELECT * FROM young_driver      
    UNION ALL
    SELECT * FROM garage            
    UNION ALL
    SELECT * FROM bi_per_person     
    UNION ALL
    SELECT * FROM adv_qte_days      
    UNION ALL
    SELECT * FROM min_rating_factor 
    UNION ALL
    SELECT * FROM renew_uw_tier     
    UNION ALL
    SELECT * FROM max_veh_age       
    UNION ALL
    SELECT * FROM single_vehicle    
    UNION ALL
    SELECT * FROM mature_driving    
    UNION ALL
    SELECT * FROM incident_free_yr  
    UNION ALL
    SELECT * FROM naf_acct_count    
    UNION ALL
    SELECT * FROM incident_month_age
)
ORDER BY KEY
"""

In [23]:
cs = ctx.cursor()
try:
    cs.execute(inforce_segmentation)
finally:
    cs.close()
print('Done')

Done


## 9. Grid Table Creation (DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_GRID_EXCEL)

In [24]:
grid_excel = """
CREATE OR REPLACE TRANSIENT TABLE DSC_PLBI_DB.APP_AUTO_DEV.SCORECARD_GRID_EXCEL AS

WITH response_state AS -- Gathers # of responses by state
(
    SELECT CONCAT('Response', RISK_ST_ABBR) AS Key,
    TRUNC(SUM(CASE WHEN DATEDIFF(DAY,TRANS_DT,SYSDATE()) <= 90 THEN RESP_IND_CNT ELSE 0 END), 0) AS "90days"
    FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW" 
    WHERE BU_ABBR ='AARP' AND RISK_ST_ABBR IS NOT NULL AND ORIG_QTE_CNTCT_METH_CD = 'P'
    GROUP BY Key
    UNION ALL
    
    SELECT CONCAT('Response',RISK_STATE_ABBR) AS Key,
    TRUNC(SUM(CASE WHEN DATEDIFF(DAY,QUOTE_LAST_TRANSACTION_DT,SYSDATE()) <= 90 THEN RESPONSE_IND_CNT ELSE 0 END), 0) AS "90days"
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW" 
    WHERE BUS_UNIT_ABBR = 'AARP' AND ORIG_QUOTE_PROCESS_METH_CD = 'T' 
    GROUP BY Key
),

issues_state AS -- Gathers # of issues by state
(
    SELECT CONCAT('Issues',RISK_ST_ABBR) AS Key,
    TRUNC(SUM(CASE WHEN DATEDIFF(DAY,TRANS_DT,SYSDATE())<=90 THEN ISSUE_IND_CNT ELSE 0 END),0) AS "90days",
    FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW" 
    WHERE BU_ABBR ='AARP' AND RISK_ST_ABBR IS NOT NULL AND ORIG_QTE_CNTCT_METH_CD = 'P'
    GROUP BY Key
    UNION ALL

    SELECT CONCAT('Issues',RISK_STATE_ABBR) AS Key,
    TRUNC(SUM(CASE WHEN DATEDIFF(DAY,QUOTE_LAST_TRANSACTION_DT,SYSDATE())<=90 THEN ISSUE_IND_CNT ELSE 0 END),0) AS "90days",
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW" 
    WHERE BUS_UNIT_ABBR = 'AARP' AND ORIG_QUOTE_PROCESS_METH_CD = 'T' 
    GROUP BY Key
),

response_countrywide AS -- Gathers # of responses countrywide
(
    SELECT CONCAT('Response','CW') AS Key,
    TRUNC(SUM(CASE WHEN DATEDIFF(DAY,TRANS_DT,SYSDATE())<=90 THEN RESP_IND_CNT ELSE 0 END),0) AS "90days",
    FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW" 
    WHERE BU_ABBR ='AARP' AND RISK_ST_ABBR IS NOT NULL AND ORIG_QTE_CNTCT_METH_CD = 'P'
    GROUP BY Key
    UNION ALL
    
    SELECT CONCAT('Response','CW') AS Key,
    TRUNC(SUM(CASE WHEN DATEDIFF(DAY,QUOTE_LAST_TRANSACTION_DT,SYSDATE())<=90 THEN RESPONSE_IND_CNT ELSE 0 END),0) AS "90days",
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW" 
    WHERE BUS_UNIT_ABBR = 'AARP' AND ORIG_QUOTE_PROCESS_METH_CD = 'T' 
    GROUP BY Key
),

issues_countrywide AS -- Gathers # of issues Countrywide
(
    SELECT 'IssuesCW' AS Key,
    TRUNC(SUM(CASE WHEN DATEDIFF(DAY,TRANS_DT,SYSDATE())<=90 THEN ISSUE_IND_CNT ELSE 0 END),0) AS "90days",
    FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_QUOTE_LATST_VW" 
    WHERE BU_ABBR ='AARP' AND RISK_ST_ABBR IS NOT NULL AND ORIG_QTE_CNTCT_METH_CD = 'P'
    GROUP BY Key
    UNION ALL
    
    SELECT 'IssuesCW' AS Key,
    TRUNC(SUM(CASE WHEN DATEDIFF(DAY,QUOTE_LAST_TRANSACTION_DT,SYSDATE())<=90 THEN ISSUE_IND_CNT ELSE 0 END),0) AS "90days",
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_QUOTE_FLOW_SCDI_MODEL_VW" 
    WHERE BUS_UNIT_ABBR = 'AARP' AND ORIG_QUOTE_PROCESS_METH_CD = 'T' 
    GROUP BY Key
),

e_60_state AS -- Gathers E+60 Persistency by state
(
    SELECT CONCAT('E+60 R',RISK_STATE) AS Key,
    1-(Div0null((SUM(CASE WHEN DATEDIFF(Day,POL_EFF_DT,SYSDATE())<=150 THEN CANCEL_E_60 ELSE 0 END)), 
    (SUM(CASE WHEN DATEDIFF(Day,POL_EFF_DT,SYSDATE())<=150 THEN ATR_E_60 ELSE 0 END)))) AS "90days"
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"  
    WHERE POL_NEW_RENEW_CD = 'R' AND BUS_UNIT_ABBR = 'AARP' AND LOB = 'AUTO' AND ATR_E_60 = 1
    GROUP BY Key
),

e_60_countrywide AS -- Gathers E+60 Persistency Countrywide
(
    SELECT 'E+60 RCW' AS Key,
    1-(Div0null((SUM(CASE WHEN DATEDIFF(Day,POL_EFF_DT,SYSDATE())<=150 THEN CANCEL_E_60 ELSE 0 END)), 
    (SUM(CASE WHEN DATEDIFF(Day,POL_EFF_DT,SYSDATE())<=150 THEN ATR_E_60 ELSE 0 END)))) AS "90days"
    FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."PERSISTENCY_COMBINED"
    WHERE POL_NEW_RENEW_CD = 'R' AND BUS_UNIT_ABBR = 'AARP' AND LOB = 'AUTO' AND ATR_E_60 = 1
    GROUP BY Key
)

SELECT KEY, SUM("90days") AS "90Day", TO_CHAR(SYSDATE(),'MM-DD-YYYY') AS AS_OF
FROM 
(
    SELECT * FROM response_state
    UNION ALL
    SELECT * FROM issues_state
    UNION ALL 
    SELECT * FROM response_countrywide
    UNION ALL
    SELECT * FROM issues_countrywide
    UNION ALL
    SELECT * FROM e_60_state
    UNION ALL
    SELECT * FROM e_60_countrywide
)
GROUP BY KEY
ORDER BY KEY
"""

In [25]:
cs = ctx.cursor()
try:
    cs.execute(grid_excel)
finally:
    cs.close()
ctx.close()
print('Done')

Done
